In [ ]:
# CELL 01 - Setup (NO REPO)
import os
from pathlib import Path

# Always run from a simple writable location; never cd into a repo.
if os.path.exists('/content'):
    os.chdir('/content')
RUNTIME_ROOT = Path.cwd()
DATA_ROOT = (RUNTIME_ROOT / 'cafa6_data')
DATA_ROOT.mkdir(parents=True, exist_ok=True)
print(f'CWD: {Path.cwd()}')
print(f'DATA_ROOT: {DATA_ROOT.resolve()}')

In [ ]:
# CELL 02 - Install dependencies (mandatory, early)
# Local (Windows): install ONLY missing packages to avoid long-path issues.
import importlib.util
import os
import subprocess
import sys


def _detect_kaggle() -> bool:
    return bool(
        os.environ.get("KAGGLE_KERNEL_RUN_TYPE")
        or os.environ.get("KAGGLE_URL_BASE")
        or os.environ.get("KAGGLE_DATA_PROXY_URL")
    )


def _detect_colab() -> bool:
    return bool(
        os.environ.get("COLAB_RELEASE_TAG")
        or os.environ.get("COLAB_GPU")
        or os.environ.get("COLAB_TPU_ADDR")
    )


IS_KAGGLE = _detect_kaggle()
IS_COLAB = (not IS_KAGGLE) and _detect_colab()
if IS_KAGGLE:
    print("Environment: Kaggle Detected")
elif IS_COLAB:
    print("Environment: Colab Detected")
else:
    print("Environment: Local Detected")


def _pip_install(pkgs: list[str]) -> None:
    if not pkgs:
        return
    print("+", sys.executable, "-m", "pip", "install", *pkgs)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])


# We still guarantee requirements are present by installing missing ones.
# IMPORTANT (Windows): avoid forcing heavy Jupyter installs; the notebook already runs inside a kernel.
REQUIRED = {
    # Core
    "pandas": "pandas",
    "numpy": "numpy",
    "scipy": "scipy",
    "pyarrow": "pyarrow",
    # ML
    "scikit-learn": "sklearn",
    "torch": "torch",
    "transformers": "transformers",
    "py-boost": "py_boost",
    # Bio / graph
    "biopython": "Bio",
    "obonet": "obonet",
    "networkx": "networkx",
    # Visualisation
    "matplotlib": "matplotlib",
    "seaborn": "seaborn",
    # Utils
    "tqdm": "tqdm",
    "requests": "requests",
    "urllib3": "urllib3",
    "joblib": "joblib",
    "psutil": "psutil",
    "fastparquet": "fastparquet",
    "pyyaml": "yaml",
    # Checkpointing
    "huggingface_hub": "huggingface_hub",
}

missing = [pkg for pkg, mod in REQUIRED.items() if importlib.util.find_spec(mod) is None]

if IS_KAGGLE:
    if missing:
        _pip_install(missing)
    else:
        print("Kaggle: skipping pip install (already satisfied).")
else:
    if missing:
        try:
            _pip_install(missing)
        except Exception as e:
            raise RuntimeError(f"Failed to install required packages: {missing}. Error: {e}") from e
    else:
        print("Local/Colab: skipping pip install (already satisfied).")


In [ ]:
# CELL 02b - Hugging Face auth bootstrap (token only)
# Reads from env (optionally loaded from .env). Never prints the token.

import os
from pathlib import Path


def _detect_kaggle_env() -> bool:
    return bool(
        os.environ.get("KAGGLE_KERNEL_RUN_TYPE")
        or os.environ.get("KAGGLE_URL_BASE")
        or os.environ.get("KAGGLE_DATA_PROXY_URL")
    )


def _detect_colab_env() -> bool:
    return bool(
        os.environ.get("COLAB_RELEASE_TAG")
        or os.environ.get("COLAB_GPU")
        or os.environ.get("COLAB_TPU_ADDR")
    )


IS_KAGGLE_ENV = _detect_kaggle_env()
IS_COLAB_ENV = (not IS_KAGGLE_ENV) and _detect_colab_env()


def _load_dotenv_if_present(dotenv_path: Path) -> None:
    try:
        dotenv_path = Path(dotenv_path)
        if not dotenv_path.exists():
            return
        for raw in dotenv_path.read_text(encoding="utf-8").splitlines():
            line = raw.strip()
            if not line or line.startswith("#") or "=" not in line:
                continue
            k, v = line.split("=", 1)
            k = k.strip()
            v = v.strip().strip('"').strip("'")
            if k and k not in os.environ:
                os.environ[k] = v
    except Exception:
        return


# Local convenience: allow a repo-level .env (gitignored) to populate env vars.
_cwd = Path.cwd()
_load_dotenv_if_present(_cwd / ".env")
_load_dotenv_if_present(_cwd.parent / ".env")


def _get_secret_env_first(name: str) -> str:
    # Policy:
    # - Colab: ONLY use userdata.get(name)
    # - Elsewhere: env vars (possibly loaded from .env). We do not rely on Kaggle uploads.
    v = (os.environ.get(name, "") or "").strip()
    if v:
        return v
    if IS_COLAB_ENV:
        try:
            from google.colab import userdata  # type: ignore

            return (userdata.get(name) or "").strip()
        except Exception:
            return ""
    if IS_KAGGLE_ENV:
        # Kaggle notebooks can still use Secrets to inject tokens.
        try:
            from kaggle_secrets import UserSecretsClient  # type: ignore

            return (UserSecretsClient().get_secret(name) or "").strip()
        except Exception:
            return ""
    return ""


_tok = _get_secret_env_first("HUGGINGFACE_TOKEN") or _get_secret_env_first("HF_TOKEN")
if _tok:
    # huggingface_hub uses HF_TOKEN by default
    os.environ.setdefault("HF_TOKEN", _tok)
    os.environ.setdefault("HUGGINGFACE_TOKEN", _tok)
    print("Hugging Face token: present")
else:
    print("Hugging Face token: MISSING (uploads will fail)")


In [ ]:
# CELL 02c - Hugging Face Hub sanity check (debug)
# Verifies the token can authenticate and (optionally) access the target repo.

import os

try:
    from huggingface_hub import HfApi
except Exception as e:
    raise RuntimeError("huggingface_hub not installed; run the setup cell(s) above") from e

def _get_conf(name: str) -> str:
    # Check env var first
    v = (os.environ.get(name) or "").strip()
    if v:
        return v
    # Check Colab secrets
    try:
        from google.colab import userdata
        return (userdata.get(name) or "").strip()
    except Exception:
        pass
    return ""

repo_id = _get_conf("CAFA_HF_REPO_ID")
repo_type = _get_conf("CAFA_HF_REPO_TYPE") or "dataset"

token = _get_conf("HF_TOKEN") or _get_conf("HUGGINGFACE_TOKEN")
api = HfApi(token=token or None)

if not token:
    print("HF token: MISSING (run Cell 3 / set HF_TOKEN or HUGGINGFACE_TOKEN)")
else:
    print("HF token: present")

try:
    me = api.whoami() if token else {}
    print("HF whoami:", (me.get("name") or me.get("fullname") or "<unknown>") if me else "<skipped>")
except Exception as e:
    print("HF whoami failed:", repr(e))

if repo_id:
    try:
        info = api.repo_info(repo_id=repo_id, repo_type=repo_type)
        # huggingface_hub uses different info types per repo_type; dataset returns DatasetInfo with .id
        resolved = getattr(info, "id", None) or getattr(info, "repo_id", None) or repo_id
        print("HF repo accessible:", resolved)
    except Exception as e:
        print("HF repo not accessible yet (or missing):", repr(e))
else:
    print("CAFA_HF_REPO_ID not set (skip repo check)")

In [ ]:
# CELL 03 - Solution: 1. SETUP, CONFIG & DIAGNOSTICS
# PRODUCES: manifest.json (checkpoint artefact manifest)
# 1. SETUP, CONFIG & DIAGNOSTICS
# ==========================================
# HARDWARE: CPU (Standard)
# ==========================================
import json
import os
import shutil
import subprocess
import sys
import time
import zipfile
from dataclasses import dataclass
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# ------------------------------------------
# Environment Detection & Paths
# ------------------------------------------
# Kaggle images can have `google-colab` installed; never use `import google.colab` as a signal.

def _detect_kaggle() -> bool:
    # Kaggle kernels reliably set at least one of these env vars.
    return bool(
        os.environ.get('KAGGLE_KERNEL_RUN_TYPE')
        or os.environ.get('KAGGLE_URL_BASE')
        or os.environ.get('KAGGLE_DATA_PROXY_URL')
    )

def _detect_colab() -> bool:
    # Colab sets these env vars; this avoids false positives on Kaggle.
    return bool(os.environ.get('COLAB_RELEASE_TAG') or os.environ.get('COLAB_GPU') or os.environ.get('COLAB_TPU_ADDR'))

IS_KAGGLE = _detect_kaggle()
IS_COLAB = (not IS_KAGGLE) and _detect_colab()

KAGGLE_INPUT_ROOT = Path('/kaggle/input')
if IS_KAGGLE:
    print('Environment: Kaggle Detected')
    WORKING_ROOT = Path('/kaggle/working')
    if KAGGLE_INPUT_ROOT.exists():
        for dirname, _, filenames in os.walk(str(KAGGLE_INPUT_ROOT)):
            for filename in filenames:
                print(os.path.join(dirname, filename))
elif IS_COLAB:
    print('Environment: Colab Detected')
    WORKING_ROOT = Path(os.environ.get('CAFA_WORKING_ROOT', str(Path('/content'))))
else:
    print('Environment: Local Detected')
    # No repo assumptions: treat current working dir as runtime root.
    WORKING_ROOT = Path.cwd()
# ------------------------------------------
# Local cache roots (ephemeral) + artefacts root
# ------------------------------------------
# Single source of truth for this notebook: everything lives under cafa6_data/.
# If Cell 1 ran, reuse its DATA_ROOT so we don't fork paths.
if 'DATA_ROOT' in globals():
    WORK_ROOT = Path(DATA_ROOT)
    WORKING_ROOT = WORK_ROOT.parent
else:
    WORK_ROOT = WORKING_ROOT / 'cafa6_data'
WORK_ROOT.mkdir(parents=True, exist_ok=True)
for _d in ['parsed', 'features', 'external', 'Train', 'Test']:
    (WORK_ROOT / _d).mkdir(parents=True, exist_ok=True)
# Keep caches OUT of WORK_ROOT so we never accidentally publish them.
CACHE_ROOT = WORKING_ROOT / 'cache'
CACHE_ROOT.mkdir(parents=True, exist_ok=True)
os.environ.setdefault('HF_HOME', str(CACHE_ROOT / 'hf_home'))
os.environ.setdefault('TRANSFORMERS_CACHE', str(CACHE_ROOT / 'hf_home'))
os.environ.setdefault('HF_HUB_CACHE', str(CACHE_ROOT / 'hf_hub'))
os.environ.setdefault('TORCH_HOME', str(CACHE_ROOT / 'torch_home'))
# Runtime provenance guards: never publish downloaded artefacts.
RUN_START_TS = time.time()
DOWNLOADED_PATHS: set[Path] = set()

def _mark_downloaded(p: Path) -> None:
    DOWNLOADED_PATHS.add(Path(p).resolve())
# ------------------------------------------
# Dataset Discovery (competition data)
DATASET_SLUG = 'cafa-6-protein-function-prediction'
# Required competition files (MANDATORY):
REQUIRED_COMP_FILES = [
    'IA.tsv',
    'sample_submission.tsv',
    'Train/go-basic.obo',
    'Train/train_sequences.fasta',
    'Train/train_terms.tsv',
    'Train/train_taxonomy.tsv',
    'Test/testsuperset.fasta',
    'Test/testsuperset-taxon-list.tsv',
]

def _ensure_kaggle_cli() -> None:
    try:
        subprocess.run(['kaggle', '--version'], check=True, capture_output=True, text=True, encoding='utf-8', errors='replace')
        return
    except (FileNotFoundError, subprocess.CalledProcessError):
        pass

    print("Installing kaggle...")
    subprocess.check_call([sys.executable, '-m', 'pip', '-q', 'install', 'kaggle'])
    
    # Check if it is now in PATH
    if shutil.which('kaggle'):
        return
        
    # If not, look for it in the python bin directory
    p = Path(sys.executable).parent
    candidates = [p / 'kaggle', p / 'kaggle.exe']
    for c in candidates:
        if c.exists():
            print(f"Adding {p} to PATH for kaggle")
            os.environ['PATH'] = str(p) + os.pathsep + os.environ['PATH']
            return
            
    # One more try with updated PATH
    if shutil.which('kaggle'):
        return
        
    raise RuntimeError("Kaggle CLI installed but not found in PATH. Please ensure 'kaggle' is in your PATH.")

def _get_secret(name: str) -> str:
    # Helper to get secret from env or Colab userdata
    v = os.environ.get(name)
    if v:
        return v
    if IS_COLAB:
        try:
            from google.colab import userdata
            return userdata.get(name)
        except Exception:
            pass
    return ""

def _kaggle_env(require: bool = True) -> dict[str, str]:
    env = os.environ.copy()
    # Inject secrets if missing in env
    if not env.get('KAGGLE_USERNAME'):
        k = _get_secret('KAGGLE_USERNAME')
        if k: env['KAGGLE_USERNAME'] = k
    if not env.get('KAGGLE_KEY'):
        k = _get_secret('KAGGLE_KEY')
        if k: env['KAGGLE_KEY'] = k

    if require and (not env.get('KAGGLE_USERNAME') or not env.get('KAGGLE_KEY')):
        raise RuntimeError('Missing Kaggle API auth: set KAGGLE_USERNAME and KAGGLE_KEY as secrets/env vars.')
    return env

def _download_comp_file(rel_path: str, target_root: Path) -> None:
    """Download one competition file into target_root, preserving folders."""
    rel_path = rel_path.replace('\\', '/')
    out_path = target_root / rel_path
    out_path.parent.mkdir(parents=True, exist_ok=True)
    if out_path.exists():
        return
    _ensure_kaggle_cli()
    env = _kaggle_env(require=True)
    tmp = target_root / '_tmp_download'
    tmp.mkdir(parents=True, exist_ok=True)
    cmd = ['kaggle', 'competitions', 'download', '-c', DATASET_SLUG, '-f', rel_path, '-p', str(tmp)]
    print('+', ' '.join(cmd))
    subprocess.run(cmd, check=True, env=env)
    name = Path(rel_path).name
    zip_path = tmp / f'{name}.zip'
    if zip_path.exists():
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(tmp)
        zip_path.unlink()
    cand1 = tmp / rel_path
    cand2 = tmp / name
    src = cand1 if cand1.exists() else (cand2 if cand2.exists() else None)
    if src is None:
        raise FileNotFoundError(f'Downloaded file not found after unzip: {rel_path}')
    out_path.parent.mkdir(parents=True, exist_ok=True)
    shutil.move(str(src), str(out_path))
    _mark_downloaded(out_path)

def ensure_competition_data(data_root: Path) -> Path:
    """Ensures competition data exists under cafa6_data/. Returns dataset root."""
    data_root = Path(data_root)
    # Kaggle: copy from mounted input if available.
    if IS_KAGGLE and Path('/kaggle/input').exists():
        mounted = Path('/kaggle/input') / DATASET_SLUG
        if mounted.exists():
            for rel in REQUIRED_COMP_FILES:
                dst = data_root / rel
                if dst.exists():
                    continue
                src = mounted / rel
                dst.parent.mkdir(parents=True, exist_ok=True)
                if not src.exists():
                    raise FileNotFoundError(f'Missing in mounted Kaggle input: {src}')
                shutil.copy2(src, dst)
                _mark_downloaded(dst)
            return data_root
    # Colab/Local: download file-by-file via Kaggle API.
    for rel in REQUIRED_COMP_FILES:
        _download_comp_file(rel, data_root)
    return data_root
# Canonical dataset root for the entire notebook
DATASET_ROOT = ensure_competition_data(WORK_ROOT)
# Canonical competition paths (always under WORK_ROOT/cafa6_data)
PATH_IA = WORK_ROOT / 'IA.tsv'
PATH_SAMPLE_SUB = WORK_ROOT / 'sample_submission.tsv'
PATH_GO_OBO = WORK_ROOT / 'Train' / 'go-basic.obo'
PATH_TRAIN_FASTA = WORK_ROOT / 'Train' / 'train_sequences.fasta'
PATH_TRAIN_TERMS = WORK_ROOT / 'Train' / 'train_terms.tsv'
PATH_TRAIN_TAXON = WORK_ROOT / 'Train' / 'train_taxonomy.tsv'
PATH_TEST_FASTA = WORK_ROOT / 'Test' / 'testsuperset.fasta'
PATH_TEST_TAXON = WORK_ROOT / 'Test' / 'testsuperset-taxon-list.tsv'
print('DATASET_ROOT:', DATASET_ROOT.resolve())
# Checkpoint store (Hugging Face Hub = single source of truth)
# ------------------------------------------

from huggingface_hub import CommitOperationAdd, HfApi, hf_hub_download

# No Kaggle publishing. HF repo is the only remote artefact store.
HF_REPO_ID = (_get_secret('CAFA_HF_REPO_ID') or '').strip()
HF_REPO_TYPE = (_get_secret('CAFA_HF_REPO_TYPE') or 'dataset').strip() or 'dataset'

# Upload-after-write default: ON if CAFA_HF_REPO_ID is set.
HF_PUSH = str(os.environ.get('CAFA_HF_PUSH', '') or os.environ.get('CAFA_CHECKPOINT_PUSH', '') or ('1' if HF_REPO_ID else '0')).strip().lower() in {'1', 'true', 'yes'}
HF_PULL = str(os.environ.get('CAFA_HF_PULL', '1')).strip().lower() in {'1', 'true', 'yes'}
HF_CREATE_REPO = str(os.environ.get('CAFA_HF_CREATE_REPO', '0')).strip().lower() in {'1', 'true', 'yes'}

# Keep behaviour for "push existing artefacts" knobs used later in the notebook.
PUSH_EXISTING_CHECKPOINTS = str(os.environ.get('CAFA_CHECKPOINT_PUSH_EXISTING', '0')).strip().lower() in {'1', 'true', 'yes'}

MANIFEST_PATH = WORK_ROOT / 'manifest.json'

def _load_manifest() -> dict:
    if MANIFEST_PATH.exists():
        try:
            return json.loads(MANIFEST_PATH.read_text(encoding='utf-8'))
        except Exception:
            return {}
    return {}

def _update_manifest(stage: str, required_paths: list[Path], note: str = '') -> None:
    m = _load_manifest()
    stages = m.get('stages', {})
    files = []
    for p in required_paths:
        p = Path(p)
        rel = str(p.relative_to(WORK_ROOT)) if str(p).startswith(str(WORK_ROOT)) else str(p)
        files.append({'path': rel, 'bytes': int(p.stat().st_size) if p.exists() else None})
    stages[stage] = {
        'ts_utc': time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime()),
        'note': note,
        'files': files,
    }
    m['stages'] = stages
    MANIFEST_PATH.write_text(json.dumps(m, indent=2), encoding='utf-8')

def _stage_files_signature(required_paths: list[Path]) -> list[dict]:
    sig = []
    for p in required_paths:
        p = Path(p)
        rel = str(p.relative_to(WORK_ROOT)) if str(p).startswith(str(WORK_ROOT)) else str(p)
        sig.append({'path': rel, 'bytes': int(p.stat().st_size) if p.exists() else None})
    return sorted(sig, key=lambda x: x['path'])

def _hf_token_present() -> bool:
    return bool((_get_secret('HF_TOKEN') or _get_secret('HUGGINGFACE_TOKEN') or '').strip())

@dataclass
class HfCheckpointStore:
    work_root: Path
    repo_id: str
    repo_type: str
    pull_enabled: bool
    push_enabled: bool
    create_repo: bool
    manifest_name: str = 'manifest.json'

    def _api(self) -> HfApi:
        return HfApi(token=(_get_secret('HF_TOKEN') or None))

    def _ensure_repo(self) -> None:
        if not self.repo_id:
            raise ValueError('Missing CAFA_HF_REPO_ID=<user>/<repo>; cannot use HF checkpointing.')
        if not self.create_repo:
            return
        try:
            self._api().create_repo(repo_id=self.repo_id, repo_type=self.repo_type, exist_ok=True)
        except Exception:
            # If repo already exists or token lacks create permission, we just proceed.
            pass

    def pull(self, required_files: list[str] | None = None) -> None:
        if not self.pull_enabled:
            return
        required_files = list(required_files or [])
        if not required_files:
            return
        if not self.repo_id:
            print('HF pull skipped: CAFA_HF_REPO_ID not set.')
            return

        missing_locally = [f for f in required_files if not (self.work_root / f).exists()]
        if not missing_locally:
            print(f'All {len(required_files)} required artefacts present locally. Skipping pull.')
            return

        print(f'Pulling {len(missing_locally)} checkpoint artefacts from HF: {self.repo_id} ({self.repo_type})')
        for rel in missing_locally:
            rel = str(rel).replace('\\', '/').lstrip('/')
            dst = self.work_root / rel
            dst.parent.mkdir(parents=True, exist_ok=True)
            try:
                cached = hf_hub_download(
                    repo_id=self.repo_id,
                    repo_type=self.repo_type,
                    filename=rel,
                    token=(_get_secret('HF_TOKEN') or None),
                )
            except Exception as e:
                print(f'WARNING: missing on HF (or not accessible): {rel} ({e})')
                continue
            shutil.copy2(cached, dst)
            _mark_downloaded(dst)
        print('HF checkpoint pull complete.')

    def maybe_push(self, stage: str, required_paths: list[Path], note: str = '') -> bool:
        if not self.push_enabled:
            print(f'[CHECKPOINT] {stage}: push disabled (set CAFA_HF_PUSH=1 to enable).')
            return False
        if not self.repo_id:
            print(f'[CHECKPOINT] {stage}: repo not set (CAFA_HF_REPO_ID missing).')
            return False
        if not _hf_token_present():
            print(f'[CHECKPOINT] {stage}: token missing (set HF_TOKEN/HUGGINGFACE_TOKEN).')
            return False
        return self.push(stage, required_paths, note=note)

    def push(self, stage: str, required_paths: list[Path], note: str = '') -> bool:
        if not self.push_enabled:
            return False
        if not self.repo_id:
            raise ValueError('Missing CAFA_HF_REPO_ID=<user>/<repo>; cannot checkpoint.')
        if not _hf_token_present():
            raise RuntimeError('HF token missing (set HF_TOKEN/HUGGINGFACE_TOKEN).')

        missing = [Path(p) for p in required_paths if not Path(p).exists()]
        if missing:
            print(f'[CHECKPOINT] {stage}: missing artefacts; skipping push: {missing}')
            return False
        # Never publish pulled/downloaded artefacts (non-negotiable rule).
        downloaded = []
        try:
            downloaded = [Path(p).resolve() for p in required_paths if Path(p).resolve() in DOWNLOADED_PATHS]
        except Exception:
            downloaded = []
        if downloaded:
            print(f'[CHECKPOINT] {stage}: refusing to publish pulled/downloaded artefacts ({len(downloaded)} files).')
            return False
        # Default: do not push existing artefacts on reruns unless explicitly enabled.
        if not bool(PUSH_EXISTING_CHECKPOINTS):
            try:
                fresh = [Path(p) for p in required_paths if Path(p).stat().st_mtime >= (RUN_START_TS - 2.0)]
            except Exception:
                fresh = []
            if not fresh:
                print(f'[CHECKPOINT] {stage}: nothing freshly built in this runtime; skipping push (set CAFA_CHECKPOINT_PUSH_EXISTING=1 to override).')
                return False

        # Optional skip: identical stage signature
        m = _load_manifest()
        existing = (m.get('stages', {}) or {}).get(stage) if isinstance(m, dict) else None
        if isinstance(existing, dict):
            prev_files = existing.get('files', [])
            if isinstance(prev_files, list):
                prev_sig = sorted([{'path': f.get('path'), 'bytes': f.get('bytes')} for f in prev_files if isinstance(f, dict)], key=lambda x: str(x.get('path')))
                cur_sig = _stage_files_signature(required_paths)
                if prev_sig == cur_sig:
                    print(f'[CHECKPOINT] {stage}: unchanged; skipping push')
                    return False

        _update_manifest(stage, required_paths, note=note)
        print(f'[CHECKPOINT] {stage}: publishing {len(required_paths)} files to HF repo {self.repo_id} ({self.repo_type})')

        work_root_resolved = self.work_root.resolve()
        ops: list[CommitOperationAdd] = []
        for p in required_paths:
            p = Path(p).resolve()
            try:
                rel = p.relative_to(work_root_resolved).as_posix()
            except Exception:
                raise ValueError(f'All checkpoint artefacts must live under WORK_ROOT. Got: {p}')
            ops.append(CommitOperationAdd(path_in_repo=rel, path_or_fileobj=str(p)))

        if MANIFEST_PATH.exists():
            ops.append(CommitOperationAdd(path_in_repo=self.manifest_name, path_or_fileobj=str(MANIFEST_PATH)))

        msg = f'{stage}: {note}'.strip() if note else stage
        self._ensure_repo()
        self._api().create_commit(repo_id=self.repo_id, repo_type=self.repo_type, operations=ops, commit_message=msg)
        print('[CHECKPOINT] published:', msg)
        return True

print('HF checkpoint config:')
print('  CAFA_HF_REPO_ID:', HF_REPO_ID or 'MISSING')
print('  HF_PUSH:', HF_PUSH, 'HF_PULL:', HF_PULL, 'PUSH_EXISTING_CHECKPOINTS:', PUSH_EXISTING_CHECKPOINTS)

# Checkpoint artefacts required for this notebook (MANDATORY, pulled file-by-file)
# We use stage-based required sets so you can resume from a specific milestone.
# Select via CAFA_CHECKPOINT_PULL_STAGE.
#
# Stages are cumulative by design (i.e. later stages include their prerequisites).
# This avoids half-hydrated states when you resume from later cells.
has_test = bool(PATH_TEST_FASTA.exists())
STAGE_REQUIRED_FILES: dict[str, list[str]] = {}
# Stage 01: parsed core
STAGE_REQUIRED_FILES['stage_01_parsed'] = [
    'parsed/train_seq.feather',
    'parsed/train_terms.parquet',
    'parsed/train_taxa.feather',
    'parsed/term_counts.parquet',
    'parsed/term_priors.parquet',
] + ([
    'parsed/test_seq.feather',
    'parsed/test_taxa.feather',
] if has_test else [])
# Stage 02: external text corpus
STAGE_REQUIRED_FILES['stage_02_external_text'] = STAGE_REQUIRED_FILES['stage_01_parsed'] + [
    'external/entryid_text.tsv',
]
# Stage 03: TF-IDF text embeddings
STAGE_REQUIRED_FILES['stage_03_tfidf_text'] = STAGE_REQUIRED_FILES['stage_02_external_text'] + [
    'features/text_vectorizer.joblib',
    'features/train_embeds_text.npy',
    'features/test_embeds_text.npy',
]
# Stage 04: external propagated GOA priors (IEA)
STAGE_REQUIRED_FILES['stage_04_external_goa_priors'] = STAGE_REQUIRED_FILES['stage_03_tfidf_text'] + [
    'external/goa_filtered_iea.tsv.gz',
    'external/prop_train_no_kaggle.tsv.gz',
    'external/prop_test_no_kaggle.tsv.gz',
]
# Stage 06: sequence embeddings (core modalities)
STAGE_REQUIRED_FILES['stage_06_embeddings_core'] = STAGE_REQUIRED_FILES['stage_04_external_goa_priors'] + [
    'features/train_embeds_t5.npy',
    'features/test_embeds_t5.npy',
    'features/train_embeds_esm2.npy',
    'features/test_embeds_esm2.npy',
    'features/train_embeds_esm2_3b.npy',
    'features/test_embeds_esm2_3b.npy',
    'features/train_embeds_ankh.npy',
    'features/test_embeds_ankh.npy',
]
# Stage 07: Level-1 OOF + test predictions (enables going straight to stacker)
STAGE_REQUIRED_FILES['stage_07_level1_preds'] = STAGE_REQUIRED_FILES['stage_06_embeddings_core'] + [
    'features/top_terms_13500.json',
    'features/level1_preds/oof_pred_logreg.npy',
    'features/level1_preds/test_pred_logreg.npy',
    'features/level1_preds/oof_pred_gbdt.npy',
    'features/level1_preds/test_pred_gbdt.npy',
    'features/level1_preds/oof_pred_dnn.npy',
    'features/level1_preds/test_pred_dnn.npy',
    'features/level1_preds/oof_pred_knn.npy',
    'features/level1_preds/test_pred_knn.npy',
]
# Stage 08: hierarchy-aware stacker output (enables going straight to submission)
STAGE_REQUIRED_FILES['stage_08_stacker_gcn'] = STAGE_REQUIRED_FILES['stage_07_level1_preds'] + [
    'features/test_pred_gcn.npy',
]
# Stage 09: final submission (rarely needed for pull, but included for completeness)
STAGE_REQUIRED_FILES['stage_09_submission'] = STAGE_REQUIRED_FILES['stage_08_stacker_gcn'] + [
    'features/test_pred_gcn.npy',
    'submission.tsv',
]
def _select_required_checkpoint_files() -> list[str]:
    stage = os.getenv('CAFA_CHECKPOINT_PULL_STAGE', '').strip()
    if not stage:
        # Default to pulling everything (all stages) so we have a full set of artefacts.
        stage = 'stage_09_submission'
    if stage.lower() in {'off', 'none', 'no', '0'}:
        return []
    if stage not in STAGE_REQUIRED_FILES:
        raise ValueError(f'Unknown CAFA_CHECKPOINT_PULL_STAGE={stage!r}. Known: {sorted(STAGE_REQUIRED_FILES)}')
    return list(STAGE_REQUIRED_FILES[stage])
REQUIRED_CHECKPOINT_FILES = _select_required_checkpoint_files()
STORE = HfCheckpointStore(
    work_root=WORK_ROOT,
    repo_id=HF_REPO_ID,
    repo_type=HF_REPO_TYPE,
    pull_enabled=HF_PULL,
    push_enabled=HF_PUSH,
    create_repo=HF_CREATE_REPO,
)
# Pull once at startup (fresh runtimes resume here)
STORE.pull(required_files=REQUIRED_CHECKPOINT_FILES)
# Post-pull diagnostics: make it obvious whether artefacts are present.

def _p(path: Path) -> str:
    return str(path)

def _exists_bytes(path: Path) -> str:
    if not path.exists():
        return 'MISSING'
    try:
        if path.is_dir():
            # Fast recursive size
            total = sum(f.stat().st_size for f in path.rglob('*') if f.is_file())
            return f'OK (DIR, {total / (1024**2):.1f} MB)'
        return f'OK ({path.stat().st_size / (1024**2):.1f} MB)'
    except Exception:
        return 'OK'
print('Checkpoint status (after pull):')
print('  WORK_ROOT:', _p(WORK_ROOT))
print('  parsed/:', _exists_bytes(WORK_ROOT / 'parsed'))
print('  external/:', _exists_bytes(WORK_ROOT / 'external'))
print('  features/:', _exists_bytes(WORK_ROOT / 'features'))
print('  external/entryid_text.tsv:', _exists_bytes(WORK_ROOT / 'external' / 'entryid_text.tsv'))
print('  parsed/train_seq.feather:', _exists_bytes(WORK_ROOT / 'parsed' / 'train_seq.feather'))

def stage_present(required_paths: list[Path]) -> bool:
    return all(Path(p).exists() for p in required_paths)
# ------------------------------------------
# Initial Diagnostics (Sequence Lengths)
# ------------------------------------------
%matplotlib inline
plt.rcParams.update({'font.size': 10})

def read_fasta_lengths(path: Path, max_records=20000):
    lengths = []
    current = 0
    n = 0
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if n > 0:
                    lengths.append(current)
                n += 1
                current = 0
                if max_records and n > max_records:
                    break
            else:
                current += len(line)
        if n > 0:
            lengths.append(current)
    return np.array(lengths)
plt.figure(figsize=(10, 3))
plt.hist(read_fasta_lengths(PATH_TRAIN_FASTA), bins=50, alpha=0.5, label='Train')
if PATH_TEST_FASTA.exists():
    plt.hist(read_fasta_lengths(PATH_TEST_FASTA), bins=50, alpha=0.5, label='Test')
plt.title('Sequence Length Distribution (First 20k)')
plt.legend()
plt.show()


In [ ]:
# CELL 04 - Solution: 2. PHASE 1: DATA STRUCTURING & HIERARCHY
# PRODUCES: parsed/term_counts.parquet, parsed/term_priors.parquet
# 2. PHASE 1: DATA STRUCTURING & HIERARCHY
# ==========================================
# HARDWARE: CPU (Standard)
# ==========================================
# ------------------------------------------
# B. Parse OBO & Terms (needed in-memory downstream)
# ------------------------------------------

def parse_obo(path: Path):
    parents = {}
    namespaces = {}
    cur_id, cur_ns = None, None
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == '[Term]':
                if cur_id and cur_ns:
                    namespaces[cur_id] = cur_ns
                cur_id, cur_ns = None, None
            elif line.startswith('id: GO:'):
                cur_id = line.split('id: ', 1)[1]
            elif line.startswith('namespace:'):
                cur_ns = line.split('namespace: ', 1)[1]
            elif line.startswith('is_a:') and cur_id:
                parent = line.split('is_a: ', 1)[1].split(' ! ')[0]
                parents.setdefault(cur_id, set()).add(parent)
        if cur_id and cur_ns:
            namespaces[cur_id] = cur_ns
    return parents, namespaces
print("Parsing OBO...")
go_parents, go_namespaces = parse_obo(PATH_GO_OBO)
print(f"GO Graph: {len(go_parents)} nodes with parents, {len(go_namespaces)} terms with namespace.")
# ------------------------------------------
# Milestone checkpoint: stage_01_parsed
# ------------------------------------------
parsed_dir = WORK_ROOT / 'parsed'
parsed_dir.mkdir(parents=True, exist_ok=True)
out_train_seq = parsed_dir / 'train_seq.feather'
out_test_seq = parsed_dir / 'test_seq.feather'
out_train_terms = parsed_dir / 'train_terms.parquet'
out_term_counts = parsed_dir / 'term_counts.parquet'
out_term_priors = parsed_dir / 'term_priors.parquet'
out_train_taxa = parsed_dir / 'train_taxa.feather'
out_test_taxa = parsed_dir / 'test_taxa.feather'
expected = [out_train_seq, out_train_terms, out_term_counts, out_term_priors, out_train_taxa]
if PATH_TEST_FASTA.exists():
    expected += [out_test_seq, out_test_taxa]
missing = [p for p in expected if not p.exists()]
if not missing:
    print("Parsed artefacts already exist; skipping Phase 1 writes.")
else:
    # ------------------------------------------
    # A. Parse FASTA to Feather
    # ------------------------------------------
    def parse_fasta(path: Path) -> pd.DataFrame:
        ids, seqs = [], []
        cur_id, cur_seq = None, []
        with path.open('r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    if cur_id:
                        ids.append(cur_id)
                        seqs.append(''.join(cur_seq))
                    cur_id = line[1:].split()[0]
                    cur_seq = []
                else:
                    cur_seq.append(line)
            if cur_id:
                ids.append(cur_id)
                seqs.append(''.join(cur_seq))
        return pd.DataFrame({'id': ids, 'sequence': seqs})
    print("Parsing FASTA...")
    parse_fasta(PATH_TRAIN_FASTA).to_feather(out_train_seq)
    if PATH_TEST_FASTA.exists():
        parse_fasta(PATH_TEST_FASTA).to_feather(out_test_seq)
    print("FASTA parsed and saved to artefacts.")
    # ------------------------------------------
    # C. Process Terms & Priors
    # ------------------------------------------
    terms = pd.read_csv(PATH_TRAIN_TERMS, sep='\t')
    col_term = terms.columns[1]
    terms['aspect'] = terms[col_term].map(lambda x: go_namespaces.get(x, 'UNK'))
    # Plot Aspects
    plt.figure(figsize=(6, 3))
    terms['aspect'].value_counts().plot(kind='bar', title='Annotations by Namespace')
    plt.show()
    # Save Priors
    priors = (terms[col_term].value_counts() / terms.iloc[:, 0].nunique()).reset_index()
    priors.columns = ['term', 'prior']
    if PATH_IA.exists():
        ia = pd.read_csv(PATH_IA, sep='\t', names=['term', 'ia'])
        priors = priors.merge(ia, on='term', how='left').fillna(0)
    priors.to_parquet(out_term_priors)
    print("Terms processed and priors saved.")
    # ------------------------------------------
    # D. Process Taxonomy
    # ------------------------------------------
    print("Processing Taxonomy...")
    # Train Taxonomy
    tax_train = pd.read_csv(PATH_TRAIN_TAXON, sep='\t', header=None, names=['id', 'taxon_id'])
    tax_train['taxon_id'] = tax_train['taxon_id'].astype(int)
    tax_train.to_feather(out_train_taxa)
    # Test Taxonomy (Extract from FASTA headers)
    if PATH_TEST_FASTA.exists():
        ids, taxons = [], []
        with PATH_TEST_FASTA.open('r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    parts = line[1:].split()
                    ids.append(parts[0])
                    # Assume second part is taxon if present
                    if len(parts) > 1:
                        try:
                            taxons.append(int(parts[1]))
                        except ValueError:
                            taxons.append(0)
                    else:
                        taxons.append(0)
        tax_test = pd.DataFrame({'id': ids, 'taxon_id': taxons})
        tax_test.to_feather(out_test_taxa)
        print(f"Taxonomy processed. Train: {len(tax_train)}, Test: {len(tax_test)}")
    else:
        print(f"Taxonomy processed. Train: {len(tax_train)}")
    # ------------------------------------------
    # E. Save Targets & Term List
    # ------------------------------------------
    print("Saving Targets & Term List...")
    # Save full terms list (long format)
    terms.to_parquet(out_train_terms)
    # Save unique term list with counts
    term_counts = terms['term'].value_counts().reset_index()
    term_counts.columns = ['term', 'count']
    term_counts.to_parquet(out_term_counts)
    print("Targets saved.")
    if 'STORE' in globals() and STORE is not None:
        STORE.maybe_push('stage_01_parsed', [p for p in expected if p.exists()], note='parsed FASTA/taxa/terms/priors')

In [ ]:
# CELL 05 - Inline: EntryID->text corpus builder
import argparse
import csv
import os
import re
import time
import xml.etree.ElementTree as ET
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
import requests
from tqdm import tqdm

def _norm_uniprot_accession(raw_id: str) -> str:
    s = str(raw_id).strip()
    parts = s.split("|")
    # Common UniProt FASTA header: sp|P12345|NAME_HUMAN
    if len(parts) >= 2 and parts[0] in {"sp", "tr"}:
        return parts[1]
    if len(parts) >= 3:
        return parts[1]
    return s
def _detect_id_col(df: pd.DataFrame) -> str:
    for cand in ["id", "EntryID", "entry_id", "accession"]:
        if cand in df.columns:
            return cand
    return df.columns[0]
def _read_ids(feather_path: Path) -> list[str]:
    df = pd.read_feather(feather_path)
    col = _detect_id_col(df)
    return df[col].astype(str).tolist()
def _strip_go_leakage(text: str) -> str:
    # Remove explicit GO identifiers and obvious "GO:" tokens.
    text = re.sub(r"\bGO:\d{7}\b", " ", text)
    text = text.replace("GO:", " ")
    return re.sub(r"\s+", " ", text).strip()
def _make_session(total_retries: int = 5) -> requests.Session:
    from requests.adapters import HTTPAdapter
    from urllib3.util.retry import Retry
    retry = Retry(
        total=total_retries,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        raise_on_status=False,
    )
    s = requests.Session()
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s
@dataclass

class UniProtRow:
    accession: str
    protein_name: str
    organism_name: str
    keywords: str
    protein_families: str
    cc_function: str
    cc_subcellular_location: str
    pubmed_ids: list[str]
def fetch_uniprot_tsv(
    session: requests.Session,
    accessions: list[str],
    sleep_s: float,
    fields: str,
) -> dict[str, UniProtRow]:
    """Fetch UniProt rows for a small batch of accessions using the search endpoint (TSV)."""
    # Query is a disjunction of accessions.
    # Keep the batch modest to avoid URL length limits.
    query = " OR ".join([f"accession:{a}" for a in accessions])
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": f"({query})",
        "format": "tsv",
        "fields": fields,
        "size": 500,
    }
    resp = session.get(url, params=params, timeout=60)
    # UniProt will return HTTP 400 for overly long/complex queries (often URL-length related).
    # Fall back to a recursive split so large runs complete reliably.
    if resp.status_code in {400, 414} and len(accessions) > 1:
        mid = len(accessions) // 2
        left = fetch_uniprot_tsv(session, accessions[:mid], sleep_s=sleep_s, fields=fields)
        right = fetch_uniprot_tsv(session, accessions[mid:], sleep_s=sleep_s, fields=fields)
        left.update(right)
        return left
    try:
        resp.raise_for_status()
    except requests.HTTPError as e:
        detail = (resp.text or '')[:1000] if resp is not None else ''
        raise requests.HTTPError(f'{e} :: UniProt response: {detail}') from e
    if sleep_s:
        time.sleep(sleep_s)
    lines = resp.text.splitlines()
    if not lines:
        return {}
    reader = csv.DictReader(lines, delimiter="\t")
    out: dict[str, UniProtRow] = {}
    for row in reader:
        acc = (row.get("Entry") or row.get("accession") or "").strip()
        if not acc:
            continue
        def _get(k: str) -> str:
            v = row.get(k)
            return str(v).strip() if v is not None else ""
        pmids_raw = (_get("PubMed ID") or _get("lit_pubmed_id")).strip()
        pmids: list[str] = []
        if pmids_raw:
            pmids = [p.strip() for p in re.split(r"[;\s]+", pmids_raw) if p.strip().isdigit()]
        out[acc] = UniProtRow(
            accession=acc,
            protein_name=_get("Protein names"),
            organism_name=_get("Organism"),
            keywords=_get("Keywords"),
            protein_families=_get("Protein families"),
            cc_function=_get("Function [CC]"),
            cc_subcellular_location=_get("Subcellular location [CC]"),
            pubmed_ids=pmids,
        )
    return out
def fetch_pubmed_abstracts(
    session: requests.Session,
    pmids: list[str],
    sleep_s: float,
    email: str,
    api_key: str,
    _depth: int = 0,
) -> dict[str, tuple[str, str]]:
    """Fetch PubMed title+abstract via NCBI E-utilities (XML)."""
    if not pmids:
        return {}
    pmids = [str(p).strip() for p in pmids if str(p).strip().isdigit()]
    if not pmids:
        return {}
    pmids = list(dict.fromkeys(pmids))
    def _looks_like_html(text: str) -> bool:
        t = (text or "").lstrip().lower()
        return t.startswith("<!doctype html") or t.startswith("<html") or "<html" in t[:2000]
    def _sanitize_xml(text: str) -> str:
        # Remove illegal XML control chars (keep \t \n \r).
        return re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", " ", text or "")
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pubmed",
        "id": ",".join(pmids),
        "retmode": "xml",
    }
    if email:
        params["email"] = email
    if api_key:
        params["api_key"] = api_key
    root = None
    for attempt in range(3):
        try:
            resp = session.get(url, params=params, timeout=60)
            if resp.status_code in {429, 500, 502, 503, 504}:
                raise requests.HTTPError(f"PubMed transient HTTP {resp.status_code}")
            resp.raise_for_status()
            if sleep_s:
                time.sleep(sleep_s)
            text = resp.text or ""
            if _looks_like_html(text):
                snippet = text[:300].replace("\n", " " )
                raise ValueError(f"PubMed returned HTML instead of XML: {snippet}")
            text = _sanitize_xml(text)
            root = ET.fromstring(text)
            break
        except Exception:
            if attempt < 2:
                time.sleep(1.5 * (attempt + 1))
            continue
    if root is None:
        if len(pmids) > 1 and _depth < 10:
            mid = len(pmids) // 2
            left = fetch_pubmed_abstracts(session, pmids[:mid], sleep_s=sleep_s, email=email, api_key=api_key, _depth=_depth + 1)
            right = fetch_pubmed_abstracts(session, pmids[mid:], sleep_s=sleep_s, email=email, api_key=api_key, _depth=_depth + 1)
            left.update(right)
            return left
        return {}
    out: dict[str, tuple[str, str]] = {}
    for article in root.findall(".//PubmedArticle"):
        pmid_el = article.find(".//PMID")
        if pmid_el is None or pmid_el.text is None:
            continue
        pmid = pmid_el.text.strip()
        title_el = article.find(".//ArticleTitle")
        abs_el = article.find(".//Abstract")
        title = ("" if title_el is None or title_el.text is None else title_el.text.strip())
        abstract = ""
        if abs_el is not None:
            parts = []
            for t in abs_el.findall(".//AbstractText"):
                if t.text:
                    parts.append(t.text.strip())
            abstract = " ".join(parts).strip()
        out[pmid] = (title, abstract)
    return out
def main() -> int:
    p = argparse.ArgumentParser(description="Build EntryID->text corpus from UniProt (fields) + PubMed abstracts for --mode text.")
    p.add_argument("--artefacts-dir", default=Path("artefacts_local") / "artefacts", type=Path, help="Artefacts root")
    p.add_argument("--out-path", default=Path("artefacts_local") / "artefacts" / "external" / "entryid_text.tsv", type=Path, help="Output TSV path (default: artefacts_local/artefacts/external/entryid_text.tsv)")
    p.add_argument("--cache-dir", default=Path("artefacts_local") / "artefacts" / "external" / "uniprot_pubmed_cache", type=Path, help="Cache dir for UniProt+PubMed lookups")
    p.add_argument("--max-ids", type=int, default=0, help="If >0, cap number of proteins (debug)")
    p.add_argument("--uniprot-batch-size", type=int, default=80)
    p.add_argument("--pubmed-batch-size", type=int, default=50)
    p.add_argument("--max-pubmed-per-protein", type=int, default=3)
    p.add_argument("--max-abstract-chars", type=int, default=2000)
    p.add_argument("--sleep-uniprot", type=float, default=0.1)
    p.add_argument("--sleep-pubmed", type=float, default=0.34)
    p.add_argument("--email", type=str, default=os.environ.get("NCBI_EMAIL", ""))
    p.add_argument("--api-key", type=str, default=os.environ.get("NCBI_API_KEY", ""))
    p.add_argument("--strip-go", action="store_true")
    args = p.parse_args()
    args.artefacts_dir = Path(args.artefacts_dir)
    args.out_path = Path(args.out_path)
    args.cache_dir = Path(args.cache_dir)
    args.out_path.parent.mkdir(parents=True, exist_ok=True)
    args.cache_dir.mkdir(parents=True, exist_ok=True)
    train_feather = args.artefacts_dir / "parsed" / "train_seq.feather"
    test_feather = args.artefacts_dir / "parsed" / "test_seq.feather"
    ids = []
    if train_feather.exists():
        ids.extend(_read_ids(train_feather))
    if test_feather.exists():
        ids.extend(_read_ids(test_feather))
    ids = [_norm_uniprot_accession(i) for i in ids]
    uniq = sorted(set(ids))
    if args.max_ids and args.max_ids > 0:
        uniq = uniq[: int(args.max_ids)]
    print(f"Unique accessions: {len(uniq)}")
    fields = ",".join(
        [
            "accession",
            "protein_name",
            "organism_name",
            "keyword",
            "protein_families",
            "cc_function",
            "cc_subcellular_location",
            "lit_pubmed_id",
        ]
    )
    session = _make_session()
    # UniProt rows cache
    uniprot_cache = args.cache_dir / "uniprot_rows.tsv"
    uniprot_rows: dict[str, UniProtRow] = {}
    if uniprot_cache.exists():
        print(f"Loading UniProt cache: {uniprot_cache}")
        df_u = pd.read_csv(uniprot_cache, sep="\t", dtype=str)
        for _, r in df_u.iterrows():
            acc = str(r["accession"])
            pmids = [p for p in str(r.get("pubmed_ids", "")).split(";") if p]
            uniprot_rows[acc] = UniProtRow(
                accession=acc,
                protein_name=str(r.get("protein_name", "")) if r.get("protein_name") is not None else "",
                organism_name=str(r.get("organism_name", "")) if r.get("organism_name") is not None else "",
                keywords=str(r.get("keywords", "")) if r.get("keywords") is not None else "",
                protein_families=str(r.get("protein_families", "")) if r.get("protein_families") is not None else "",
                cc_function=str(r.get("cc_function", "")) if r.get("cc_function") is not None else "",
                cc_subcellular_location=str(r.get("cc_subcellular_location", "")) if r.get("cc_subcellular_location") is not None else "",
                pubmed_ids=pmids,
            )
    missing = [a for a in uniq if a not in uniprot_rows]
    if missing:
        print(f"Fetching UniProt rows (missing={len(missing)})...")
        write_header = not uniprot_cache.exists()
        with uniprot_cache.open("a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(
                f,
                delimiter="\t",
                fieldnames=[
                    "accession",
                    "protein_name",
                    "organism_name",
                    "keywords",
                    "protein_families",
                    "cc_function",
                    "cc_subcellular_location",
                    "pubmed_ids",
                ],
            )
            if write_header:
                writer.writeheader()
            for i in tqdm(range(0, len(missing), args.uniprot_batch_size), desc="UniProt"):
                batch = missing[i : i + args.uniprot_batch_size]
                got = fetch_uniprot_tsv(
                    session=session,
                    accessions=batch,
                    sleep_s=args.sleep_uniprot,
                    fields=fields,
                )
                for acc, r in got.items():
                    uniprot_rows[acc] = r
                    writer.writerow(
                        {
                            "accession": r.accession,
                            "protein_name": r.protein_name,
                            "organism_name": r.organism_name,
                            "keywords": r.keywords,
                            "protein_families": r.protein_families,
                            "cc_function": r.cc_function,
                            "cc_subcellular_location": r.cc_subcellular_location,
                            "pubmed_ids": ";".join(r.pubmed_ids),
                        }
                    )
    # PubMed cache
    pubmed_cache = args.cache_dir / "pubmed_abstracts.tsv"
    pmid_sets: dict[str, list[str]] = {}
    for acc, r in uniprot_rows.items():
        pmids = [p for p in r.pubmed_ids if p]
        if args.max_pubmed_per_protein and len(pmids) > args.max_pubmed_per_protein:
            pmids = pmids[: int(args.max_pubmed_per_protein)]
        pmid_sets[acc] = pmids
    pmids_uniq = sorted({p for ps in pmid_sets.values() for p in ps})
    pubmed_map: dict[str, tuple[str, str]] = {}
    if pubmed_cache.exists():
        print(f"Loading PubMed cache: {pubmed_cache}")
        df_p = pd.read_csv(pubmed_cache, sep="\t", dtype=str)
        for _, r in df_p.iterrows():
            pmid = str(r["pmid"])
            pubmed_map[pmid] = (
                str(r.get("title", "")) if r.get("title") is not None else "",
                str(r.get("abstract", "")) if r.get("abstract") is not None else "",
            )
    missing_pmids = [p for p in pmids_uniq if p not in pubmed_map]
    if missing_pmids:
        print(f"Fetching PubMed abstracts (missing={len(missing_pmids)})...")
        write_header = not pubmed_cache.exists()
        with pubmed_cache.open("a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, delimiter="\t", fieldnames=["pmid", "title", "abstract"])
            if write_header:
                writer.writeheader()
            for i in tqdm(range(0, len(missing_pmids), args.pubmed_batch_size), desc="PubMed"):
                batch = missing_pmids[i : i + args.pubmed_batch_size]
                got = fetch_pubmed_abstracts(
                    session=session,
                    pmids=batch,
                    sleep_s=args.sleep_pubmed,
                    email=args.email,
                    api_key=args.api_key,
                )
                for pmid, (title, abstract) in got.items():
                    if args.max_abstract_chars and abstract:
                        abstract = abstract[: int(args.max_abstract_chars)]
                    pubmed_map[pmid] = (title, abstract)
                    writer.writerow({"pmid": pmid, "title": title, "abstract": abstract})
    # Build final EntryID -> text
    print("Writing entryid_text.tsv...")
    with args.out_path.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f, delimiter="\t")
        w.writerow(["EntryID", "text"])
        n_with_uniprot = 0
        n_with_pubmed = 0
        for acc in tqdm(uniq, desc="Assemble"):
            r = uniprot_rows.get(acc)
            if r is None:
                w.writerow([acc, ""])
                continue
            parts: list[str] = []
            # UniProt text (short, curated)
            uniprot_bits = [
                r.protein_name,
                r.organism_name,
                r.keywords,
                r.protein_families,
                r.cc_function,
                r.cc_subcellular_location,
            ]
            uniprot_text = " ".join([b for b in uniprot_bits if b])
            if uniprot_text:
                n_with_uniprot += 1
                parts.append(uniprot_text)
            # PubMed abstracts (richer)
            pmids = pmid_sets.get(acc, [])
            abs_parts: list[str] = []
            for pmid in pmids:
                title, abstract = pubmed_map.get(pmid, ("", ""))
                if title or abstract:
                    abs_parts.append(f"{title}. {abstract}".strip(" ."))
            if abs_parts:
                n_with_pubmed += 1
                parts.append(" ".join(abs_parts))
            text = " ".join(parts).strip()
            if args.strip_go and text:
                text = _strip_go_leakage(text)
            w.writerow([acc, text])
    print(f"Saved: {args.out_path}")
    print(f"Coverage: UniProt text {n_with_uniprot}/{len(uniq)} = {n_with_uniprot/ max(1,len(uniq)):.3f}")
    print(f"Coverage: PubMed text  {n_with_pubmed}/{len(uniq)} = {n_with_pubmed/ max(1,len(uniq)):.3f}")
    return 0
# Notebook safety:
# Jupyter/IPython runs with __name__ == '__main__', which would trigger argparse via main() and fail
# due to extra kernel arguments (e.g. '-f <kernel.json>').
# This cell is intentionally library-only; run the next notebook cell to execute it.
print('EntryID→text corpus builder loaded. Run Notebook Cell 06 to build WORK_ROOT/external/entryid_text.tsv.')

In [ ]:
# CELL 06 - Run: build EntryID->text corpus (UniProt+PubMed)
# PRODUCES: external/entryid_text.tsv
# Milestone checkpoint: stage_02_external_text
import os
import sys
from pathlib import Path

print('Stage stage_02_external_text: start')
out_path = WORK_ROOT / 'external' / 'entryid_text.tsv'
cache_dir = CACHE_ROOT / 'uniprot_pubmed_cache'
print(f'  out_path: {out_path} (exists={out_path.exists()})')
if out_path.exists():
    print('  out_path already exists; skipping rebuild')
else:
    cache_dir.mkdir(parents=True, exist_ok=True)
    sys.argv = [
        '03_build_entryid_text_from_uniprot_pubmed.py',
        '--artefacts-dir', str(WORK_ROOT),
        '--out-path', str(out_path),
        '--cache-dir', str(cache_dir),
        '--max-ids', '0',
        '--uniprot-batch-size', '80',
        '--max-pubmed-per-protein', '3',
        '--strip-go',
        '--sleep-uniprot', '0.1',
        '--sleep-pubmed', '0.34',
    ]
    _ = main()
if out_path.exists():
    STORE.maybe_push('stage_02_external_text', [out_path], note='entryid->text corpus')

In [ ]:
# CELL 07 - Inline: embeddings generator (used by TF-IDF; also supports extras)
print('Cell 7: embeddings generator loaded')
import argparse
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch

def _assert_all_finite_np(arr: np.ndarray, *, name: str) -> None:
    if not np.isfinite(arr).all():
        nan = int(np.isnan(arr).sum())
        inf = int(np.isinf(arr).sum())
        raise ValueError(f'{name}: non-finite values detected (nan={nan}, inf={inf}). Refusing to save.')

def _read_sequences(feather_path: Path) -> tuple[list[str], list[str]]:
    df = pd.read_feather(feather_path)
    if "id" not in df.columns or "sequence" not in df.columns:
        raise ValueError(f"Expected columns id, sequence in {feather_path}; got {list(df.columns)}")
    ids = df["id"].astype(str).tolist()
    seqs = df["sequence"].astype(str).tolist()
    return ids, seqs
def _smart_order(seqs: list[str]) -> np.ndarray:
    lengths = np.fromiter((len(s) for s in seqs), dtype=np.int64)
    return np.argsort(lengths)[::-1]
def _mean_pool(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    # last_hidden_state: (B, L, D), attention_mask: (B, L)
    mask = attention_mask.unsqueeze(-1).to(last_hidden_state.dtype)
    x = last_hidden_state * mask
    denom = mask.sum(dim=1).clamp(min=1.0)
    return x.sum(dim=1) / denom
@torch.no_grad()
def embed_esm2(
    seqs: list[str],
    model_name: str,
    batch_size: int,
    max_len: int,
    device: torch.device,
) -> np.ndarray:
    from transformers import EsmModel, EsmTokenizer
    tok = EsmTokenizer.from_pretrained(model_name)
    model = EsmModel.from_pretrained(model_name).to(device)
    model.eval()
    order = _smart_order(seqs)
    seqs_sorted = [seqs[i] for i in order]
    outs: list[np.ndarray] = []
    use_amp = device.type == "cuda"
    for i in range(0, len(seqs_sorted), batch_size):
        batch = seqs_sorted[i : i + batch_size]
        ids = tok.batch_encode_plus(
            batch,
            add_special_tokens=True,
            padding="longest",
            truncation=True,
            max_length=max_len,
            return_tensors="pt",
        )
        ids = {k: v.to(device) for k, v in ids.items()}
        ctx = torch.amp.autocast("cuda") if use_amp else torch.autocast("cpu", enabled=False)
        with ctx:
            out = model(**ids)
        pooled = _mean_pool(out.last_hidden_state.float(), ids["attention_mask"])
        outs.append(pooled.cpu().numpy())
        if device.type == "cuda":
            torch.cuda.empty_cache()
    embs_sorted = np.vstack(outs).astype(np.float32)
    embs = np.zeros_like(embs_sorted)
    embs[order] = embs_sorted
    return embs
@torch.no_grad()
def embed_ankh(
    seqs: list[str],
    model_name: str,
    batch_size: int,
    max_len: int,
    device: torch.device,
    trust_remote_code: bool,
) -> np.ndarray:
    # Ankh models are HuggingFace-hosted and may require trust_remote_code.
    # We treat them as encoder-style models and mean-pool hidden states over attention_mask.
    from transformers import AutoModel, AutoTokenizer
    tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    model = AutoModel.from_pretrained(model_name, trust_remote_code=trust_remote_code).to(device)
    model.eval()
    order = _smart_order(seqs)
    seqs_sorted = [seqs[i] for i in order]
    outs: list[np.ndarray] = []
    amp_enabled = device.type == "cuda"
    for i in range(0, len(seqs_sorted), batch_size):
        batch = seqs_sorted[i : i + batch_size]
        # Many protein LMs expect space-separated amino acids; if the tokenizer has a small vocab,
        # this typically helps. If it hurts, you can disable via --ankh-space-sep 0.
        batch = [" ".join(list(s.replace("U", "X").replace("Z", "X").replace("O", "X").replace("B", "X"))) for s in batch]
        ids = tok(
            batch,
            add_special_tokens=True,
            padding=True,
            truncation=True,
            max_length=max_len,
            return_tensors="pt",
        )
        ids = {k: v.to(device) for k, v in ids.items()}
        if "attention_mask" not in ids:
            raise RuntimeError(f"Tokenizer for {model_name} did not return attention_mask")
        if (ids["attention_mask"].sum(dim=1) == 0).any():
            raise RuntimeError(f"Ankh tokenisation produced an all-zero attention_mask (batch_start={i}).")
        def _forward(use_amp: bool):
            ctx = (
                torch.amp.autocast("cuda")
                if (use_amp and device.type == "cuda")
                else torch.autocast("cpu", enabled=False)
            )
            with ctx:
                return model(**ids)
        out = _forward(amp_enabled)
        # HuggingFace convention: encoder outputs `last_hidden_state`
        last = getattr(out, "last_hidden_state", None)
        if last is None:
            raise RuntimeError(f"Model {model_name} did not return last_hidden_state")
        pooled = _mean_pool(last.float(), ids["attention_mask"])
        if not bool(torch.isfinite(pooled).all().item()):
            if amp_enabled:
                print(f"WARNING: Non-finite Ankh embeddings under AMP at batch_start={i}; retrying without AMP.")
                amp_enabled = False
                out = _forward(False)
                last = getattr(out, "last_hidden_state", None)
                if last is None:
                    raise RuntimeError(f"Model {model_name} did not return last_hidden_state")
                pooled = _mean_pool(last.float(), ids["attention_mask"])
            if not bool(torch.isfinite(pooled).all().item()):
                raise RuntimeError(f"Ankh produced non-finite embeddings even without AMP (batch_start={i}).")
        outs.append(pooled.cpu().numpy())
        if device.type == "cuda":
            torch.cuda.empty_cache()
    embs_sorted = np.vstack(outs).astype(np.float32)
    _assert_all_finite_np(embs_sorted, name="ankh_embeds")
    embs = np.zeros_like(embs_sorted)
    embs[order] = embs_sorted
    return embs
def main() -> int:
    ap = argparse.ArgumentParser(description="Generate optional multimodal embeddings as .npy artefacts.")
    ap.add_argument(
        "--artefacts-dir",
        type=Path,
        default=Path("artefacts_local") / "artefacts",
        help="Path containing parsed/ and features/ (default: artefacts_local/artefacts)",
    )
    ap.add_argument(
        "--mode",
        choices=["esm2_3b", "ankh", "text"],
        required=True,
        help="Which optional embedding to generate.",
    )
    ap.add_argument(
        "--device",
        type=str,
        default="cuda",
        help="cuda|cpu (default: cuda)",
    )
    ap.add_argument("--batch-size", type=int, default=2)
    ap.add_argument("--max-len", type=int, default=1024)
    ap.add_argument(
        "--esm2-3b-model",
        type=str,
        default="facebook/esm2_t36_3B_UR50D",
        help="HF model id for ESM2-3B",
    )
    ap.add_argument(
        "--ankh-model",
        type=str,
        default="ElnaggarLab/ankh-large",
        help="HF model id for Ankh (large is typically 1536D)",
    )
    ap.add_argument(
        "--trust-remote-code",
        action="store_true",
        help="Pass trust_remote_code=True for Ankh loading (often required).",
    )
    # Text mode (fixed-width sparse->dense features; default dimension = 10279)
    ap.add_argument(
        "--text-path",
        type=Path,
        default=None,
        help="Path to a 2-column TSV/CSV with EntryID and text (required for --mode text).",
    )
    ap.add_argument(
        "--text-sep",
        type=str,
        default="\t",
        help="Separator for --text-path (default: tab).",
    )
    ap.add_argument(
        "--text-id-col",
        type=str,
        default="EntryID",
        help="ID column name in --text-path (default: EntryID).",
    )
    ap.add_argument(
        "--text-col",
        type=str,
        default="text",
        help="Text column name in --text-path (default: text).",
    )
    ap.add_argument(
        "--text-dim",
        type=int,
        default=10279,
        help="Output feature dimension for text TF-IDF (default: 10279).",
    )
    ap.add_argument(
        "--text-dtype",
        choices=["float16", "float32"],
        default="float16",
        help="On-disk dtype for text .npy (default: float16 to keep size sane).",
    )
    ap.add_argument(
        "--text-ngram-max",
        type=int,
        default=2,
        help="Max n-gram for TF-IDF (default: 2).",
    )
    args = ap.parse_args()
    artefacts_dir: Path = args.artefacts_dir
    parsed_dir = artefacts_dir / "parsed"
    feat_dir = artefacts_dir / "features"
    feat_dir.mkdir(parents=True, exist_ok=True)
    train_feather = parsed_dir / "train_seq.feather"
    test_feather = parsed_dir / "test_seq.feather"
    if not train_feather.exists() or not test_feather.exists():
        raise FileNotFoundError(
            f"Expected {train_feather} and {test_feather}. Run Phase 1 parsing first."
        )
    # Make HF caches relocatable (handy on Colab/Kaggle)
    os.environ.setdefault("HF_HOME", str(artefacts_dir / "hf_cache"))
    os.environ.setdefault("TRANSFORMERS_CACHE", str(artefacts_dir / "hf_cache"))
    os.environ.setdefault("TORCH_HOME", str(artefacts_dir / "torch_cache"))
    device = torch.device("cuda" if (args.device == "cuda" and torch.cuda.is_available()) else "cpu")
    print(f"Device: {device}")
    print("Loading sequences...")
    train_ids, train_seqs = _read_sequences(train_feather)
    test_ids, test_seqs = _read_sequences(test_feather)
    if args.mode == "esm2_3b":
        print(f"Embedding ESM2-3B: {args.esm2_3b_model}")
        train_emb = embed_esm2(train_seqs, args.esm2_3b_model, args.batch_size, args.max_len, device)
        test_emb = embed_esm2(test_seqs, args.esm2_3b_model, args.batch_size, args.max_len, device)
        np.save(feat_dir / "train_embeds_esm2_3b.npy", train_emb)
        np.save(feat_dir / "test_embeds_esm2_3b.npy", test_emb)
        print(f"Saved: {feat_dir / 'train_embeds_esm2_3b.npy'}")
        print(f"Saved: {feat_dir / 'test_embeds_esm2_3b.npy'}")
        return 0
    if args.mode == "ankh":
        print(f"Embedding Ankh: {args.ankh_model}")
        train_emb = embed_ankh(
            train_seqs,
            args.ankh_model,
            args.batch_size,
            args.max_len,
            device,
            trust_remote_code=args.trust_remote_code,
        )
        test_emb = embed_ankh(
            test_seqs,
            args.ankh_model,
            args.batch_size,
            args.max_len,
            device,
            trust_remote_code=args.trust_remote_code,
        )
        np.save(feat_dir / "train_embeds_ankh.npy", train_emb)
        np.save(feat_dir / "test_embeds_ankh.npy", test_emb)
        print(f"Saved: {feat_dir / 'train_embeds_ankh.npy'}")
        print(f"Saved: {feat_dir / 'test_embeds_ankh.npy'}")
        return 0
    if args.mode == "text":
        if args.text_path is None:
            raise ValueError("--text-path is required for --mode text")
        from sklearn.feature_extraction.text import TfidfVectorizer
        import joblib
        if not args.text_path.exists():
            raise FileNotFoundError(f"text-path not found: {args.text_path}")
        print(f"Loading text corpus: {args.text_path}")
        df = pd.read_csv(args.text_path, sep=args.text_sep, dtype=str)
        if args.text_id_col not in df.columns or args.text_col not in df.columns:
            raise ValueError(
                f"Expected columns {args.text_id_col!r}, {args.text_col!r} in {args.text_path}; got {list(df.columns)}"
            )
        # Many-to-one mapping is possible (multiple pubs per EntryID). Concatenate.
        df = df[[args.text_id_col, args.text_col]].dropna()
        df[args.text_id_col] = df[args.text_id_col].astype(str)
        df[args.text_col] = df[args.text_col].astype(str)
        grouped = df.groupby(args.text_id_col, sort=False)[args.text_col].apply(lambda x: " \n ".join(x.tolist()))
        text_map = grouped.to_dict()
        def _norm_uniprot_id(pid: str) -> str:
            # Common UniProt FASTA headers: sp|P12345|NAME_HUMAN ...
            parts = str(pid).split("|")
            if len(parts) >= 2 and parts[0] in {"sp", "tr"}:
                return parts[1]
            if len(parts) >= 3:
                return parts[1]
            return str(pid)
        # Support both raw IDs and normalised accessions for joining.
        text_map_norm: dict[str, str] = {}
        for k, v in text_map.items():
            text_map_norm[str(k)] = v
            text_map_norm[_norm_uniprot_id(k)] = v
        train_texts = [text_map_norm.get(str(pid), "") for pid in train_ids]
        test_texts = [text_map_norm.get(str(pid), "") for pid in test_ids]
        # TF-IDF gives a fixed-width, high-dimensional text modality with controllable size.
        # This is the most practical way to realise a 10279D "text embedding" without a bespoke LLM pipeline.
        print(f"Fitting TF-IDF (dim={args.text_dim}, ngram<= {args.text_ngram_max})...")
        vec = TfidfVectorizer(
            max_features=args.text_dim,
            ngram_range=(1, max(1, int(args.text_ngram_max))),
            min_df=2,
            strip_accents="unicode",
            lowercase=True,
        )
        vec.fit(train_texts + test_texts)
        n_features = len(vec.get_feature_names_out())
        print(f"TF-IDF vocab size: {n_features} (padded to {args.text_dim})")
        # Persist vectorizer for reproducibility
        joblib.dump(vec, feat_dir / "text_vectorizer.joblib")
        print(f"Saved: {feat_dir / 'text_vectorizer.joblib'}")
        out_dtype = np.float16 if args.text_dtype == "float16" else np.float32
        def _write_memmap(name: str, texts: list[str]):
            path = feat_dir / name
            mm = np.lib.format.open_memmap(path, mode="w+", dtype=out_dtype, shape=(len(texts), args.text_dim))
            bs = 2048
            for i in range(0, len(texts), bs):
                chunk = texts[i : i + bs]
                X = vec.transform(chunk)  # sparse
                # Convert per-chunk to dense to write to .npy
                arr = X.toarray().astype(out_dtype, copy=False)
                if arr.shape[1] == args.text_dim:
                    mm[i : i + arr.shape[0], :] = arr
                else:
                    dense = np.zeros((arr.shape[0], args.text_dim), dtype=out_dtype)
                    if arr.shape[1] > 0:
                        dense[:, : arr.shape[1]] = arr
                    mm[i : i + dense.shape[0], :] = dense
            mm.flush()
            return path
        train_path = _write_memmap("train_embeds_text.npy", train_texts)
        test_path = _write_memmap("test_embeds_text.npy", test_texts)
        print(f"Saved: {train_path}")
        print(f"Saved: {test_path}")
        return 0
    raise RuntimeError("unreachable")
# Notebook safety:
# This cell is intentionally library-only (no auto-run).
# Jupyter passes extra kernel args (e.g. '-f <kernel.json>') that break argparse-based mains.
# Use the dedicated run cell that sets sys.argv explicitly.

In [ ]:
# CELL 08 - Run: TF-IDF embeddings (10279D)
# PRODUCES: features/text_vectorizer.joblib, features/train_embeds_text.npy, features/test_embeds_text.npy
# Milestone checkpoint: stage_03_tfidf_text
# Option B strictness: TF-IDF is required, but we allow building it in-place when missing.
import sys
from pathlib import Path

STRICT_OPTION_B = True
USE_TFIDF_TEXT = True

feat_dir = WORK_ROOT / 'features'
text_vect = feat_dir / 'text_vectorizer.joblib'
train_text_npy = feat_dir / 'train_embeds_text.npy'
test_text_npy = feat_dir / 'test_embeds_text.npy'

generated = False
if USE_TFIDF_TEXT:
    have_all = text_vect.exists() and train_text_npy.exists() and test_text_npy.exists()
    if have_all:
        print('TF-IDF artefacts already exist; skipping generation')
    else:
        text_path = WORK_ROOT / 'external' / 'entryid_text.tsv'
        if not text_path.exists():
            raise FileNotFoundError(f'Missing corpus file: {text_path}. Run the corpus milestone first.')
        sys.argv = [
            '02_generate_optional_embeddings.py',
            '--artefacts-dir', str(WORK_ROOT),
            '--mode', 'text',
            '--text-path', str(text_path),
            '--text-dim', '10279',
        ]
        _ = main()
        generated = True

if text_vect.exists() and train_text_npy.exists() and test_text_npy.exists():
    # Avoid creating a new HF commit every time we re-run this cell.
    # Publish if we just generated TF-IDF, or if the user explicitly allows pushing existing artefacts.
    if generated:
        STORE.maybe_push('stage_03_tfidf_text', [text_vect, train_text_npy, test_text_npy], note='tf-idf 10279D')
    else:
        if PUSH_EXISTING_CHECKPOINTS:
            STORE.maybe_push('stage_03_tfidf_text', [text_vect, train_text_npy, test_text_npy], note='tf-idf 10279D (existing)')
        else:
            print('TF-IDF present; not pushing checkpoints (PUSH_EXISTING_CHECKPOINTS=False).')

    # Diagnostics: TF-IDF sparsity + row norms (sampled)
    try:
        import numpy as np
        import matplotlib.pyplot as plt

        plt.rcParams.update({'font.size': 12})

        def _sample_rows(x: np.ndarray, n: int = 20000) -> np.ndarray:
            m = min(int(x.shape[0]), int(n))
            if m <= 0:
                return np.zeros((0, int(x.shape[1])), dtype=np.float32)
            idx = np.linspace(0, int(x.shape[0]) - 1, num=m, dtype=np.int64)
            return np.asarray(x[idx], dtype=np.float32)

        x_tr = np.load(train_text_npy, mmap_mode='r')
        x_te = np.load(test_text_npy, mmap_mode='r')
        tr = _sample_rows(x_tr)
        te = _sample_rows(x_te)

        tr_nnz = (tr > 0).sum(axis=1)
        te_nnz = (te > 0).sum(axis=1)
        plt.figure(figsize=(10, 4))
        plt.hist(tr_nnz, bins=60, alpha=0.6, label=f'train nnz/row (n={len(tr_nnz)})')
        plt.hist(te_nnz, bins=60, alpha=0.6, label=f'test nnz/row (n={len(te_nnz)})')
        plt.title('TF-IDF: non-zeros per row (sampled)')
        plt.xlabel('nnz per row')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

        tr_norm = np.linalg.norm(tr, axis=1)
        te_norm = np.linalg.norm(te, axis=1)
        plt.figure(figsize=(10, 4))
        plt.hist(tr_norm, bins=60, alpha=0.6, label='train L2')
        plt.hist(te_norm, bins=60, alpha=0.6, label='test L2')
        plt.title('TF-IDF: L2 norm per row (sampled)')
        plt.xlabel('L2 norm')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
    except Exception as e:
        print('TF-IDF diagnostics skipped:', repr(e))
elif STRICT_OPTION_B:
    missing = [p for p in [text_vect, train_text_npy, test_text_npy] if not p.exists()]
    raise FileNotFoundError('Option B requires TF-IDF artefacts, but these are missing:\n' + '\n'.join([f' - {m}' for m in missing]))


In [ ]:
# CELL 08b - Manual publish: TF-IDF stage to Hugging Face (safe gating)
# Use this only if you intentionally want to publish Stage 03 to the HF checkpoint repo.
# This cell refuses to auto-enable pushes; you must enable via env/flags upstream (HF_PUSH / CAFA_HF_PUSH).

import numpy as np
from pathlib import Path

feat_dir = WORK_ROOT / 'features'
train_text_npy = feat_dir / 'train_embeds_text.npy'
test_text_npy = feat_dir / 'test_embeds_text.npy'
text_vect = feat_dir / 'text_vectorizer.joblib'

def _sample_rows(x: np.ndarray, n: int = 20000) -> np.ndarray:
    m = min(int(x.shape[0]), int(n))
    if m <= 0:
        return np.zeros((0, int(x.shape[1])), dtype=np.float32)
    idx = np.linspace(0, int(x.shape[0]) - 1, num=m, dtype=np.int64)
    return np.asarray(x[idx], dtype=np.float32)

def _looks_nonzero(path: Path, name: str) -> bool:
    if not path.exists():
        print(f'[CHECKPOINT] Missing: {path}')
        return False
    x = np.load(path, mmap_mode='r')
    sub = _sample_rows(x)
    if sub.size == 0:
        print(f'[CHECKPOINT] Empty array sample for {name}')
        return False
    if not np.isfinite(sub).all():
        print(f'[CHECKPOINT] Non-finite detected in {name} sample')
        return False
    if float(np.max(sub)) <= 0.0:
        print(f'[CHECKPOINT] {name} sample max==0 (all-zero suspected)')
        return False
    return True

# Sanity checks first
if not (text_vect.exists() and train_text_npy.exists() and test_text_npy.exists()):
    raise FileNotFoundError('TF-IDF artefacts missing; run Cell 10 first.')
if not (_looks_nonzero(train_text_npy, 'train_embeds_text') and _looks_nonzero(test_text_npy, 'test_embeds_text')):
    raise RuntimeError('Refusing to push: TF-IDF embeddings look broken (all-zero or non-finite).')

if 'STAGE_REQUIRED_FILES' not in globals():
    raise RuntimeError('Missing STAGE_REQUIRED_FILES. Run the earlier setup cell that defines stage requirements.')
if 'STORE' not in globals() or STORE is None:
    raise RuntimeError('Missing STORE. Run the checkpoint store setup cell first.')

# Refuse to override safety switches
if not bool(getattr(STORE, 'push_enabled', False)):
    raise RuntimeError('Checkpoint pushing is disabled (STORE.push_enabled=False). Enable CAFA_HF_PUSH=1 (and set CAFA_HF_REPO_ID) if you really want to publish.')

rels = STAGE_REQUIRED_FILES.get('stage_03_tfidf_text', [])
if not rels:
    raise RuntimeError("STAGE_REQUIRED_FILES['stage_03_tfidf_text'] is empty; refusing to push")
required_paths = [WORK_ROOT / r for r in rels]

STORE.maybe_push('stage_03_tfidf_text', required_paths, note='TF-IDF Stage 03 (manual publish)')

In [ ]:
# CELL 09a - Legacy GOA download/upload helper (DISABLED by default)
# This cell previously mixed in Kaggle dataset uploads and forced checkpoint pushing.
# That behaviour is intentionally disabled: the project now treats Hugging Face as the canonical artefact store,
# and expects GOA filtered artefacts to be provided via mounted datasets or precomputed files.

import os
from pathlib import Path

EXTERNAL_DIR = WORK_ROOT / 'external'
EXTERNAL_DIR.mkdir(parents=True, exist_ok=True)

filtered_path = EXTERNAL_DIR / 'goa_filtered_iea.tsv.gz'
print('External GOA (filtered) expected at:', filtered_path)
print('  exists:', filtered_path.exists())

if filtered_path.exists():
    # Enable downstream external processing cells
    os.environ['CAFA_PROCESS_EXTERNAL'] = '1'
    print('CAFA_PROCESS_EXTERNAL=1 (external GOA enabled).')
else:
    os.environ['CAFA_PROCESS_EXTERNAL'] = '0'
    print('CAFA_PROCESS_EXTERNAL=0 (external GOA disabled).')
    print('To enable external GOA priors, provide goa_filtered_iea.tsv.gz (or goa_filtered_all.tsv.gz) via:')
    print('  - a mounted Kaggle Dataset on Kaggle, or')
    print('  - a precomputed file placed under WORK_ROOT/external/, or')
    print('  - your offline pipeline (scripts/01_build_goa_features.py), then publish via HF checkpointing if needed.')

In [ ]:
# CELL 09 - Solution: 2.1 PHASE 1 (Step 3): EXTERNAL DATA & EVIDENCE CODES (ARTEFACT-FIRST)
# 2.1 PHASE 1 (Step 3): EXTERNAL DATA & EVIDENCE CODES (ARTEFACT-FIRST)
# ================================================================
# Option B strictness: external GOA artefacts are REQUIRED on Kaggle/Colab.
# Local runs default to best-effort (skip if missing) unless you opt-in.
from pathlib import Path
import os


def _detect_kaggle() -> bool:
    return bool(
        os.environ.get("KAGGLE_KERNEL_RUN_TYPE")
        or os.environ.get("KAGGLE_URL_BASE")
        or os.environ.get("KAGGLE_DATA_PROXY_URL")
    )


def _detect_colab() -> bool:
    return bool(
        os.environ.get("COLAB_RELEASE_TAG")
        or os.environ.get("COLAB_GPU")
        or os.environ.get("COLAB_TPU_ADDR")
    )


IS_KAGGLE = _detect_kaggle()
IS_COLAB = (not IS_KAGGLE) and _detect_colab()
IS_LOCAL = (not IS_KAGGLE) and (not IS_COLAB)

# Defaults:
# - Kaggle/Colab: strict + enabled
# - Local: non-strict + disabled (opt-in)
STRICT_OPTION_B = bool(int(os.getenv("CAFA_STRICT_OPTION_B", "0" if IS_LOCAL else "1")))
PROCESS_EXTERNAL = bool(int(os.getenv("CAFA_PROCESS_EXTERNAL", "0" if IS_LOCAL else "1")))

# Recommended (Kaggle): set CAFA_GOA_DATASET_DIR to your dataset folder,
# e.g. /kaggle/input/goa-filtered-all-tsv-gz
GOA_ENV_DIR = os.getenv("CAFA_GOA_DATASET_DIR", "").strip()
GOA_ARTEFACT_DIRS = [
    Path(GOA_ENV_DIR) if GOA_ENV_DIR else None,
    Path("/content/cafa6_data/external"),
    # Path('/kaggle/input/<dataset-folder>'),
]

if PROCESS_EXTERNAL:
    EXT_DIR = WORK_ROOT / "external"
    EXT_DIR.mkdir(exist_ok=True)

    def _existing(dirs: list[Path]) -> list[Path]:
        return [
            Path(d)
            for d in dirs
            if d is not None and str(d).strip() and Path(d).exists()
        ]

    def _discover(dirs: list[Path]) -> list[Path]:
        patterns = ("**/goa_filtered_*.tsv.gz", "**/goa_filtered_*.tsv")
        found: list[Path] = []
        for d in dirs:
            found += [
                d / "goa_filtered_all.tsv.gz",
                d / "goa_filtered_iea.tsv.gz",
                d / "goa_filtered_all.tsv",
                d / "goa_filtered_iea.tsv",
            ]
            for pat in patterns:
                found += sorted(Path(d).glob(pat))
        out: list[Path] = []
        seen = set()
        for p in found:
            p = Path(p)
            if p in seen:
                continue
            seen.add(p)
            if p.exists() and p.is_file():
                out.append(p)
        return out

    def _pick_best(paths: list[Path]) -> Path:
        def score(p: Path) -> tuple[int, int]:
            name = p.name.lower()
            ext_rank = 0 if name.endswith(".tsv.gz") else 1
            if "goa_filtered_all" in name:
                return (0, ext_rank)
            if "goa_filtered_iea" in name:
                return (1, ext_rank)
            if "all" in name:
                return (2, ext_rank)
            if "iea" in name:
                return (3, ext_rank)
            return (4, ext_rank)

        return sorted(paths, key=score)[0]

    roots = _existing(GOA_ARTEFACT_DIRS)
    if not roots and Path("/kaggle/input").exists():
        # fallback discovery, but still strict if nothing found
        roots = [Path("/kaggle/input")]
    elif not roots:
        roots = [EXT_DIR, Path("artefacts_local/artefacts/external")]

    goa_paths = _discover(roots)
    if not goa_paths:
        msg = (
            "External GOA artefacts not found.\n"
            "Fix:\n"
            " - Publish goa_filtered_all.tsv.gz (and/or goa_filtered_iea.tsv.gz) as a Kaggle Dataset\n"
            " - Attach it to this notebook\n"
            " - Set env var CAFA_GOA_DATASET_DIR=/kaggle/input/<dataset-folder> (recommended)\n"
            " - Or place the file under artefacts/external locally\n"
            "\n"
            f"Mode: STRICT_OPTION_B={STRICT_OPTION_B} PROCESS_EXTERNAL={PROCESS_EXTERNAL}"
        )
        if STRICT_OPTION_B:
            raise FileNotFoundError(msg)
        print(msg)
        PROCESS_EXTERNAL = False
    else:
        GOA_FEATURE_PATH = _pick_best(goa_paths)
        print("Using GOA artefact:", GOA_FEATURE_PATH)
        print("Format expected: EntryID<TAB>term<TAB>evidence (header included)")
else:
    print("Skipping external GOA features (PROCESS_EXTERNAL=0).")


In [ ]:
# CELL 10 - Solution: 2.2 PHASE 1 (Step 4): HIERARCHY PROPAGATION FOR EXTERNAL GOA (NO-KAGGLE / IEA)
# PRODUCES (when PROCESS_EXTERNAL=True): external/prop_train_no_kaggle.tsv, external/prop_test_no_kaggle.tsv
# Milestone checkpoint: stage_04_external_goa_priors
# Produces (when enabled):
# - external/prop_train_no_kaggle.tsv.gz
# - external/prop_test_no_kaggle.tsv.gz
# Option B strictness: this step MUST be satisfied when PROCESS_EXTERNAL=True.
import gzip
import pandas as pd
from pathlib import Path

EXTERNAL_PRIOR_SCORE = 1.0  # binary prior (later we down-weight when injecting into models)

if not PROCESS_EXTERNAL:
    print("Skipping external GOA hierarchy propagation (PROCESS_EXTERNAL=0).")
else:
    if "GOA_FEATURE_PATH" not in locals():
        raise RuntimeError(
            "PROCESS_EXTERNAL=True but GOA_FEATURE_PATH is missing. Run the external GOA discovery cell first."
        )

    # Ensure we have GO parents (hierarchy)
    if "go_parents" not in locals():
        def parse_obo(path: Path):
            parents = {}
            namespaces = {}
            cur_id, cur_ns = None, None
            with path.open("r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line == "[Term]":
                        if cur_id and cur_ns:
                            namespaces[cur_id] = cur_ns
                        cur_id, cur_ns = None, None
                    elif line.startswith("id: GO:"):
                        cur_id = line.split("id: ", 1)[1]
                    elif line.startswith("namespace:"):
                        cur_ns = line.split("namespace: ", 1)[1]
                    elif line.startswith("is_a:") and cur_id:
                        parent = line.split("is_a: ", 1)[1].split(" ! ")[0]
                        parents.setdefault(cur_id, set()).add(parent)
                if cur_id and cur_ns:
                    namespaces[cur_id] = cur_ns
            return parents, namespaces

        go_parents, go_namespaces = parse_obo(PATH_GO_OBO)

    # Ensure train/test IDs are available
    train_seq_path = WORK_ROOT / "parsed" / "train_seq.feather"
    test_seq_path = WORK_ROOT / "parsed" / "test_seq.feather"
    if not train_seq_path.exists():
        raise FileNotFoundError(
            "Missing train_seq.feather. Run Phase 1 Step 2 (FASTA parse) first."
        )
    if not test_seq_path.exists():
        raise FileNotFoundError(
            "Missing test_seq.feather. Run Phase 1 Step 2 (FASTA parse) first."
        )

    # IMPORTANT: GOA artefacts usually use UniProt accessions, but FASTA IDs may be like sp|P12345|NAME.
    # Normalise for matching, but write the original IDs so downstream joins (reindex on train_ids/test_ids) still work.
    def _norm_entry_id(raw: str) -> str:
        s = str(raw).strip()
        parts = s.split("|")
        if len(parts) >= 2 and parts[0] in {"sp", "tr"}:
            return parts[1]
        if len(parts) >= 3:
            return parts[1]
        return s

    train_ids_raw = pd.read_feather(train_seq_path)["id"].astype(str).tolist()
    test_ids_raw = pd.read_feather(test_seq_path)["id"].astype(str).tolist()
    train_id_map = {_norm_entry_id(i): i for i in train_ids_raw}
    test_id_map = {_norm_entry_id(i): i for i in test_ids_raw}
    train_ids = set(train_id_map.keys())
    test_ids = set(test_id_map.keys())

    # Get top-K train terms (defines the external feature space)
    # Champion Strategy: 13,500 terms (10k BP, 2k MF, 1.5k CC)
    train_terms_path = WORK_ROOT / "parsed" / "train_terms.parquet"
    if not train_terms_path.exists():
        raise FileNotFoundError(
            "Missing train_terms.parquet. Run Phase 1 Step 2 (targets parse) first."
        )
    train_terms = pd.read_parquet(train_terms_path)

    # Map terms to namespaces
    term_counts = train_terms["term"].value_counts()
    # Ensure all terms have a namespace (fallback to unknown if missing in OBO)
    term_ns = term_counts.index.map(lambda t: go_namespaces.get(t, "unknown"))

    # Select top K per aspect
    top_bp = term_counts[term_ns == "biological_process"].head(10000).index.tolist()
    top_mf = term_counts[term_ns == "molecular_function"].head(2000).index.tolist()
    top_cc = term_counts[term_ns == "cellular_component"].head(1500).index.tolist()

    top_terms = list(set(top_bp + top_mf + top_cc))
    top_terms_set = set(top_terms)
    print(f"External propagation restricted to top {len(top_terms)} train terms (BP={len(top_bp)}, MF={len(top_mf)}, CC={len(top_cc)}).")

    EXT_DIR = WORK_ROOT / "external"
    EXT_DIR.mkdir(exist_ok=True)
    out_train = EXT_DIR / "prop_train_no_kaggle.tsv.gz"
    out_test = EXT_DIR / "prop_test_no_kaggle.tsv.gz"

    if out_train.exists() and out_test.exists():
        print("Propagated GOA priors already present; skipping propagation.")
    else:
        # Ancestor closure with memoisation
        _anc_cache: dict[str, set[str]] = {}

        def ancestors(term: str) -> set[str]:
            if term in _anc_cache:
                return _anc_cache[term]
            seen = {term}
            stack = [term]
            while stack:
                t = stack.pop()
                for p in go_parents.get(t, ()):
                    if p not in seen:
                        seen.add(p)
                        stack.append(p)
            _anc_cache[term] = seen
            return seen

        cols = ["EntryID", "term", "evidence"]
        print("Streaming GOA artefact:", GOA_FEATURE_PATH)
        print("Writing:", out_train)
        print("Writing:", out_test)
        n_train = 0
        n_test = 0
        with gzip.open(out_train, "wt", encoding="utf-8") as ftr, gzip.open(
            out_test, "wt", encoding="utf-8"
        ) as fte:
            ftr.write("EntryID\tterm\tscore\n")
            fte.write("EntryID\tterm\tscore\n")
            for chunk in pd.read_csv(
                GOA_FEATURE_PATH,
                sep="\t",
                dtype=str,
                usecols=lambda c: c in cols,
                chunksize=500_000,
            ):
                missing_cols = [c for c in cols if c not in chunk.columns]
                if missing_cols:
                    raise ValueError(
                        f"GOA artefact missing columns: {missing_cols}. Found: {list(chunk.columns)}"
                    )
                chunk = chunk[cols].dropna()
                chunk = chunk[chunk["evidence"] == "IEA"]
                if chunk.empty:
                    continue
                chunk = chunk.drop_duplicates(subset=["EntryID", "term"])
                for entry_id_raw, term in zip(
                    chunk["EntryID"].tolist(), chunk["term"].tolist()
                ):
                    entry_norm = _norm_entry_id(entry_id_raw)
                    if entry_norm in train_ids:
                        target = ftr
                        out_entry = train_id_map[entry_norm]
                    elif entry_norm in test_ids:
                        target = fte
                        out_entry = test_id_map[entry_norm]
                    else:
                        continue
                    keep = ancestors(term) & top_terms_set
                    if not keep:
                        continue
                    for t in keep:
                        target.write(f"{out_entry}\t{t}\t{EXTERNAL_PRIOR_SCORE}\n")
                    if target is ftr:
                        n_train += len(keep)
                    else:
                        n_test += len(keep)
        print(f"Wrote propagated IEA edges: train={n_train:,} test={n_test:,}")
        print(
            "Outputs are intentionally sparse priors (score=1.0) and will be down-weighted when injected."
        )

    if out_train.exists() and out_test.exists():
        STORE.maybe_push(
            "stage_04_external_goa_priors",
            [out_train, out_test],
            note="propagated IEA priors",
        )

In [ ]:
# CELL 10b - Diagnostics: artefact manifest (existence + sizes)
%matplotlib inline
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({'font.size': 12})

def _mb(p: Path) -> float:
    return p.stat().st_size / (1024**2)

WORK_ROOT = Path(WORK_ROOT)
PRED_DIR = WORK_ROOT / 'features' / 'level1_preds'

# Minimal contract: Phase 1 outputs + Option B required artefacts + later model outputs
paths = {
    # Phase 1 parsed
    'parsed/train_seq.feather': WORK_ROOT / 'parsed' / 'train_seq.feather',
    'parsed/test_seq.feather': WORK_ROOT / 'parsed' / 'test_seq.feather',
    'parsed/train_terms.parquet': WORK_ROOT / 'parsed' / 'train_terms.parquet',
    'parsed/term_priors.parquet': WORK_ROOT / 'parsed' / 'term_priors.parquet',
    'parsed/train_taxa.feather': WORK_ROOT / 'parsed' / 'train_taxa.feather',
    'parsed/test_taxa.feather': WORK_ROOT / 'parsed' / 'test_taxa.feather',

    # Text pipeline
    'external/entryid_text.tsv': WORK_ROOT / 'external' / 'entryid_text.tsv',
    'features/text_vectorizer.joblib': WORK_ROOT / 'features' / 'text_vectorizer.joblib',
    'features/train_embeds_text.npy': WORK_ROOT / 'features' / 'train_embeds_text.npy',
    'features/test_embeds_text.npy': WORK_ROOT / 'features' / 'test_embeds_text.npy',

    # Sequence embeddings (core)
    'features/train_embeds_t5.npy': WORK_ROOT / 'features' / 'train_embeds_t5.npy',
    'features/test_embeds_t5.npy': WORK_ROOT / 'features' / 'test_embeds_t5.npy',
    'features/train_embeds_esm2.npy': WORK_ROOT / 'features' / 'train_embeds_esm2.npy',
    'features/test_embeds_esm2.npy': WORK_ROOT / 'features' / 'test_embeds_esm2.npy',
    'features/train_embeds_esm2_3b.npy': WORK_ROOT / 'features' / 'train_embeds_esm2_3b.npy',
    'features/test_embeds_esm2_3b.npy': WORK_ROOT / 'features' / 'test_embeds_esm2_3b.npy',
    'features/train_embeds_ankh.npy': WORK_ROOT / 'features' / 'train_embeds_ankh.npy',
    'features/test_embeds_ankh.npy': WORK_ROOT / 'features' / 'test_embeds_ankh.npy',

    # External priors
    'external/prop_train_no_kaggle.tsv.gz': WORK_ROOT / 'external' / 'prop_train_no_kaggle.tsv.gz',
    'external/prop_test_no_kaggle.tsv.gz': WORK_ROOT / 'external' / 'prop_test_no_kaggle.tsv.gz',

    # Term contract
    'features/top_terms_13500.json': WORK_ROOT / 'features' / 'top_terms_13500.json',

    # Level-1 predictions (canonical location)
    'features/level1_preds/oof_pred_logreg.npy': PRED_DIR / 'oof_pred_logreg.npy',
    'features/level1_preds/test_pred_logreg.npy': PRED_DIR / 'test_pred_logreg.npy',
    'features/level1_preds/oof_pred_logreg_BP.npy': PRED_DIR / 'oof_pred_logreg_BP.npy',
    'features/level1_preds/test_pred_logreg_BP.npy': PRED_DIR / 'test_pred_logreg_BP.npy',
    'features/level1_preds/oof_pred_logreg_MF.npy': PRED_DIR / 'oof_pred_logreg_MF.npy',
    'features/level1_preds/test_pred_logreg_MF.npy': PRED_DIR / 'test_pred_logreg_MF.npy',
    'features/level1_preds/oof_pred_logreg_CC.npy': PRED_DIR / 'oof_pred_logreg_CC.npy',
    'features/level1_preds/test_pred_logreg_CC.npy': PRED_DIR / 'test_pred_logreg_CC.npy',
    'features/level1_preds/oof_pred_gbdt.npy': PRED_DIR / 'oof_pred_gbdt.npy',
    'features/level1_preds/test_pred_gbdt.npy': PRED_DIR / 'test_pred_gbdt.npy',
    'features/level1_preds/oof_pred_dnn.npy': PRED_DIR / 'oof_pred_dnn.npy',
    'features/level1_preds/test_pred_dnn.npy': PRED_DIR / 'test_pred_dnn.npy',
    'features/level1_preds/oof_pred_knn.npy': PRED_DIR / 'oof_pred_knn.npy',
    'features/level1_preds/test_pred_knn.npy': PRED_DIR / 'test_pred_knn.npy',

    # Stacker output
    'features/test_pred_gcn.npy': WORK_ROOT / 'features' / 'test_pred_gcn.npy',
}

rows = []
for name, p in paths.items():
    rows.append({'artefact': name, 'exists': p.exists(), 'mb': _mb(p) if p.exists() else 0.0, 'path': str(p)})
df = pd.DataFrame(rows).sort_values(['exists', 'mb'], ascending=[True, False])
print('WORK_ROOT:', WORK_ROOT)

try:
    from IPython.display import display  # type: ignore
except Exception:
    def display(x):
        print(x)

display(df)

# Visual: top 25 largest artefacts
df2 = df[df['exists']].sort_values('mb', ascending=False).head(25)
if len(df2) > 0:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df2, y='artefact', x='mb')
    plt.title('Largest artefacts (MB)')
    plt.xlabel('MB')
    plt.ylabel('artefact')
    plt.tight_layout()
    plt.show()

# Option B strict check (external priors + embeddings + term contract)
option_b_required = [
    'external/prop_train_no_kaggle.tsv.gz',
    'external/prop_test_no_kaggle.tsv.gz',
    'features/text_vectorizer.joblib',
    'features/train_embeds_text.npy',
    'features/test_embeds_text.npy',
    'features/train_embeds_t5.npy',
    'features/test_embeds_t5.npy',
    'features/train_embeds_esm2.npy',
    'features/test_embeds_esm2.npy',
    'features/train_embeds_esm2_3b.npy',
    'features/test_embeds_esm2_3b.npy',
    'features/train_embeds_ankh.npy',
    'features/test_embeds_ankh.npy',
    'parsed/train_taxa.feather',
    'parsed/test_taxa.feather',
    'features/top_terms_13500.json',
]
missing = [a for a in option_b_required if not paths[a].exists()]
if missing:
    print('\nOption B missing artefacts:')
    for m in missing:
        print(' -', m)
else:
    print('\nOption B artefacts OK: priors + embeddings + TF-IDF + taxonomy + top terms present.')


In [ ]:
# CELL 11 - Solution: 3a. PHASE 1: EMBEDDINGS GENERATION (T5 only)
# PRODUCES: features/train_embeds_t5.npy, features/test_embeds_t5.npy
# 3a. PHASE 1: EMBEDDINGS GENERATION (T5 only)
# ============================================
# HARDWARE: GPU recommended
# ============================================
# Split from ESM2 so you can run each independently on Kaggle.
import os

# IMPORTANT: pushing checkpoints republishes a whole Kaggle Dataset version (but only the staged flat files we include),
# which can mean re-uploading multi-GB zips even when embeddings already exist.
t5_train_path = WORK_ROOT / 'features' / 'train_embeds_t5.npy'
t5_test_path = WORK_ROOT / 'features' / 'test_embeds_t5.npy'
has_test = (WORK_ROOT / 'parsed' / 'test_seq.feather').exists()
train_ready = t5_train_path.exists()
test_ready = (not has_test) or t5_test_path.exists()
COMPUTE_T5 = True  # <--- enable/disable T5 run
FORCE_REBUILD = False  # manual toggle; keep default strict
train_needed = COMPUTE_T5 and (FORCE_REBUILD or (not train_ready))
train_needed = COMPUTE_T5 and (FORCE_REBUILD or (not train_ready))
test_needed = COMPUTE_T5 and has_test and (FORCE_REBUILD or (not t5_test_path.exists()))
# If artefacts already exist, skip compute unless forced.
if COMPUTE_T5 and (not train_needed) and (not test_needed):
    COMPUTE_T5 = False
    if not PUSH_EXISTING_CHECKPOINTS:
        print('T5 embeddings already exist; skipping compute and not pushing checkpoints (PUSH_EXISTING_CHECKPOINTS=False).')
    if PUSH_EXISTING_CHECKPOINTS and 'STORE' in globals() and STORE is not None:
        # Granular checkpoints: allow resuming test without losing train.
        try:
            STORE.maybe_push('stage_05a_embeddings_t5_train', [t5_train_path], note='ProtT5 mean-pool embeddings (train only, existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_05a_embeddings_t5_train: {e}')
        if t5_test_path.exists():
            try:
                STORE.maybe_push('stage_05b_embeddings_t5_test', [t5_test_path], note='ProtT5 mean-pool embeddings (test only, existing)')
            except Exception as e:
                print(f'WARN: failed to push stage_05b_embeddings_t5_test: {e}')
        # Backwards-compatible combined stage.
        paths = [t5_train_path]
        if t5_test_path.exists():
            paths.append(t5_test_path)
        try:
            STORE.maybe_push('stage_05_embeddings_t5', paths, note='ProtT5 mean-pool embeddings (existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_05_embeddings_t5: {e}')
if COMPUTE_T5:
    import gc
    import numpy as np
    import pandas as pd
    import torch
    from transformers import T5Tokenizer, T5EncoderModel
    from tqdm.auto import tqdm
    import contextlib
    # Optimise CUDA memory allocation
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    # Fix Protobuf 'GetPrototype' error
    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    amp_ctx = torch.amp.autocast('cuda') if device.type == 'cuda' else contextlib.nullcontext()
    def get_t5_model():
        print("Loading T5 Model...")
        tokenizer = T5Tokenizer.from_pretrained(
            "Rostlab/prot_t5_xl_half_uniref50-enc", do_lower_case=False, legacy=True
        )
        model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(device)
        model.eval()
        return tokenizer, model
    def generate_embeddings_t5(model, tokenizer, sequences, batch_size=4, max_len=1024):
        seq_lens = [len(s) for s in sequences]
        sort_idx = np.argsort(seq_lens)[::-1]
        sorted_seqs = [sequences[i] for i in sort_idx]
        embeddings_list = []
        for i in tqdm(range(0, len(sorted_seqs), batch_size), desc="Embedding T5 (Smart Batch)"):
            batch_seqs = sorted_seqs[i : i + batch_size]
            batch_seqs = [seq.replace('U', 'X').replace('Z', 'X').replace('O', 'X').replace('B', 'X') for seq in batch_seqs]
            batch_seqs = [" ".join(list(seq)) for seq in batch_seqs]
            ids = tokenizer.batch_encode_plus(
                batch_seqs,
                add_special_tokens=True,
                padding="longest",
                truncation=True,
                max_length=max_len,
                return_tensors="pt",
            ).to(device)
            with torch.no_grad():
                with amp_ctx:
                    embedding_repr = model(input_ids=ids['input_ids'], attention_mask=ids['attention_mask'])
            emb = embedding_repr.last_hidden_state.float().detach().cpu().numpy()
            mask = ids['attention_mask'].detach().cpu().numpy()
            for j in range(len(batch_seqs)):
                seq_len = int(mask[j].sum())
                valid_emb = emb[j, :seq_len]
                embeddings_list.append(valid_emb.mean(axis=0))
            del ids, embedding_repr, emb, mask
            if device.type == 'cuda':
                torch.cuda.empty_cache()
        sorted_embeddings = np.vstack(embeddings_list)
        original_order_embeddings = np.zeros_like(sorted_embeddings)
        original_order_embeddings[sort_idx] = sorted_embeddings
        return original_order_embeddings
    tokenizer, model = get_t5_model()
    if train_needed:
        print("Loading train sequences for T5 embedding...")
        train_df = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')
        print(f"Generating Train Embeddings T5 ({len(train_df)})...")
        train_emb = generate_embeddings_t5(model, tokenizer, train_df['sequence'].tolist())
        np.save(t5_train_path, train_emb)
        # Push immediately after train completes so a crash during test doesn't lose progress.
        if 'STORE' in globals() and STORE is not None:
            try:
                STORE.maybe_push('stage_05a_embeddings_t5_train', [t5_train_path], note='ProtT5 mean-pool embeddings (train only)')
            except Exception as e:
                print(f'WARN: failed to push stage_05a_embeddings_t5_train: {e}')
        del train_emb, train_df
        gc.collect()
    else:
        print(f"Train embeddings already exist at {t5_train_path}; skipping train compute.")
    if test_needed:
        print("Loading test sequences for T5 embedding...")
        test_df = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')
        print(f"Generating Test Embeddings T5 ({len(test_df)})...")
        test_emb = generate_embeddings_t5(model, tokenizer, test_df['sequence'].tolist())
        np.save(t5_test_path, test_emb)
        # Push immediately after test completes as well.
        if 'STORE' in globals() and STORE is not None:
            try:
                STORE.maybe_push('stage_05b_embeddings_t5_test', [t5_test_path], note='ProtT5 mean-pool embeddings (test only)')
            except Exception as e:
                print(f'WARN: failed to push stage_05b_embeddings_t5_test: {e}')
        del test_emb, test_df
        gc.collect()
    else:
        if has_test:
            print(f"Test embeddings already exist at {t5_test_path}; skipping test compute.")
        else:
            print("No test sequences found; skipping test embedding.")
    del model, tokenizer
    gc.collect()
    if device.type == 'cuda':
        torch.cuda.empty_cache()
    print("T5 embeddings ready.")
    # Backwards-compatible combined stage.
    if 'STORE' in globals() and STORE is not None:
        paths = [t5_train_path]
        if t5_test_path.exists():
            paths.append(t5_test_path)
        try:
            STORE.maybe_push('stage_05_embeddings_t5', paths, note='ProtT5 mean-pool embeddings')
        except Exception as e:
            print(f'WARN: failed to push stage_05_embeddings_t5: {e}')
else:
    print("Skipping T5 embedding generation (COMPUTE_T5=False).")
# Diagnostics: embedding norms (train vs test; sampled)
try:
    import numpy as np
    import matplotlib.pyplot as plt
    from pathlib import Path
    plt.rcParams.update({'font.size': 12})
    def _plot_embed_norms(name: str, train_path: Path, test_path: Path):
        if not Path(train_path).exists():
            print(f'[{name}] missing train embeddings: {train_path}')
            return
        x_tr = np.load(train_path, mmap_mode='r')
        x_te = np.load(test_path, mmap_mode='r') if Path(test_path).exists() else None
        def _sample_norms(x: np.ndarray, n: int = 20000) -> np.ndarray:
            m = min(int(x.shape[0]), int(n))
            if m <= 0:
                return np.zeros((0,), dtype=np.float32)
            idx = np.linspace(0, int(x.shape[0]) - 1, num=m, dtype=np.int64)
            return np.linalg.norm(np.asarray(x[idx], dtype=np.float32), axis=1).astype(np.float32)
        tr_norm = _sample_norms(x_tr)
        plt.figure(figsize=(10, 4))
        plt.hist(tr_norm, bins=60, alpha=0.6, label=f'train (n={len(tr_norm)})')
        if x_te is not None:
            te_norm = _sample_norms(x_te)
            plt.hist(te_norm, bins=60, alpha=0.6, label=f'test (n={len(te_norm)})')
        plt.title(f'{name}: L2 norm distribution (sampled)')
        plt.xlabel('L2 norm')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
        def _stats(a: np.ndarray) -> dict:
            if a.size == 0:
                return {}
            return {
                'min': float(np.min(a)),
                'p50': float(np.median(a)),
                'p95': float(np.quantile(a, 0.95)),
                'max': float(np.max(a)),
            }
        print(f'[{name}] train norm stats:', _stats(tr_norm))
        if x_te is not None:
            print(f'[{name}] test norm stats:', _stats(te_norm))
    _plot_embed_norms('ProtT5', t5_train_path, t5_test_path)
except Exception as e:
    print('T5 embedding diagnostics skipped:', repr(e))

In [ ]:
# CELL 12 - Solution: 3b. PHASE 1: EMBEDDINGS GENERATION (ESM2 only)
# PRODUCES: features/train_embeds_esm2.npy, features/test_embeds_esm2.npy, features/train_embeds_esm2_3b.npy, features/test_embeds_esm2_3b.npy

# 3b. PHASE 1: EMBEDDINGS GENERATION (ESM2 only)

# HARDWARE: GPU recommended

# Split from Ankh so you can run each independently.

import os

FORCE_REBUILD = bool(globals().get('FORCE_REBUILD', False))

esm2_train_path = WORK_ROOT / 'features' / 'train_embeds_esm2.npy'

esm2_test_path = WORK_ROOT / 'features' / 'test_embeds_esm2.npy'

esm2_3b_train_path = WORK_ROOT / 'features' / 'train_embeds_esm2_3b.npy'

esm2_3b_test_path = WORK_ROOT / 'features' / 'test_embeds_esm2_3b.npy'

has_test = (WORK_ROOT / 'parsed' / 'test_seq.feather').exists()

COMPUTE_ESM2 = True

COMPUTE_ESM2_3B = True

esm2_train_needed = COMPUTE_ESM2 and (FORCE_REBUILD or (not esm2_train_path.exists()))

esm2_test_needed = COMPUTE_ESM2 and has_test and (FORCE_REBUILD or (not esm2_test_path.exists()))

esm2_3b_train_needed = COMPUTE_ESM2_3B and (FORCE_REBUILD or (not esm2_3b_train_path.exists()))

esm2_3b_test_needed = COMPUTE_ESM2_3B and has_test and (FORCE_REBUILD or (not esm2_3b_test_path.exists()))

if COMPUTE_ESM2 and (not esm2_train_needed) and (not esm2_test_needed):

    COMPUTE_ESM2 = False

    if PUSH_EXISTING_CHECKPOINTS and 'STORE' in globals() and STORE is not None:

        try:

            STORE.maybe_push('stage_06_embeddings_esm2_train', [esm2_train_path], note='ESM2-650M mean-pool embeddings (train only, existing)')

        except Exception as e:

            print(f'WARN: failed to push stage_06_embeddings_esm2_train: {e}')

        if esm2_test_path.exists():

            try:

                STORE.maybe_push('stage_06_embeddings_esm2_test', [esm2_test_path], note='ESM2-650M mean-pool embeddings (test only, existing)')

            except Exception as e:

                print(f'WARN: failed to push stage_06_embeddings_esm2_test: {e}')

        paths = [esm2_train_path]

        if esm2_test_path.exists():

            paths.append(esm2_test_path)

        try:

            STORE.maybe_push('stage_06_embeddings_esm2', paths, note='ESM2-650M mean-pool embeddings (existing)')

        except Exception as e:

            print(f'WARN: failed to push stage_06_embeddings_esm2: {e}')

if COMPUTE_ESM2_3B and (not esm2_3b_train_needed) and (not esm2_3b_test_needed):

    COMPUTE_ESM2_3B = False

    if PUSH_EXISTING_CHECKPOINTS and 'STORE' in globals() and STORE is not None:

        try:

            STORE.maybe_push('stage_06b_embeddings_esm2_3b_train', [esm2_3b_train_path], note='ESM2-3B mean-pool embeddings (train only, existing)')

        except Exception as e:

            print(f'WARN: failed to push stage_06b_embeddings_esm2_3b_train: {e}')

        if esm2_3b_test_path.exists():

            try:

                STORE.maybe_push('stage_06b_embeddings_esm2_3b_test', [esm2_3b_test_path], note='ESM2-3B mean-pool embeddings (test only, existing)')

            except Exception as e:

                print(f'WARN: failed to push stage_06b_embeddings_esm2_3b_test: {e}')

        paths = [esm2_3b_train_path]

        if esm2_3b_test_path.exists():

            paths.append(esm2_3b_test_path)

        try:

            STORE.maybe_push('stage_06b_embeddings_esm2_3b', paths, note='ESM2-3B mean-pool embeddings (existing)')

        except Exception as e:

            print(f'WARN: failed to push stage_06b_embeddings_esm2_3b: {e}')

if COMPUTE_ESM2 or COMPUTE_ESM2_3B:

    import gc, numpy as np, pandas as pd, torch

    from transformers import EsmTokenizer, EsmModel

    from tqdm.auto import tqdm

    import contextlib

    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print(f"Using device: {device}")

    amp_ctx = torch.amp.autocast('cuda') if device.type == 'cuda' else contextlib.nullcontext()

    def _mean_pool(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:

        mask = attention_mask.unsqueeze(-1).to(last_hidden_state.dtype)

        x = last_hidden_state * mask

        denom = mask.sum(dim=1).clamp(min=1.0)

        return x.sum(dim=1) / denom

    def get_esm2_model(model_name: str):

        print(f"Loading ESM2 model: {model_name}")

        tokenizer = EsmTokenizer.from_pretrained(model_name)

        model = EsmModel.from_pretrained(model_name).to(device)

        model.eval()

        return tokenizer, model

    def generate_embeddings_esm2(model, tokenizer, sequences, batch_size=16, max_len=1024):

        seq_lens = [len(s) for s in sequences]

        sort_idx = np.argsort(seq_lens)[::-1]

        sorted_seqs = [sequences[i] for i in sort_idx]

        embeddings_list = []

        for i in tqdm(range(0, len(sorted_seqs), batch_size), desc="Embedding ESM2 (Smart Batch)"):

            batch_seqs = sorted_seqs[i : i + batch_size]

            ids = tokenizer.batch_encode_plus(

                batch_seqs,

                add_special_tokens=True,

                padding="longest",

                truncation=True,

                max_length=max_len,

                return_tensors="pt",

            ).to(device)

            with torch.no_grad():

                with amp_ctx:

                    output = model(input_ids=ids['input_ids'], attention_mask=ids['attention_mask'])

            pooled = _mean_pool(output.last_hidden_state.float(), ids['attention_mask']).detach().cpu().numpy().astype(np.float32)

            embeddings_list.append(pooled)

            del ids, output, pooled

            if device.type == 'cuda':

                torch.cuda.empty_cache()

        sorted_embeddings = np.vstack(embeddings_list)

        original_order_embeddings = np.zeros_like(sorted_embeddings)

        original_order_embeddings[sort_idx] = sorted_embeddings

        return original_order_embeddings

    if esm2_train_needed or esm2_test_needed or esm2_3b_train_needed or esm2_3b_test_needed:

        print("Loading sequences...")

        train_df = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')

        test_df = None

        if (WORK_ROOT / 'parsed' / 'test_seq.feather').exists():

            test_df = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')

        train_seqs = train_df['sequence'].tolist()

        test_seqs = test_df['sequence'].tolist() if test_df is not None else None

    else:

        train_df = None

        test_df = None

        train_seqs = None

        test_seqs = None

    if COMPUTE_ESM2:

        tokenizer, model = get_esm2_model('facebook/esm2_t33_650M_UR50D')

        if esm2_train_needed:

            print(f"Generating Train Embeddings ESM2-650M ({len(train_seqs)})...")

            train_emb = generate_embeddings_esm2(model, tokenizer, train_seqs, batch_size=16)

            np.save(esm2_train_path, train_emb)

            if 'STORE' in globals() and STORE is not None:

                try:

                    STORE.maybe_push('stage_06_embeddings_esm2_train', [esm2_train_path], note='ESM2-650M mean-pool embeddings (train only)')

                except Exception as e:

                    print(f'WARN: failed to push stage_06_embeddings_esm2_train: {e}')

            del train_emb

            gc.collect()

        else:

            print(f"Train embeddings already exist at {esm2_train_path}; skipping train compute.")

        if esm2_test_needed and test_seqs is not None:

            print(f"Generating Test Embeddings ESM2-650M ({len(test_seqs)})...")

            test_emb = generate_embeddings_esm2(model, tokenizer, test_seqs, batch_size=16)

            np.save(esm2_test_path, test_emb)

            if 'STORE' in globals() and STORE is not None:

                try:

                    STORE.maybe_push('stage_06_embeddings_esm2_test', [esm2_test_path], note='ESM2-650M mean-pool embeddings (test only)')

                except Exception as e:

                    print(f'WARN: failed to push stage_06_embeddings_esm2_test: {e}')

            del test_emb

        else:

            if has_test:

                print(f"Test embeddings already exist at {esm2_test_path}; skipping test compute.")

            else:

                print("No test sequences found; skipping test embedding.")

        del model, tokenizer

        gc.collect()

        if device.type == 'cuda':

            torch.cuda.empty_cache()

        print("ESM2-650M embeddings ready.")

        if 'STORE' in globals() and STORE is not None:

            paths = [esm2_train_path]

            if esm2_test_path.exists():

                paths.append(esm2_test_path)

            try:

                STORE.maybe_push('stage_06_embeddings_esm2', paths, note='ESM2-650M mean-pool embeddings')

            except Exception as e:

                print(f'WARN: failed to push stage_06_embeddings_esm2: {e}')

    else:

        print("Skipping ESM2-650M embedding generation (COMPUTE_ESM2=False).")

    if COMPUTE_ESM2_3B:

        tokenizer, model = get_esm2_model('facebook/esm2_t36_3B_UR50D')

        if esm2_3b_train_needed:

            print(f"Generating Train Embeddings ESM2-3B ({len(train_seqs)})...")

            train_emb = generate_embeddings_esm2(model, tokenizer, train_seqs, batch_size=4)

            np.save(esm2_3b_train_path, train_emb)

            if 'STORE' in globals() and STORE is not None:

                try:

                    STORE.maybe_push('stage_06b_embeddings_esm2_3b_train', [esm2_3b_train_path], note='ESM2-3B mean-pool embeddings (train only)')

                except Exception as e:

                    print(f'WARN: failed to push stage_06b_embeddings_esm2_3b_train: {e}')

            del train_emb

            gc.collect()

        else:

            print(f"Train embeddings already exist at {esm2_3b_train_path}; skipping train compute.")

        if esm2_3b_test_needed and test_seqs is not None:

            print(f"Generating Test Embeddings ESM2-3B ({len(test_seqs)})...")

            test_emb = generate_embeddings_esm2(model, tokenizer, test_seqs, batch_size=4)

            np.save(esm2_3b_test_path, test_emb)

            if 'STORE' in globals() and STORE is not None:

                try:

                    STORE.maybe_push('stage_06b_embeddings_esm2_3b_test', [esm2_3b_test_path], note='ESM2-3B mean-pool embeddings (test only)')

                except Exception as e:

                    print(f'WARN: failed to push stage_06b_embeddings_esm2_3b_test: {e}')

            del test_emb

            gc.collect()

        else:

            if has_test:

                print(f"Test embeddings already exist at {esm2_3b_test_path}; skipping test compute.")

            else:

                print("No test sequences found; skipping test embedding.")

        del model, tokenizer

        gc.collect()

        if device.type == 'cuda':

            torch.cuda.empty_cache()

        print("ESM2-3B embeddings ready.")

        if 'STORE' in globals() and STORE is not None:

            paths = [esm2_3b_train_path]

            if esm2_3b_test_path.exists():

                paths.append(esm2_3b_test_path)

            try:

                STORE.maybe_push('stage_06b_embeddings_esm2_3b', paths, note='ESM2-3B mean-pool embeddings')

            except Exception as e:

                print(f'WARN: failed to push stage_06b_embeddings_esm2_3b: {e}')

    else:

        print("Skipping ESM2-3B embedding generation (COMPUTE_ESM2_3B=False).")

else:

    print("All requested ESM2 embedding artefacts already exist; skipping compute.")


In [ ]:
# CELL 12b - Solution: 3c. PHASE 1: EMBEDDINGS GENERATION (ANKH only)

# 3c. PHASE 1: EMBEDDINGS GENERATION (ANKH only)

# HARDWARE: GPU strongly recommended

# Split from ESM2 so you can run Ankh independently.

import os

DEFAULT_ANKH_MODEL = 'ElnaggarLab/ankh-large'
ANKH_MODEL = os.environ.get('CAFA_ANKH_MODEL', DEFAULT_ANKH_MODEL)

# BATCH SIZE CONFIGURATION
# Local (8GB VRAM): 2
# Colab L4/A100 (22.5GB+ VRAM): 8 (safe for Large), 32 (safe for Base)
BATCH_SIZE = 8 if globals().get('IS_COLAB', False) else 2

# Champion-aligned tokenisation: space-separate residues by default.
# Set CAFA_ANKH_SPACE_SEP=0 to disable.
ANKH_SPACE_SEP = bool(int(os.environ.get('CAFA_ANKH_SPACE_SEP', '1')))


def _safe_model_slug(model_name: str) -> str:
    return ''.join(ch if (ch.isalnum() or ch in ['-', '_']) else '_' for ch in model_name)


ankh_suffix = '' if ANKH_MODEL == DEFAULT_ANKH_MODEL else f"_{_safe_model_slug(ANKH_MODEL)}"

ankh_train_path = WORK_ROOT / 'features' / f'train_embeds_ankh{ankh_suffix}.npy'
ankh_test_path = WORK_ROOT / 'features' / f'test_embeds_ankh{ankh_suffix}.npy'

has_test = (WORK_ROOT / 'parsed' / 'test_seq.feather').exists()

ankh_train_exists = ankh_train_path.exists()
ankh_test_exists = ankh_test_path.exists()

# Required behaviour: skip train if it already exists; otherwise build it.
# Then do the same for test.
ankh_train_needed = not ankh_train_exists
ankh_test_needed = has_test and (not ankh_test_exists)

print(f"Ankh model: {ANKH_MODEL}")
print(f"Train embeddings: {ankh_train_path} (exists={ankh_train_exists}, needed={ankh_train_needed})")
print(f"Test embeddings : {ankh_test_path} (exists={ankh_test_exists}, needed={ankh_test_needed}, has_test={has_test})")

if (not ankh_train_needed) and (not ankh_test_needed):
    if PUSH_EXISTING_CHECKPOINTS and 'STORE' in globals() and STORE is not None:
        try:
            STORE.maybe_push(
                'stage_06c_embeddings_ankh_train',
                [ankh_train_path],
                note=f'Ankh mean-pool embeddings (train only, existing) {ANKH_MODEL}',
            )
        except Exception as e:
            print(f'WARN: failed to push stage_06c_embeddings_ankh_train: {e}')

        if ankh_test_path.exists():
            try:
                STORE.maybe_push(
                    'stage_06c_embeddings_ankh_test',
                    [ankh_test_path],
                    note=f'Ankh mean-pool embeddings (test only, existing) {ANKH_MODEL}',
                )
            except Exception as e:
                print(f'WARN: failed to push stage_06c_embeddings_ankh_test: {e}')

        paths = [ankh_train_path]
        if ankh_test_path.exists():
            paths.append(ankh_test_path)

        try:
            STORE.maybe_push(
                'stage_06c_embeddings_ankh',
                paths,
                note=f'Ankh mean-pool embeddings (existing) {ANKH_MODEL}',
            )
        except Exception as e:
            print(f'WARN: failed to push stage_06c_embeddings_ankh: {e}')

    print("Skipping Ankh embedding generation (all requested artefacts already exist).")
else:
    import contextlib
    import gc

    import numpy as np
    import pandas as pd
    import torch
    from tqdm.auto import tqdm
    from transformers import AutoTokenizer, T5EncoderModel

    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device} with Batch Size: {BATCH_SIZE}")
    print(f"ANKH_SPACE_SEP={ANKH_SPACE_SEP} (set CAFA_ANKH_SPACE_SEP=0 to disable)")

    # Disable AMP for Ankh to prevent instability/retries
    # amp_ctx = torch.amp.autocast('cuda') if device.type == 'cuda' else contextlib.nullcontext()

    def _mean_pool(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        mask = attention_mask.unsqueeze(-1).to(last_hidden_state.dtype)
        x = last_hidden_state * mask
        denom = mask.sum(dim=1).clamp(min=1.0)
        return x.sum(dim=1) / denom

    def get_ankh_model(model_name: str):
        print(f"Loading Ankh model: {model_name}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Ankh is T5-based. AutoModel would load encoder-decoder and require decoder inputs.
        # Use encoder-only model for embeddings.
        model = T5EncoderModel.from_pretrained(model_name).to(device)
        model.eval()
        return tokenizer, model

    def _sanitize_seq(seq: str) -> str:
        # Replace rare/ambiguous residues with 'X'
        seq = seq.replace('U', 'X').replace('Z', 'X').replace('O', 'X').replace('B', 'X')
        # Champion-aligned: space-separated residues
        if ANKH_SPACE_SEP:
            seq = ' '.join(list(seq))
        return seq

    def generate_embeddings_ankh(model, tokenizer, sequences, batch_size=2, max_len=1024):
        # Sort by *true residue length* (pre-space) for stable smart batching.
        seq_lens = [len(s) for s in sequences]
        sort_idx = np.argsort(seq_lens)[::-1]

        sorted_seqs = [_sanitize_seq(sequences[i]) for i in sort_idx]

        embeddings_list = []

        for batch_start in tqdm(range(0, len(sorted_seqs), batch_size), desc="Embedding ANKH (Smart Batch)"):
            batch_seqs = sorted_seqs[batch_start : batch_start + batch_size]

            ids = tokenizer.batch_encode_plus(
                batch_seqs,
                add_special_tokens=True,
                padding="longest",
                truncation=True,
                max_length=max_len,
                return_tensors="pt",
            ).to(device)

            # AMP DISABLED: Ankh is unstable in fp16, causing infinite retry loops.
            # We run in full precision (float32) to ensure stability and avoid 2x runtime from retries.
            with torch.no_grad():
                output = model(input_ids=ids['input_ids'], attention_mask=ids['attention_mask'])

            last = output.last_hidden_state

            # Force float32 for pooling/aggregation
            pooled_t = _mean_pool(last.float(), ids['attention_mask'])

            # Batch-level finite check
            if torch.isfinite(pooled_t).all():
                pooled = pooled_t.detach().cpu().numpy().astype(np.float32)
            else:
                raise RuntimeError(
                    f"Ankh produced non-finite embeddings even in float32 (batch_start={batch_start})."
                )

            embeddings_list.append(pooled)

            del ids, output, pooled_t, pooled
            if device.type == 'cuda':
                torch.cuda.empty_cache()

        sorted_embeddings = np.vstack(embeddings_list)

        # Final guardrail: never return corrupted arrays.
        if not np.isfinite(sorted_embeddings).all():
            raise RuntimeError("Final embeddings contain non-finite values; refusing to proceed.")

        original_order_embeddings = np.zeros_like(sorted_embeddings)
        original_order_embeddings[sort_idx] = sorted_embeddings
        return original_order_embeddings

    # Load only what we actually need.
    train_seqs = None
    test_seqs = None
    if ankh_train_needed:
        print("Loading train sequences...")
        train_df = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')
        train_seqs = train_df['sequence'].tolist()
    if ankh_test_needed:
        if (WORK_ROOT / 'parsed' / 'test_seq.feather').exists():
            print("Loading test sequences...")
            test_df = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')
            test_seqs = test_df['sequence'].tolist()
        else:
            test_seqs = None

    tokenizer, model = get_ankh_model(ANKH_MODEL)

    if ankh_train_needed:
        print(f"Generating Train Embeddings ANKH ({len(train_seqs)}) -> {ankh_train_path}")
        train_emb = generate_embeddings_ankh(model, tokenizer, train_seqs, batch_size=BATCH_SIZE)

        if not np.isfinite(train_emb).all():
            raise RuntimeError("Refusing to save: train embeddings contain non-finite values.")

        np.save(ankh_train_path, train_emb)

        if 'STORE' in globals() and STORE is not None:
            try:
                STORE.maybe_push(
                    'stage_06c_embeddings_ankh_train',
                    [ankh_train_path],
                    note=f'Ankh mean-pool embeddings (train only) {ANKH_MODEL}',
                )
            except Exception as e:
                print(f'WARN: failed to push stage_06c_embeddings_ankh_train: {e}')

        del train_emb
        gc.collect()
    else:
        print(f"Train embeddings already exist at {ankh_train_path}; skipping train compute.")

    if ankh_test_needed and test_seqs is not None:
        print(f"Generating Test Embeddings ANKH ({len(test_seqs)}) -> {ankh_test_path}")
        test_emb = generate_embeddings_ankh(model, tokenizer, test_seqs, batch_size=BATCH_SIZE)

        if not np.isfinite(test_emb).all():
            raise RuntimeError("Refusing to save: test embeddings contain non-finite values.")

        np.save(ankh_test_path, test_emb)

        if 'STORE' in globals() and STORE is not None:
            try:
                STORE.maybe_push(
                    'stage_06c_embeddings_ankh_test',
                    [ankh_test_path],
                    note=f'Ankh mean-pool embeddings (test only) {ANKH_MODEL}',
                )
            except Exception as e:
                print(f'WARN: failed to push stage_06c_embeddings_ankh_test: {e}')

        del test_emb
        gc.collect()
    else:
        if has_test:
            print(f"Test embeddings already exist at {ankh_test_path}; skipping test compute.")
        else:
            print("No test sequences found; skipping test embedding.")

    del model, tokenizer
    gc.collect()

    if device.type == 'cuda':
        torch.cuda.empty_cache()

    print("ANKH embeddings ready.")

    if 'STORE' in globals() and STORE is not None:
        paths = [ankh_train_path]
        if ankh_test_path.exists():
            paths.append(ankh_test_path)

        try:
            STORE.maybe_push('stage_06c_embeddings_ankh', paths, note=f'Ankh mean-pool embeddings {ANKH_MODEL}')
        except Exception as e:
            print(f'WARN: failed to push stage_06c_embeddings_ankh: {e}')

In [6]:
# CELL 13a - Setup & Data Loading (Phase 2 canonical)
# =============================================
# 4. PHASE 2: LEVEL-1 MODELS (DIVERSE ENSEMBLE)
# =============================================
# Target selection source-of-truth: Colab_04b_first_submission_no_ankh.ipynb (aspect-split Top-K)

TRAIN_LEVEL1 = True

if TRAIN_LEVEL1:
    import gc
    import json
    import os
    from pathlib import Path

    import numpy as np
    import pandas as pd
    import psutil

    # AUDITOR: Hardware Check
    try:
        import torch

        if torch.cuda.is_available():
            print(f"[AUDITOR] GPU Detected: {torch.cuda.get_device_name(0)}")
            print(
                f"[AUDITOR] VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB"
            )
        else:
            print("[AUDITOR] WARNING: No GPU detected.")
    except Exception:
        pass

    def log_mem(tag: str = "") -> None:
        try:
            mem = psutil.virtual_memory()
            print(
                f"[MEM] {tag:<30} | Used: {mem.used/1e9:.2f}GB / {mem.total/1e9:.2f}GB ({mem.percent}%)"
            )
        except Exception:
            pass

    # WORK_ROOT recovery (safety)
    # Prefer canonical dataset root (cafa6_data/) and validate by presence of parsed artefacts.
    if "WORK_ROOT" not in locals() and "WORK_ROOT" not in globals():
        candidates = [
            Path("/content/cafa6_data"),
            Path("/content/work"),
            Path("/kaggle/working/work"),
            Path.cwd() / "cafa6_data",
            Path.cwd() / "artefacts_local" / "work",
        ]

        WORK_ROOT = None
        for c in candidates:
            if (c / "parsed" / "train_terms.parquet").exists():
                WORK_ROOT = c
                break

        if WORK_ROOT is None:
            for c in candidates:
                if c.exists():
                    WORK_ROOT = c
                    break

        if WORK_ROOT is None:
            WORK_ROOT = Path.cwd() / "cafa6_data"

        print(f"WORK_ROOT recovered: {WORK_ROOT}")

    # -----------------------------
    # Load targets + ids
    # -----------------------------
    print("Loading targets...")
    train_terms = pd.read_parquet(WORK_ROOT / "parsed" / "train_terms.parquet")
    train_ids = pd.read_feather(WORK_ROOT / "parsed" / "train_seq.feather")["id"].astype(str)
    test_ids = pd.read_feather(WORK_ROOT / "parsed" / "test_seq.feather")["id"].astype(str)

    # FIX: Clean IDs in train_ids to match EntryID format
    print("Applying ID cleaning fix...")
    train_ids_clean = train_ids.str.extract(r"\|(.*?)\|")[0]
    train_ids_clean = train_ids_clean.fillna(train_ids)

    # -----------------------------
    # Target Matrix Construction (Champion Strategy: 13,500 Terms)
    # 10,000 BP + 2,000 MF + 1,500 CC
    # -----------------------------
    print("Selecting Top-K terms per aspect (Champion Strategy)...")

    try:
        import obonet

        # Robust OBO Path Search
        possible_paths = [
            WORK_ROOT / "go-basic.obo",
            WORK_ROOT / "Train" / "go-basic.obo",
            WORK_ROOT.parent / "go-basic.obo",
            Path("go-basic.obo"),
            Path("Train/go-basic.obo"),
            Path("../Train/go-basic.obo"),
            Path("/content/cafa6_data/Train/go-basic.obo"),
        ]

        obo_path = None
        for p in possible_paths:
            if p.exists():
                obo_path = p
                break

        if obo_path is None:
            raise FileNotFoundError(
                f"CRITICAL: go-basic.obo not found. Searched: {[str(p) for p in possible_paths]}"
            )

        global PATH_GO_OBO
        PATH_GO_OBO = obo_path
        print(f"Global PATH_GO_OBO set to: {PATH_GO_OBO}")

        print(f"Loading OBO from {obo_path}...")
        graph = obonet.read_obo(obo_path)
        term_to_ns = {
            node: data.get("namespace", "unknown") for node, data in graph.nodes(data=True)
        }

        # Keep compatibility with downstream code that expects go_namespaces
        go_namespaces = term_to_ns

        ns_map = {
            "biological_process": "BP",
            "molecular_function": "MF",
            "cellular_component": "CC",
        }

        # Normalise any existing aspect column (some artefacts store full namespace strings)
        aspect_aliases = {
            "biological_process": "BP",
            "molecular_function": "MF",
            "cellular_component": "CC",
            "BP": "BP",
            "MF": "MF",
            "CC": "CC",
        }
        if "aspect" in train_terms.columns:
            train_terms["aspect"] = train_terms["aspect"].map(
                lambda a: aspect_aliases.get(str(a), "UNK")
            )
        else:
            train_terms["aspect"] = train_terms["term"].map(
                lambda t: ns_map.get(term_to_ns.get(t), "UNK")
            )

    except ImportError as e:
        raise RuntimeError("obonet not installed. Please install it.") from e

    # Canonical aspect split (04b)
    term_counts = train_terms.groupby(["aspect", "term"]).size().reset_index(name="count")
    targets_bp = (
        term_counts[term_counts["aspect"] == "BP"].nlargest(10000, "count")["term"].tolist()
    )
    targets_mf = (
        term_counts[term_counts["aspect"] == "MF"].nlargest(2000, "count")["term"].tolist()
    )
    targets_cc = (
        term_counts[term_counts["aspect"] == "CC"].nlargest(1500, "count")["term"].tolist()
    )

    # Guardrail: avoid silently switching target strategy due to aspect encoding mismatch
    ALLOW_GLOBAL_FALLBACK = False
    if len(targets_bp) == 0 and len(targets_mf) == 0 and len(targets_cc) == 0:
        aspect_vc = train_terms["aspect"].value_counts().to_dict() if "aspect" in train_terms.columns else {}
        msg = (
            "No BP/MF/CC aspect split found after normalisation. "
            f"aspect_vc={aspect_vc}. This would fall back to global Top-13,500; "
            "set ALLOW_GLOBAL_FALLBACK=True to override."
        )
        if ALLOW_GLOBAL_FALLBACK:
            print("  [WARNING] " + msg)
            top_terms = train_terms["term"].value_counts().head(13500).index.tolist()
        else:
            raise RuntimeError(msg)
    else:
        # Stable, deterministic ordering: BP then MF then CC with de-dup preserving order
        top_terms = []
        seen = set()
        for t in (targets_bp + targets_mf + targets_cc):
            if t not in seen:
                top_terms.append(t)
                seen.add(t)
        print(f"  Selected: {len(targets_bp)} BP + {len(targets_mf)} MF + {len(targets_cc)} CC")

    # Persist label contract for downstream stages
    top_terms_path = WORK_ROOT / "features" / "top_terms_13500.json"
    top_terms_path.parent.mkdir(parents=True, exist_ok=True)
    if top_terms_path.exists():
        try:
            with open(top_terms_path, "r", encoding="utf-8") as f:
                top_terms_disk = json.load(f)
            if isinstance(top_terms_disk, list) and len(top_terms_disk) > 0:
                top_terms = [str(x) for x in top_terms_disk]
                print(f"Loaded existing top_terms_13500.json (n={len(top_terms)})")
        except Exception as e:
            print(f"[WARNING] Failed to load existing top_terms_13500.json: {e}")
    else:
        with open(top_terms_path, "w", encoding="utf-8") as f:
            json.dump(list(top_terms), f)
        print("Saved: top_terms_13500.json")

    # -----------------------------
    # Stable target contract (audited: 1,585 terms)
    # Definition: GO terms with >= 50 positives AND valid namespace (BP/MF/CC)
    # Stored separately from top_terms_13500.json (do not mix contracts).
    # -----------------------------
    stable_terms_path = WORK_ROOT / "features" / "stable_terms_1585.json"
    stable_meta_path = WORK_ROOT / "features" / "stable_terms_1585_meta.json"
    noise_floor = 50

    if stable_terms_path.exists():
        try:
            stable_terms = json.loads(stable_terms_path.read_text(encoding="utf-8"))
            stable_terms = [str(t) for t in stable_terms]
            print(f"Loaded existing stable_terms_1585.json (n={len(stable_terms)})")
        except Exception as e:
            raise RuntimeError(f"Failed to load {stable_terms_path}: {e}")
    else:
        # Compute from Phase-1 truth (train_terms.parquet) and OBO namespace mapping already loaded above.
        stable_bp = (
            term_counts[(term_counts["aspect"] == "BP") & (term_counts["count"] >= noise_floor)]
            .sort_values(["count", "term"], ascending=[False, True])["term"]
            .astype(str)
            .tolist()
        )
        stable_mf = (
            term_counts[(term_counts["aspect"] == "MF") & (term_counts["count"] >= noise_floor)]
            .sort_values(["count", "term"], ascending=[False, True])["term"]
            .astype(str)
            .tolist()
        )
        stable_cc = (
            term_counts[(term_counts["aspect"] == "CC") & (term_counts["count"] >= noise_floor)]
            .sort_values(["count", "term"], ascending=[False, True])["term"]
            .astype(str)
            .tolist()
        )
        stable_terms = stable_bp + stable_mf + stable_cc
        stable_terms_path.write_text(json.dumps(stable_terms), encoding="utf-8")
        stable_meta_path.write_text(
            json.dumps(
                {
                    "noise_floor": noise_floor,
                    "counts": {"BP": len(stable_bp), "MF": len(stable_mf), "CC": len(stable_cc)},
                    "total": len(stable_terms),
                },
                indent=2,
            ),
            encoding="utf-8",
        )
        print(f"Saved: stable_terms_1585.json (n={len(stable_terms)})")

    if len(stable_terms) != 1585:
        raise RuntimeError(f"Stable term contract mismatch: expected 1585, got {len(stable_terms)}")

    top_term_to_idx = {t: i for i, t in enumerate(top_terms)}
    missing_stable = [t for t in stable_terms if t not in top_term_to_idx]
    if missing_stable:
        raise RuntimeError(
            "Stable terms contain items not present in top_terms_13500.json. "
            f"Missing={len(missing_stable)} (example: {missing_stable[:10]})"
        )

    stable_idx = np.asarray([top_term_to_idx[t] for t in stable_terms], dtype=np.int64)
    print(f"Stable targets ready: n={int(stable_idx.shape[0])} (expected 1585)")

    train_terms_top = train_terms[train_terms["term"].isin(top_terms)]
    Y_df = train_terms_top.pivot_table(index="EntryID", columns="term", aggfunc="size", fill_value=0)
    Y_df = Y_df.reindex(train_ids_clean, fill_value=0)
    Y = Y_df.values.astype(np.float32)
    print(f"Targets: Y={Y.shape}")

    # -----------------------------
    # Feature loading helper (Memory Optimised)
    # -----------------------------
    FEAT_DIR = WORK_ROOT / "features"

    def load_features_dict(split: str = "both"):
        log_mem(f"Start load_features_dict({split})")
        print(f"Loading multimodal features (mode={split})...")

        ft_train = {}
        ft_test = {}

        def _load_pair(stem: str):
            tr = FEAT_DIR / f"train_embeds_{stem}.npy"
            te = FEAT_DIR / f"test_embeds_{stem}.npy"
            return tr, te

        # All modalities are mandatory.
        stems = [
            ("t5", "t5"),
            ("esm2", "esm2_650m"),
            ("esm2_3b", "esm2_3b"),
            ("ankh", "ankh"),
            ("text", "text"),
        ]

        for stem, key in stems:
            tr_path, te_path = _load_pair(stem)
            if not (tr_path.exists() and te_path.exists()):
                raise FileNotFoundError(f"Missing mandatory embeddings for {stem}: {tr_path} or {te_path}")

            if split in ["both", "train"]:
                ft_train[key] = np.load(tr_path, mmap_mode="r")
            if split in ["both", "test"]:
                ft_test[key] = np.load(te_path, mmap_mode="r")

        taxa_train_path = WORK_ROOT / "parsed" / "train_taxa.feather"
        taxa_test_path = WORK_ROOT / "parsed" / "test_taxa.feather"

        if not (taxa_train_path.exists() and taxa_test_path.exists()):
            raise FileNotFoundError(f"Missing mandatory taxa features: {taxa_train_path} or {taxa_test_path}")

        from sklearn.preprocessing import OneHotEncoder

        tax_tr = pd.read_feather(taxa_train_path).astype({"id": str})
        tax_te = pd.read_feather(taxa_test_path).astype({"id": str})
        enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=np.float32)
        enc.fit(pd.concat([tax_tr[["taxon_id"]], tax_te[["taxon_id"]]], axis=0))

        if split in ["both", "train"]:
            tax_tr = tax_tr.set_index("id").reindex(train_ids, fill_value=0).reset_index()
            ft_train["taxa"] = enc.transform(tax_tr[["taxon_id"]]).astype(np.float32)
        if split in ["both", "test"]:
            tax_te = tax_te.set_index("id").reindex(test_ids, fill_value=0).reset_index()
            ft_test["taxa"] = enc.transform(tax_te[["taxon_id"]]).astype(np.float32)

        log_mem(f"End load_features_dict({split})")
        if split == "train":
            return ft_train
        if split == "test":
            return ft_test
        return ft_train, ft_test

    # Materialise feature dicts (mmap arrays where possible)
    features_train, features_test = load_features_dict(split="both")

    # Flat concatenation order for classical models (LR/GBDT)
    FLAT_KEYS = [k for k in ["t5", "esm2_650m", "esm2_3b", "ankh", "text", "taxa"] if k in features_train]
    if "ankh" not in FLAT_KEYS:
        raise RuntimeError("Ankh is mandatory but was not loaded into features_train.")
    print(f"Flat X keys={FLAT_KEYS}")

    # -----------------------------
    # Disk-backed X / X_test (for RAM-safe downstream cells)
    # -----------------------------
    X_train_path = FEAT_DIR / "X_train_mmap.npy"
    X_test_path = FEAT_DIR / "X_test_mmap.npy"

    def _build_X_memmaps(chunk_size: int = 10000) -> None:
        dims = {k: int(features_train[k].shape[1]) for k in FLAT_KEYS}
        total_dim = int(sum(dims.values()))
        n_tr = int(len(train_ids))
        n_te = int(len(test_ids))

        print(f"Building X memmaps: train=({n_tr}, {total_dim}) test=({n_te}, {total_dim})")
        X_mm = np.lib.format.open_memmap(
            str(X_train_path), mode="w+", dtype=np.float32, shape=(n_tr, total_dim)
        )
        Xte_mm = np.lib.format.open_memmap(
            str(X_test_path), mode="w+", dtype=np.float32, shape=(n_te, total_dim)
        )

        col = 0
        for k in FLAT_KEYS:
            d = dims[k]
            print(f"  Streaming {k} into cols {col}:{col + d}")
            for i in range(0, n_tr, chunk_size):
                j = min(i + chunk_size, n_tr)
                X_mm[i:j, col : col + d] = np.asarray(features_train[k][i:j], dtype=np.float32)
            for i in range(0, n_te, chunk_size):
                j = min(i + chunk_size, n_te)
                Xte_mm[i:j, col : col + d] = np.asarray(features_test[k][i:j], dtype=np.float32)
            col += d

        X_mm.flush()
        Xte_mm.flush()

    if X_train_path.exists() and X_test_path.exists():
        print("X memmaps already exist; skipping build.")
    else:
        _build_X_memmaps(chunk_size=5000)

    X = np.load(X_train_path, mmap_mode="r")
    X_test = np.load(X_test_path, mmap_mode="r")

    log_mem("Phase 2 setup done")


[AUDITOR] GPU Detected: NVIDIA GeForce RTX 2070
[AUDITOR] VRAM: 8.59 GB
Loading targets...
Applying ID cleaning fix...
Selecting Top-K terms per aspect (Champion Strategy)...
Global PATH_GO_OBO set to: c:\Users\Olale\Documents\Codebase\Science\cafa-6-protein-function-prediction\cafa6_data\Train\go-basic.obo
Loading OBO from c:\Users\Olale\Documents\Codebase\Science\cafa-6-protein-function-prediction\cafa6_data\Train\go-basic.obo...
  Selected: 10000 BP + 2000 MF + 1500 CC
Loaded existing top_terms_13500.json (n=13500)
Loaded existing stable_terms_1585.json (n=1585)
Stable targets ready: n=1585 (expected 1585)


MemoryError: Unable to allocate 8.23 GiB for an array with shape (13500, 81865) and data type int64

In [ ]:
# CELL 13b-DEBUG - Pre-flight checks for GBDT
# ==========================================
import numpy as np
import sys

print("--- GBDT Pre-flight Checks ---")

# 1. Check Globals
required_globals = ['X', 'Y', 'X_test', 'stable_idx']
missing = [g for g in required_globals if g not in globals()]
if missing:
    raise RuntimeError(f"Missing required globals: {missing}. Did you run Cell 13a?")

# 2. Shape Checks
print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"stable_idx count: {len(stable_idx)}")

if X.shape[0] != Y.shape[0]:
    raise ValueError(f"Mismatch: X has {X.shape[0]} rows, Y has {Y.shape[0]} rows")

if len(stable_idx) != 1585:
    print(f"WARNING: stable_idx has {len(stable_idx)} terms (expected 1585). Proceeding, but verify this is intended.")

if np.max(stable_idx) >= Y.shape[1]:
    raise ValueError(f"Invalid stable_idx: max index {np.max(stable_idx)} >= Y columns {Y.shape[1]}")

# 3. NaN/Inf Checks (Sampled for speed)
sample_size = min(5000, X.shape[0])
print(f"Checking first {sample_size} rows for NaNs/Infs...")

if np.isnan(X[:sample_size]).any():
    raise ValueError("NaNs detected in X (training features)!")
if np.isinf(X[:sample_size]).any():
    raise ValueError("Infs detected in X (training features)!")

if np.isnan(X_test[:sample_size]).any():
    raise ValueError("NaNs detected in X_test (test features)!")

# 4. Target Checks (Elite subset)
Y_elite_sample = Y[:sample_size, stable_idx]
if np.isnan(Y_elite_sample).any():
    raise ValueError("NaNs detected in Y (elite targets)!")

# Check for dead targets in the elite set (using full Y if possible, or warn)
# Since Y might be large, we'll do a quick sum check if it's in memory, otherwise skip or use sample
if isinstance(Y, np.ndarray) and not isinstance(Y, np.memmap):
    # In-memory, fast enough
    positives = Y[:, stable_idx].sum(axis=0)
    dead_targets = np.where(positives == 0)[0]
    if len(dead_targets) > 0:
        print(f"WARNING: {len(dead_targets)} elite targets have 0 positive samples!")
else:
    print("Skipping full target sum check (Y is memmap or large).")

# 5. Library Check
try:
    import py_boost
    print(f"py_boost version: {py_boost.__version__ if hasattr(py_boost, '__version__') else 'installed'}")
except ImportError:
    print("WARNING: py_boost not found! Cell 13b will fail.")

print("--- CHECKS PASSED. Ready for GBDT. ---")

In [ ]:
# CELL 13b-DIAGNOSTIC - Inspect py_boost + GPU backend (CuPy) pre-flight
# Combines library inspection and the CuPy/py_boost GPU backend patch.
#
# Recommended setup (do this in the TERMINAL, not here): docs/ENV_GPU_PYBOOST_PY311.md
#
import importlib

# --- Inspect py_boost ---
import py_boost
import py_boost.multioutput
print("py_boost version:", py_boost.__version__)
print("py_boost contents:", dir(py_boost))
print("py_boost.multioutput contents:", dir(py_boost.multioutput))
try:
    from py_boost import GradientBoosting
    print("GradientBoosting imported successfully")
except ImportError as e:
    raise RuntimeError(f"GradientBoosting import failed: {repr(e)}") from e

# --- CuPy pre-flight for py_boost GPU path ---
# py_boost GPU path expects CuPy as `cp`. If CuPy isn't installed, py_boost can crash with: NameError: name 'cp' is not defined.
try:
    import cupy as cp  # type: ignore
except Exception as e:
    import sys
    msg = [
        "CuPy is required for py_boost GPU training but is not importable in this kernel.",
        f"Root cause: {repr(e)}",
        "Preferred fix: follow docs/ENV_GPU_PYBOOST_PY311.md (Python 3.11 + CUDA-matched CuPy wheel).",
    ]
    raise RuntimeError("\n".join(msg)) from e

# Sanity-check CuPy can allocate on GPU
try:
    _ = cp.zeros((1,), dtype=cp.float32)
    cp.cuda.runtime.deviceSynchronize()
except Exception as e:
    raise RuntimeError(f"CuPy imported but GPU allocation failed: {repr(e)}") from e

# Patch py_boost's GPU module to have `cp` in scope (workaround for some builds)
try:
    pb_boosting = importlib.import_module("py_boost.gpu.boosting")
    setattr(pb_boosting, "cp", cp)
    print("[OK] CuPy available and patched into py_boost.gpu.boosting as `cp`.")
except Exception as e:
    raise RuntimeError(f"Failed to patch py_boost GPU backend: {repr(e)}") from e

In [ ]:
# TEMP CELL — GBDT tiny smoke-run (fold‑1 only)
# Run this on a GPU VM before the GBDT training cell to make the run cheap and audit-friendly.

# Behaviour: GBDT cell will run only the first fold, then save checkpoints and stop.
GBDT_TINY_MAX_FOLDS = 1
print(f"[GBDT tiny] Enabled: will run at most {GBDT_TINY_MAX_FOLDS} fold(s).")

In [ ]:
# CELL 13b - Level 1: GBDT (py-boost) — Elite 1,585 stable terms + checkpoint push
# =====================================================================
# Track A (auditor): train GBDT only on the stable 1,585 terms (>=50 positives),
# but emit full 13,500-wide predictions so the stacker contract stays unchanged.

# Resume behaviour (added):
# - Saves `oof_pred_gbdt.npy` + `test_pred_gbdt.npy` after EACH fold.
# - Writes a small state file `gbdt_state_elite1585.json` listing completed folds.
# - On restart, loads the arrays + state and skips completed folds.

if not TRAIN_LEVEL1:
    print('Skipping GBDT (TRAIN_LEVEL1=False).')
else:
    import json
    import time
    import numpy as np
    from sklearn.model_selection import KFold
    from tqdm import tqdm

    PRED_DIR = WORK_ROOT / 'features' / 'level1_preds'
    PRED_DIR.mkdir(parents=True, exist_ok=True)

    gbdt_oof_path = PRED_DIR / 'oof_pred_gbdt.npy'
    gbdt_test_path = PRED_DIR / 'test_pred_gbdt.npy'
    gbdt_state_path = PRED_DIR / 'gbdt_state_elite1585.json'

    if 'stable_idx' not in globals():
        raise RuntimeError(
            "Missing stable_idx. Run Phase 2 setup (Cell 13a) first to generate stable_terms_1585.json."
        )

    elite_cols = np.asarray(stable_idx, dtype=np.int64)
    if elite_cols.ndim != 1 or elite_cols.size != 1585:
        raise RuntimeError(f"Expected elite_cols to have shape (1585,), got {elite_cols.shape}")

    n_splits = 5
    completed_folds: set[int] = set()

    if gbdt_state_path.exists():
        try:
            state = json.loads(gbdt_state_path.read_text(encoding='utf-8'))
            completed_folds = {int(x) for x in state.get('completed_folds', [])}
        except Exception:
            completed_folds = set()

    have_arrays = gbdt_oof_path.exists() and gbdt_test_path.exists()
    if have_arrays:
        oof_pred_gbdt = np.load(gbdt_oof_path)
        test_pred_gbdt = np.load(gbdt_test_path)
        if len(completed_folds) >= n_splits:
            print('Loaded existing GBDT preds (complete):', gbdt_oof_path.name, gbdt_test_path.name)
        else:
            print('Loaded partial GBDT preds; will resume.')
            print('  completed folds:', sorted(completed_folds) if completed_folds else 'none')
    else:
        oof_pred_gbdt = np.zeros((X.shape[0], Y.shape[1]), dtype=np.float32)
        test_pred_gbdt = np.zeros((X_test.shape[0], Y.shape[1]), dtype=np.float32)

    if len(completed_folds) < n_splits:
        # Auditor-aligned: manual CuPy injection into py_boost before training.
        try:
            import py_boost
        except Exception as e:
            raise RuntimeError(
                'py_boost is REQUIRED for this pipeline (GBDT is mandatory). Install it with: pip install py-boost',
            ) from e

        try:
            import cupy as cp
        except Exception as e:
            raise RuntimeError(
                'CuPy is required for py_boost GPU training but is not importable in this kernel. '
                'Install a CUDA-matched wheel (e.g. `pip install cupy-cuda12x`) and re-run.',
            ) from e

        try:
            import py_boost.gpu.boosting as gpu_boosting
            gpu_boosting.cp = cp
        except Exception as e:
            raise RuntimeError(f'Failed to inject CuPy into py_boost.gpu.boosting: {repr(e)}') from e

        # Robustness: patch cp into all py_boost submodules (some builds reference `cp` in other modules too).
        try:
            import importlib
            import pkgutil

            def _patch_pyboost_cp() -> int:
                patched = 0
                for modinfo in pkgutil.walk_packages(py_boost.__path__, prefix=py_boost.__name__ + '.'):
                    name = modinfo.name
                    try:
                        m = importlib.import_module(name)
                    except Exception:
                        continue
                    try:
                        setattr(m, 'cp', cp)
                        patched += 1
                    except Exception:
                        pass
                return patched

            _n_patched = _patch_pyboost_cp()
            print(f'[GBDT] Patched CuPy alias `cp` into {_n_patched} py_boost modules.')
        except Exception as e:
            raise RuntimeError(f'Failed to patch py_boost modules with CuPy alias `cp`: {repr(e)}') from e

        from py_boost import GradientBoosting

        # GPU kernel sanity: py_boost sometimes silently fails to compile/init CuPy kernels
        # (then later crashes with: TypeError: 'NoneType' object is not callable).
        try:
            import py_boost.gpu.utils as gpu_utils
            if hasattr(gpu_utils, 'init_kernels'):
                gpu_utils.init_kernels()
            _k = getattr(gpu_utils, 'feature_grouper_kernel', None)
            if _k is None:
                raise RuntimeError(
                    'py_boost GPU kernels are not initialised (feature_grouper_kernel is None). '
                    'This is usually a broken CUDA/CuPy toolchain on this VM (common on Python 3.12 or '
                    'a CUDA wheel/driver mismatch). Recommended: create a fresh Python 3.11 env and reinstall '
                    'py-boost + a CUDA-matched CuPy wheel (e.g. cupy-cuda11x or cupy-cuda12x), then restart the kernel.',
                )
        except Exception as e:
            raise RuntimeError(f'py_boost GPU backend is not usable: {repr(e)}') from e

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        # TEMP: allow a tiny audit run (e.g. fold-1 only) by setting GBDT_TINY_MAX_FOLDS in a prior cell.
        tiny_max_folds = int(globals().get('GBDT_TINY_MAX_FOLDS', 0) or 0)
        if tiny_max_folds > 0:
            print(f'[GBDT tiny] Limiting run to first {tiny_max_folds} fold(s).')

        # Train only elite targets
        Y_elite = Y[:, elite_cols].astype(np.float32, copy=False)
        print(f"[GBDT] Training elite targets: {Y_elite.shape[1]} / {Y.shape[1]}")

        base_params = dict(
            loss='logloss',
            ntrees=150,
            lr=0.05,
            max_depth=6,
            subsample=0.8,
            colsample=0.8,
        )

        # Cheap sanity check (seconds): validates the API before a long run.
        try:
            mini_n = int(min(100, X.shape[0]))
            if mini_n >= 50:
                base_check = GradientBoosting(loss='logloss', ntrees=1, seed=123)
                X_mini = np.ascontiguousarray(np.asarray(X[:mini_n], dtype=np.float32).copy())
                y_mini = np.ascontiguousarray(np.asarray(Y_elite[:mini_n, 0], dtype=np.float32).copy())
                base_check.fit(X_mini, y_mini)
                _p = np.asarray(base_check.predict(X_mini[: min(64, mini_n)]))
                if not np.isfinite(_p).all():
                    raise RuntimeError('Non-finite predictions in mini-fit (NaN/Inf).')
                del base_check, X_mini, y_mini, _p
                print('[GBDT] Mini-fit OK (API + finite preds).')
        except Exception as e:
            raise RuntimeError(f"GBDT mini-fit failed; aborting before long training. Root cause: {repr(e)}") from e

        # Performance: avoid re-reading disk-backed memmaps on every target.
        # Materialise X_test once into RAM (best effort; safe fallback if RAM is tight).
        try:
            print('[GBDT] Loading X_test into RAM (one-off) ...')
            X_test_ram = np.ascontiguousarray(np.asarray(X_test, dtype=np.float32))
            print(f'[GBDT] X_test_ram shape={X_test_ram.shape} dtype={X_test_ram.dtype}')
        except MemoryError:
            X_test_ram = X_test
            print('[GBDT] WARNING: insufficient RAM to materialise X_test; using X_test as-is (may be slower).')

        for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
            if tiny_max_folds > 0 and fold > tiny_max_folds:
                print(f'[GBDT tiny] Stop after fold {tiny_max_folds}.')
                break
            if fold in completed_folds:
                print(f'Fold {fold}/{n_splits} (GBDT elite) — SKIP (resume)')
                continue

            print(f'Fold {fold}/{n_splits} (GBDT elite)')

            # py_boost 0.5.2 uses 'seed' (auditor guidance)
            fold_seed = 42 + fold

            # Ensure fold slices are contiguous RAM arrays once per fold (numba/memmap safety).
            # Also avoids repeated random I/O into disk-backed memmaps.
            print('   [System] Pulling fold X into RAM ...')
            X_tr = np.ascontiguousarray(np.asarray(X[tr_idx], dtype=np.float32))
            X_va = np.ascontiguousarray(np.asarray(X[va_idx], dtype=np.float32))
            # Multi-output training: single model predicts ALL 1,585 targets per fold.
            # This removes the dominant bottleneck: calling predict(X_test) 1,585×.
            print('   [System] Pulling fold Y_elite into RAM ...')
            Y_tr = np.ascontiguousarray(np.asarray(Y_elite[tr_idx], dtype=np.float32))
            Y_va = np.ascontiguousarray(np.asarray(Y_elite[va_idx], dtype=np.float32))
            if Y_tr.ndim != 2 or Y_tr.shape[1] != int(len(elite_cols)):
                raise RuntimeError(f'Unexpected Y_tr shape: {Y_tr.shape} (expected (n_tr, {int(len(elite_cols))}))')
            try:
                from py_boost.multioutput.sketching import TopOutputsSketch
                _sketch = TopOutputsSketch(topk=int(len(elite_cols)))
            except Exception:
                _sketch = None
            model = GradientBoosting(
                **base_params,
                seed=fold_seed,
                target_splitter='OneVsAll',
                multioutput_sketch=_sketch,
            )
            t_fit = time.time()
            # Important: pass a validation set so `es=` early stopping can actually trigger.
            model.fit(X_tr, Y_tr, eval_sets=[{'X': X_va, 'y': Y_va}])
            print(f'   [GBDT] Fit done in {time.time() - t_fit:.1f}s')
            def _fix_pred_shape(p: np.ndarray, n_rows: int, n_cols: int, name: str) -> np.ndarray:
                a = np.asarray(p, dtype=np.float32)
                if a.ndim == 3 and a.shape[-1] == 1:
                    a = a[..., 0]
                if a.ndim == 1:
                    if n_cols == 1:
                        a = a.reshape(-1, 1)
                    else:
                        raise RuntimeError(f'{name} prediction is 1D {a.shape}, expected 2D (n_rows, n_cols)')
                if a.ndim != 2:
                    raise RuntimeError(f'{name} prediction has unexpected ndim={a.ndim}, shape={a.shape}')
                # Some backends return transposed (n_cols, n_rows)
                if a.shape == (n_cols, n_rows):
                    a = a.T
                if a.shape != (n_rows, n_cols):
                    raise RuntimeError(f'{name} prediction shape mismatch: got {a.shape}, expected {(n_rows, n_cols)}')
                return a
            t_pred = time.time()
            # GPU memory telemetry (best-effort): CuPy memory pool retains peak allocations.
            try:
                import cupy as cp
                pool = cp.get_default_memory_pool()
                pinned = cp.get_default_pinned_memory_pool()
                free0, total0 = cp.cuda.runtime.memGetInfo()
                pool_total0 = int(pool.total_bytes())
                pool_used0 = int(pool.used_bytes())
                pinned_total0 = int(pinned.total_bytes())
            except Exception:
                cp = None
                free0 = total0 = pool_total0 = pool_used0 = pinned_total0 = -1
            va = _fix_pred_shape(model.predict(X_va), n_rows=int(len(va_idx)), n_cols=int(len(elite_cols)), name='val')
            te = _fix_pred_shape(model.predict(X_test_ram), n_rows=int(X_test_ram.shape[0]), n_cols=int(len(elite_cols)), name='test')
            pred_s = time.time() - t_pred
            print(f'   [GBDT] Predict done in {pred_s:.1f}s')
            try:
                if cp is not None:
                    free1, total1 = cp.cuda.runtime.memGetInfo()
                    pool_total1 = int(pool.total_bytes())
                    pool_used1 = int(pool.used_bytes())
                    pinned_total1 = int(pinned.total_bytes())
                    # Note: pool_total_bytes is effectively a 'peak' allocation watermark.
                    print(
                        '   [GBDT][GPU mem] VRAM free/total (GiB):',
                        f'{free1 / (1024**3):.2f}/{total1 / (1024**3):.2f}',
                    )
                    print(
                        '   [GBDT][GPU mem] CuPy pool total/used (GiB):',
                        f'{pool_total1 / (1024**3):.2f}/{pool_used1 / (1024**3):.2f}',
                        '(total is peak-ish)',
                    )
                    print(
                        '   [GBDT][GPU mem] CuPy pinned pool total (GiB):',
                        f'{pinned_total1 / (1024**3):.2f}',
                    )
            except Exception as e:
                print('   [GBDT][GPU mem] telemetry skipped:', repr(e))
            va = np.clip(va, 0.0, 1.0)
            te = np.clip(te, 0.0, 1.0)
            oof_pred_gbdt[np.ix_(va_idx, elite_cols)] = va
            test_pred_gbdt[:, elite_cols] += te / float(n_splits)

            if 'ia_weighted_f1' in globals():
                try:
                    va_elite = oof_pred_gbdt[np.ix_(va_idx, elite_cols)]
                    print('  IA-F1 (elite only):', ia_weighted_f1(Y_elite[va_idx], va_elite, thr=0.3))
                except Exception as e:
                    print('  IA-F1 skipped:', repr(e))

            # Fold checkpoint (resume)
            completed_folds.add(fold)
            np.save(gbdt_oof_path, oof_pred_gbdt)
            np.save(gbdt_test_path, test_pred_gbdt)
            gbdt_state_path.write_text(
                json.dumps(
                    {
                        'n_splits': n_splits,
                        'completed_folds': sorted(completed_folds),
                    },
                    indent=2,
                ),
                encoding='utf-8',
            )
            print(f'[GBDT] Saved checkpoint after fold {fold}/{n_splits}: {gbdt_oof_path.name}, {gbdt_test_path.name}')

        # Final save (also covers the no-resume path)
        np.save(gbdt_oof_path, oof_pred_gbdt)
        np.save(gbdt_test_path, test_pred_gbdt)
        gbdt_state_path.write_text(
            json.dumps(
                {
                    'n_splits': n_splits,
                    'completed_folds': sorted(completed_folds) if completed_folds else list(range(1, n_splits + 1)),
                },
                indent=2,
            ),
            encoding='utf-8',
        )
        print('Saved:', gbdt_oof_path)
        print('Saved:', gbdt_test_path)

    # Checkpoint push (always)
    STORE.maybe_push(
        stage='stage_07b_level1_gbdt',
        required_paths=[
            str((WORK_ROOT / 'features' / 'top_terms_13500.json').as_posix()),
            str((WORK_ROOT / 'features' / 'stable_terms_1585.json').as_posix()),
            str(gbdt_oof_path.as_posix()),
            str(gbdt_test_path.as_posix()),
        ],
        note='Level-1 GBDT predictions (OOF + test). Trained on elite 1,585 stable targets; emitted as 13,500-wide matrices.',
    )

    # Diagnostics: probability histograms + IA-F1 vs threshold (sampled)
    try:
        import os
        import matplotlib.pyplot as plt

        plt.rcParams.update({'font.size': 12})
        DIAG_N = int(os.getenv('CAFA_DIAG_N', '20000'))

        def _sub(y_true: np.ndarray, y_score: np.ndarray):
            n = int(y_true.shape[0])
            m = min(n, int(DIAG_N))
            if m <= 0:
                return y_true[:0], y_score[:0]
            idx = np.linspace(0, n - 1, num=m, dtype=np.int64)
            return y_true[idx], y_score[idx]

        y_t, y_s = _sub(Y, oof_pred_gbdt)
        row_max_oof = y_s.max(axis=1)
        row_mean_oof = y_s.mean(axis=1)

        plt.figure(figsize=(10, 4))
        plt.hist(row_max_oof, bins=60, alpha=0.7)
        plt.title('GBDT OOF: max probability per protein (sampled)')
        plt.xlabel('max prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()

        plt.figure(figsize=(10, 4))
        plt.hist(row_mean_oof, bins=60, alpha=0.7)
        plt.title('GBDT OOF: mean probability per protein (sampled)')
        plt.xlabel('mean prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()

        if test_pred_gbdt is not None:
            te_s = test_pred_gbdt
            te_m = min(int(te_s.shape[0]), int(DIAG_N))
            te_idx = (
                np.linspace(0, int(te_s.shape[0]) - 1, num=te_m, dtype=np.int64)
                if te_m > 0
                else np.array([], dtype=np.int64)
            )
            row_max_te = te_s[te_idx].max(axis=1) if te_m > 0 else np.array([], dtype=np.float32)
            plt.figure(figsize=(10, 4))
            plt.hist(row_max_oof, bins=60, alpha=0.5, label='OOF')
            plt.hist(row_max_te, bins=60, alpha=0.5, label='test')
            plt.title('GBDT: max probability per protein (OOF vs test; sampled)')
            plt.xlabel('max prob')
            plt.ylabel('count')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.show()

        if 'ia_weighted_f1' in globals():
            thrs = np.linspace(0.05, 0.60, 23)
            curves = {k: [] for k in ['ALL', 'MF', 'BP', 'CC']}
            for thr in thrs:
                s = ia_weighted_f1(y_t, y_s, thr=float(thr))
                for k in curves.keys():
                    curves[k].append(s[k])
            plt.figure(figsize=(10, 3))
            for k in ['ALL', 'MF', 'BP', 'CC']:
                plt.plot(thrs, curves[k], label=k)
            plt.title('GBDT OOF: IA-weighted F1 vs threshold (sampled)')
            plt.xlabel('threshold')
            plt.ylabel('IA-F1')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
    except Exception as e:
        print('GBDT diagnostics skipped:', repr(e))

NameError: name 'TRAIN_LEVEL1' is not defined

In [ ]:
# TEMP CELL — Verify GBDT artefacts + elite-only writes
# Run AFTER the GBDT cell finishes (even in fold‑1 mode).
from pathlib import Path
import numpy as np

PRED_DIR = WORK_ROOT / 'features' / 'level1_preds'
gbdt_oof_path = PRED_DIR / 'oof_pred_gbdt.npy'
gbdt_test_path = PRED_DIR / 'test_pred_gbdt.npy'
gbdt_state_path = PRED_DIR / 'gbdt_state_elite1585.json'

assert gbdt_oof_path.exists(), f"Missing {gbdt_oof_path}"
assert gbdt_test_path.exists(), f"Missing {gbdt_test_path}"
print('[GBDT verify] Found:', gbdt_oof_path.name, gbdt_test_path.name)
if gbdt_state_path.exists():
    print('[GBDT verify] State:', gbdt_state_path.read_text(encoding='utf-8')[:500])

oof = np.load(gbdt_oof_path, mmap_mode='r')
te = np.load(gbdt_test_path, mmap_mode='r')

print('[GBDT verify] oof shape:', tuple(oof.shape), 'expected:', (int(X.shape[0]), int(Y.shape[1])))
print('[GBDT verify] test shape:', tuple(te.shape), 'expected:', (int(X_test.shape[0]), int(Y.shape[1])))
assert oof.shape == (X.shape[0], Y.shape[1])
assert te.shape == (X_test.shape[0], Y.shape[1])

elite_cols = np.asarray(stable_idx, dtype=np.int64)
elite_mask = np.zeros((Y.shape[1],), dtype=bool)
elite_mask[elite_cols] = True
outside_cols = np.flatnonzero(~elite_mask)
print('[GBDT verify] elite cols:', int(elite_cols.size), 'outside cols:', int(outside_cols.size))

# Cheap check: sample outside entries and confirm they are all exactly zero.
# This avoids a full scan of a multi-GB matrix while still catching accidental writes.
rng = np.random.default_rng(123)
n_check_rows = int(min(2048, oof.shape[0]))
n_check_cols = int(min(2048, outside_cols.size))
rows = rng.choice(oof.shape[0], size=n_check_rows, replace=False) if n_check_rows < oof.shape[0] else np.arange(oof.shape[0])
cols = rng.choice(outside_cols, size=n_check_cols, replace=False) if n_check_cols < outside_cols.size else outside_cols
block = np.asarray(oof[np.ix_(rows, cols)])
max_abs = float(np.max(np.abs(block))) if block.size else 0.0
print(f"[GBDT verify] sampled outside max|value|={max_abs:.3g} (expect 0)")
assert max_abs == 0.0, 'Non-zero detected outside elite_cols (sampled). Investigate writes.'

# Sanity: elite slice should contain some non-zeros after at least one fold (unless model failed silently).
elite_sample = np.asarray(oof[np.ix_(rows, elite_cols[: min(64, elite_cols.size)])])
elite_max = float(np.max(elite_sample)) if elite_sample.size else 0.0
print(f"[GBDT verify] sampled elite max={elite_max:.3g} (expect > 0 if fold ran)")

In [ ]:
# CELL 13c - Level 1: Logistic Regression — Long tail 13,500 (Aspect Split BP/MF/CC)
# ==============================================================================
# Track B (auditor): regularised linear model over the full 13,500-term space.
# Goal: Train LR per GO aspect (BP/MF/CC) with RAM-safe target handling.
#
# Critical fix: Avoid NumPy fancy-indexing on memmaps.
# - DO NOT do: Y_aspect = Y_full[:, aspect_indices]  (this materialises a huge dense copy in host RAM)
# - Instead: slice Y_full per fold+target-chunk via np.ix_(idx_tr, cols)
#
# Protocol (recommended for stability): run ONE aspect per fresh runtime.
# If `TARGET_ASPECT` is not set, we default to BP.
# ==============================================================================

if not TRAIN_LEVEL1:
    print('Skipping LogReg (TRAIN_LEVEL1=False).')
else:
    import os
    import sys
    import time
    import gc
    import warnings
    import psutil
    import json
    import numpy as np
    import pandas as pd
    import joblib
    import torch
    from pathlib import Path
    from tqdm.auto import tqdm
    from sklearn.model_selection import KFold
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import SGDClassifier
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.metrics import f1_score
    from sklearn.exceptions import ConvergenceWarning

    warnings.filterwarnings('ignore', category=ConvergenceWarning)

    def _stage(msg: str) -> None:
        print(msg)
        try:
            sys.stdout.flush()
        except Exception:
            pass

    # -----------------------------
    # WORK_ROOT recovery (safety)
    # -----------------------------
    if 'WORK_ROOT' not in locals() and 'WORK_ROOT' not in globals():
        candidates = [
            Path('/content/cafa6_data'),
            Path('/content/work'),
            Path('/kaggle/working/work'),
            Path.cwd() / 'cafa6_data',
            Path.cwd() / 'artefacts_local' / 'work',
        ]
        WORK_ROOT = None
        for c in candidates:
            if (c / 'parsed' / 'train_terms.parquet').exists():
                WORK_ROOT = c
                break
        if WORK_ROOT is None:
            for c in candidates:
                if c.exists():
                    WORK_ROOT = c
                    break
        if WORK_ROOT is None:
            WORK_ROOT = Path.cwd() / 'cafa6_data'
        _stage(f"[AUDITOR] Recovered WORK_ROOT: {WORK_ROOT}")

    FEAT_DIR = Path(WORK_ROOT) / 'features'
    PARSED_DIR = Path(WORK_ROOT) / 'parsed'

    PRED_DIR = FEAT_DIR / 'level1_preds'
    PRED_DIR.mkdir(parents=True, exist_ok=True)

    # -----------------------------
    # Load top_terms + term_to_ns
    # -----------------------------
    top_terms_path = FEAT_DIR / 'top_terms_13500.json'
    if not top_terms_path.exists():
        raise FileNotFoundError(f"Missing {top_terms_path}. Run the Phase 2 setup cell first.")
    top_terms = json.loads(top_terms_path.read_text(encoding='utf-8'))
    top_terms = [str(t) for t in top_terms]

    # Build / reuse term_to_ns mapping
    if 'term_to_ns' in globals():
        term_to_ns = globals()['term_to_ns']
    else:
        try:
            import obonet
        except Exception as e:
            raise RuntimeError('obonet is required for aspect split (term namespaces).') from e

        possible_paths = []
        if 'PATH_GO_OBO' in globals():
            try:
                possible_paths.append(Path(globals()['PATH_GO_OBO']))
            except Exception:
                pass
        possible_paths += [
            Path(WORK_ROOT) / 'Train' / 'go-basic.obo',
            Path(WORK_ROOT) / 'go-basic.obo',
            Path('/content/cafa6_data/Train/go-basic.obo'),
            Path('Train/go-basic.obo'),
            Path('go-basic.obo'),
        ]
        obo_path = None
        for p in possible_paths:
            if p is not None and Path(p).exists():
                obo_path = Path(p)
                break
        if obo_path is None:
            raise FileNotFoundError(f"go-basic.obo not found. Candidates: {[str(p) for p in possible_paths]}")

        _stage(f"[AUDITOR] Loading GO OBO for namespaces: {obo_path}")
        graph = obonet.read_obo(obo_path)
        term_to_ns = {node: data.get('namespace', 'unknown') for node, data in graph.nodes(data=True)}

    aspect_map = {
        'biological_process': 'BP',
        'molecular_function': 'MF',
        'cellular_component': 'CC',
    }

    def _aspect_of_term(term: str) -> str:
        return aspect_map.get(term_to_ns.get(term), 'UNK')

    # -----------------------------
    # Load X / X_test (memmap)
    # -----------------------------
    x_path = FEAT_DIR / 'X_train_mmap.npy'
    xt_path = FEAT_DIR / 'X_test_mmap.npy'
    if not x_path.exists() or not xt_path.exists():
        raise FileNotFoundError(f"Missing X memmaps ({x_path} / {xt_path}). Run the Phase 2 setup cell first.")

    X = np.load(x_path, mmap_mode='r')
    X_test = np.load(xt_path, mmap_mode='r')

    # -----------------------------
    # Load / build Y (full 13,500) as memmap
    # -----------------------------
    y_full_path = FEAT_DIR / 'Y_target_13500.npy'
    if y_full_path.exists():
        Y_full = np.load(y_full_path, mmap_mode='r')
    else:
        _stage('[AUDITOR] Building Y_target_13500.npy (disk-backed) ...')
        train_terms = pd.read_parquet(PARSED_DIR / 'train_terms.parquet')
        train_ids_raw = pd.read_feather(PARSED_DIR / 'train_seq.feather')['id'].astype(str)
        train_ids = train_ids_raw.str.extract(r"\|(.*?)\|")[0].fillna(train_ids_raw)

        train_terms_top = train_terms[train_terms['term'].isin(top_terms)]
        Y_df = train_terms_top.pivot_table(index='EntryID', columns='term', aggfunc='size', fill_value=0)
        Y_df = Y_df.reindex(train_ids, fill_value=0)
        Y_df = Y_df.reindex(columns=top_terms, fill_value=0)
        np.save(y_full_path, Y_df.values.astype(np.float32))
        del train_terms, train_ids_raw, train_ids, train_terms_top, Y_df
        gc.collect()
        Y_full = np.load(y_full_path, mmap_mode='r')

    # -----------------------------
    # IA weights (optional)
    # -----------------------------
    weights_full = None
    try:
        ia_candidates = [Path(WORK_ROOT) / 'IA.tsv', FEAT_DIR / 'IA.tsv', Path('IA.tsv')]
        ia_path = next((p for p in ia_candidates if p.exists()), None)
        if ia_path is not None:
            ia = pd.read_csv(ia_path, sep='\t')
            cols = [c.lower() for c in ia.columns]
            term_col = ia.columns[cols.index('term')] if 'term' in cols else ia.columns[0]
            if 'ia' in cols:
                ia_col = ia.columns[cols.index('ia')]
            elif 'information_accretion' in cols:
                ia_col = ia.columns[cols.index('information_accretion')]
            else:
                ia_col = ia.columns[1] if len(ia.columns) > 1 else ia.columns[0]
            ia_map = dict(zip(ia[term_col].astype(str), ia[ia_col].astype(np.float32)))
            weights_full = np.asarray([ia_map.get(t, np.float32(1.0)) for t in top_terms], dtype=np.float32)
            _stage(f"[AUDITOR] Loaded IA weights from {ia_path}")
    except Exception as e:
        _stage(f"[AUDITOR] IA weights unavailable (continuing): {e}")
        weights_full = None

    def _log_mem(msg: str = ''):
        try:
            proc = psutil.Process(os.getpid())
            ram = proc.memory_info().rss / (1024**3)
            avail = psutil.virtual_memory().available / (1024**3)
            if torch.cuda.is_available():
                alloc = torch.cuda.memory_allocated() / (1024**3)
                res = torch.cuda.memory_reserved() / (1024**3)
            else:
                alloc = res = 0.0
            print(f"[MEM] {msg:22s} | RSS: {ram:6.2f}GB | RAM_avail: {avail:6.2f}GB | torch_alloc: {alloc:5.2f}GB torch_res: {res:5.2f}GB")
            sys.stdout.flush()
        except Exception:
            pass

    # Placeholder; replaced after RAPIDS discovery.
    def _log_gpu_mem(msg: str = ''):
        return

    # -----------------------------
    # RAPIDS discovery
    # -----------------------------
    try:
        import cuml  # noqa: F401
        from cuml.linear_model import LogisticRegression as cuLogReg
        from cuml.multiclass import OneVsRestClassifier as cuOVR
        import cupy as cp
        import rmm

        try:
            rmm.reinitialize(managed_memory=True)
            _stage('[AUDITOR] RAPIDS (cuML) detected. RMM Managed Memory ENABLED.')
        except Exception as e:
            _stage(f'[AUDITOR] RAPIDS detected but RMM init failed ({e}); proceeding with default memory.')

        HAS_RAPIDS = True
        _stage(
            f"[AUDITOR] versions: cupy={getattr(cp, '__version__', '?')} cuml={getattr(cuml, '__version__', '?')} rmm={getattr(rmm, '__version__', '?')}"
        )

        def _maybe_bytes(obj, method_name: str):
            fn = getattr(obj, method_name, None)
            if callable(fn):
                try:
                    return int(fn())
                except Exception:
                    return None
            return None

        def _pinned_used_bytes(pinned_pool):
            used = _maybe_bytes(pinned_pool, 'used_bytes')
            if used is not None:
                return used
            # Older CuPy versions sometimes expose total/free but not used.
            total = _maybe_bytes(pinned_pool, 'total_bytes')
            free = _maybe_bytes(pinned_pool, 'free_bytes')
            if total is not None and free is not None:
                return int(total - free)
            return None

        def _log_gpu_mem(msg: str = ''):
            try:
                free_b, total_b = cp.cuda.runtime.memGetInfo()
                pool = cp.get_default_memory_pool()
                pinned = cp.get_default_pinned_memory_pool()

                pool_b = _maybe_bytes(pool, 'used_bytes')
                pinned_b = _pinned_used_bytes(pinned)

                pool_txt = f"{pool_b/1e9:6.2f}GB" if pool_b is not None else "   n/a"
                pinned_txt = f"{pinned_b/1e9:6.2f}GB" if pinned_b is not None else "   n/a"

                print(
                    f"[GPU] {msg:22s} | free {free_b/1e9:6.2f}GB / {total_b/1e9:6.2f}GB | "
                    f"cupy_pool {pool_txt} | pinned {pinned_txt}"
                )
                try:
                    res = rmm.mr.get_current_device_resource()
                    print(f"[GPU] rmm resource: {res}")
                except Exception:
                    pass
                sys.stdout.flush()
            except Exception as e:
                print(f"[GPU] {msg:22s} | unavailable ({e})")
                sys.stdout.flush()

    except Exception:
        HAS_RAPIDS = False
        cp = None  # type: ignore
        _stage('[AUDITOR] RAPIDS NOT detected. Falling back to CPU (slow).')

    def _gpu_mem_okay(n_rows: int, n_cols: int, safety: float = 1.35) -> bool:
        if not (HAS_RAPIDS and torch.cuda.is_available()):
            return False
        try:
            free_b, _total_b = cp.cuda.runtime.memGetInfo()
            need_b = int(n_rows) * int(n_cols) * 4
            ok = free_b > int(safety * need_b)
            if not ok:
                _stage(f"[AUDITOR] VRAM insufficient: free={free_b/1e9:.1f}GB need≈{need_b/1e9:.1f}GB")
            return bool(ok)
        except Exception:
            return True

    # -----------------------------
    # Aspect selection (default BP)
    # -----------------------------
    target_aspect = (os.environ.get('TARGET_ASPECT') or '').strip().upper()
    if not target_aspect:
        target_aspect = (globals().get('TARGET_ASPECT') or '').strip().upper()
    if not target_aspect:
        target_aspect = 'BP'
        _stage('[AUDITOR] TARGET_ASPECT not set -> defaulting to BP (recommended: run one aspect per fresh runtime).')

    if target_aspect not in {'BP', 'MF', 'CC'}:
        raise RuntimeError(
            f"Invalid TARGET_ASPECT={target_aspect!r}. Must be one of: BP, MF, CC."
        )

    aspects = [target_aspect]
    _stage(f"[AUDITOR] Training only aspect: {target_aspect}")

    # Granularity knobs (auditor-aligned)
    TARGET_CHUNK = 100
    VAL_BS = 256
    TEST_BS = 256

    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    tmp_dir = FEAT_DIR / 'tmp_folds_logreg'
    tmp_dir.mkdir(parents=True, exist_ok=True)

    oof_pred_logreg_by_aspect = {}
    test_pred_logreg_by_aspect = {}
    aspect_indices_map = {}

    t0_all = time.time()
    _log_mem('start')
    _log_gpu_mem('start')

    # -----------------------------
    # Performance: materialise X_test into RAM (best-effort)
    # -----------------------------
    # X_test is used repeatedly across folds and target-chunks; keeping it disk-backed can become the bottleneck.
    try:
        _stage('[AUDITOR] LogReg: materialising X_test into RAM for faster inference...')
        X_test_ram = np.ascontiguousarray(np.asarray(X_test, dtype=np.float32))
        _stage(f'[AUDITOR] LogReg: X_test_ram shape={X_test_ram.shape} dtype={X_test_ram.dtype}')
    except MemoryError:
        X_test_ram = X_test
        _stage('[AUDITOR] WARNING: could not materialise X_test (MemoryError); using memmap (slower).')

    # -----------------------------
    # Fold streaming helpers
    # -----------------------------
    def _iter_indexed_rows(src_mm, idx: np.ndarray, bs: int, desc: str):
        for i in tqdm(range(0, int(len(idx)), int(bs)), desc=desc, unit='batch', leave=False):
            j = min(i + int(bs), int(len(idx)))
            rows = idx[i:j]
            xb = np.asarray(src_mm[rows], dtype=np.float32)
            yield i, j, xb

    def _fit_scaler_from_indexed_rows(src_mm, idx: np.ndarray, bs: int, desc: str):
        scaler = StandardScaler(with_mean=True, with_std=True)
        for _i, _j, xb in _iter_indexed_rows(src_mm, idx, bs=bs, desc=desc):
            scaler.partial_fit(xb)
        return {'mean': scaler.mean_.astype(np.float32), 'scale': scaler.scale_.astype(np.float32)}

    def _alloc_and_fill_gpu_from_indexed_rows(src_mm, idx: np.ndarray, mean: np.ndarray, scale: np.ndarray, bs: int, desc: str):
        n_rows = int(len(idx))
        n_cols = int(src_mm.shape[1])
        out = cp.empty((n_rows, n_cols), dtype=cp.float32)
        mean_gpu = cp.asarray(mean)
        scale_gpu = cp.asarray(scale)
        for i, j, xb in _iter_indexed_rows(src_mm, idx, bs=bs, desc=desc):
            xg = cp.asarray(xb)
            xg = (xg - mean_gpu) / (scale_gpu + 1e-12)
            out[i:j, :] = xg
            del xb, xg
            try:
                cp.get_default_memory_pool().free_all_blocks()
                cp.get_default_pinned_memory_pool().free_all_blocks()
            except Exception:
                pass
        return out, mean_gpu, scale_gpu

    def _predict_proba_gpu_batched(clf, src_mm, idx: np.ndarray, mean_gpu, scale_gpu, bs: int, out_np: np.ndarray, out_rows: np.ndarray, col_slice: slice, desc: str):
        for i, j, xb in _iter_indexed_rows(src_mm, idx, bs=bs, desc=desc):
            xg = cp.asarray(xb)
            xg = (xg - mean_gpu) / (scale_gpu + 1e-12)
            p = clf.predict_proba(xg)
            if hasattr(p, 'get'):
                p = p.get()
            elif hasattr(p, 'to_numpy'):
                p = p.to_numpy()
            out_np[out_rows[i:j], col_slice] = np.asarray(p, dtype=np.float32)
            del xb, xg, p
            try:
                cp.get_default_memory_pool().free_all_blocks()
                cp.get_default_pinned_memory_pool().free_all_blocks()
            except Exception:
                pass

    for aspect in aspects:
        _stage(f"\n=== LogReg Aspect: {aspect} ===")

        aspect_indices = [i for i, t in enumerate(top_terms) if _aspect_of_term(t) == aspect]
        if not aspect_indices:
            raise RuntimeError(f"No terms found for aspect {aspect} within top_terms_13500.json")

        aspect_indices = np.asarray(aspect_indices, dtype=np.int64)
        aspect_indices_map[aspect] = aspect_indices
        n_targets = int(aspect_indices.shape[0])
        aspect_terms = [top_terms[i] for i in aspect_indices.tolist()]

        # Slice IA weights for diagnostics only
        weights_aspect = weights_full[aspect_indices] if weights_full is not None else None

        # Persist per-aspect term contract
        (PRED_DIR / f'top_terms_{aspect}.json').write_text(json.dumps(aspect_terms), encoding='utf-8')

        # Per-aspect artefacts
        lr_oof_path = PRED_DIR / f'oof_pred_logreg_{aspect}.npy'
        lr_test_path = PRED_DIR / f'test_pred_logreg_{aspect}.npy'
        marker_dir = PRED_DIR / f'fold_markers_logreg_{aspect}'
        marker_dir.mkdir(parents=True, exist_ok=True)

        # Done check
        if lr_oof_path.exists() and lr_test_path.exists() and (len(list(marker_dir.glob('fold_*_done.flag'))) == n_splits):
            _stage(f"[AUDITOR] {aspect}: all folds completed; loading from disk")
            oof_pred_logreg_by_aspect[aspect] = np.load(lr_oof_path, mmap_mode='r')
            test_pred_logreg_by_aspect[aspect] = np.load(lr_test_path, mmap_mode='r')
            continue

        # Prepare memmaps for this aspect
        mode_oof = 'r+' if lr_oof_path.exists() else 'w+'
        mode_test = 'r+' if lr_test_path.exists() else 'w+'

        oof_pred = np.lib.format.open_memmap(str(lr_oof_path), mode=mode_oof, dtype=np.float32, shape=(X.shape[0], n_targets))
        test_pred = np.lib.format.open_memmap(str(lr_test_path), mode=mode_test, dtype=np.float32, shape=(X_test.shape[0], n_targets))
        if mode_oof == 'w+':
            oof_pred[:] = 0.0
            oof_pred.flush()
        if mode_test == 'w+':
            test_pred[:] = 0.0
            test_pred.flush()

        fold_iter = tqdm(kf.split(np.arange(X.shape[0])), total=kf.get_n_splits(), desc=f'LogReg {aspect} folds', unit='fold')

        for fold, (idx_tr, idx_val) in enumerate(fold_iter):
            marker_path = marker_dir / f'fold_{fold}_done.flag'
            if marker_path.exists():
                _stage(f"{aspect} Fold {fold+1}/{n_splits} already completed (marker found). Skipping.")
                continue

            t0_fold = time.time()
            _stage(f"{aspect} Fold {fold+1}/{n_splits}")
            _log_mem(f"{aspect} fold {fold+1} start")
            _log_gpu_mem(f"{aspect} fold {fold+1} start")

            idx_tr = np.asarray(idx_tr, dtype=np.int64)
            idx_val = np.asarray(idx_val, dtype=np.int64)

            # Performance: materialise fold validation rows into RAM once.
            # This avoids re-reading X (disk-backed) for every target-chunk during predict_proba.
            try:
                X_val_ram = np.ascontiguousarray(np.asarray(X[idx_val], dtype=np.float32))
                val_src = X_val_ram
                val_rows = np.arange(int(len(idx_val)), dtype=np.int64)
                _stage(f'[AUDITOR] {aspect} Fold {fold+1}: X_val_ram in RAM: {X_val_ram.shape}')
            except MemoryError:
                val_src = X
                val_rows = idx_val
                _stage(f'[AUDITOR] {aspect} Fold {fold+1}: WARNING: could not materialise X_val (MemoryError); using memmap (slower).')

            # Fit scaler by streaming fold rows from the global memmap (no fold copies).
            scaler_path = PRED_DIR / f'logreg_scaler_{aspect}_fold{fold}.pkl'
            if scaler_path.exists():
                _stage(f"[AUDIT] {aspect} Fold {fold+1}: Found existing scaler. Loading.")
                scaler_state = joblib.load(scaler_path)
            else:
                t0 = time.time()
                scaler_state = _fit_scaler_from_indexed_rows(X, idx_tr, bs=2048, desc=f'{aspect} Fold {fold+1} Fit Scaler')
                joblib.dump(scaler_state, scaler_path)
                _stage(f"[AUDIT] {aspect} Fold {fold+1}: Scaler fit in {time.time() - t0:.1f}s")

            mean = scaler_state['mean']
            scale = scaler_state['scale']
            _log_mem(f"{aspect} fold {fold+1} post scaler")
            _log_gpu_mem(f"{aspect} fold {fold+1} post scaler")

            # GPU selection: only if we can afford the full training fold on GPU.
            use_gpu = bool(HAS_RAPIDS and _gpu_mem_okay(int(len(idx_tr)), int(X.shape[1])))
            gpu_success = False

            if use_gpu:
                try:
                    _stage(f"[AUDITOR] {aspect} Fold {fold+1}: Using RAPIDS/cuML")

                    _stage(f"[AUDIT] {aspect} Fold {fold+1}: PRE X_tr_gpu build")
                    _log_mem(f"{aspect} pre X_tr_gpu")
                    _log_gpu_mem(f"{aspect} pre X_tr_gpu")

                    # Build X_tr on GPU by streaming scaled batches (avoid full host materialisation).
                    X_tr_gpu, mean_gpu, scale_gpu = _alloc_and_fill_gpu_from_indexed_rows(
                        X,
                        idx_tr,
                        mean=mean,
                        scale=scale,
                        bs=1024,
                        desc=f'{aspect} Fold {fold+1} ->GPU X_tr',
                    )

                    _stage(f"[AUDIT] {aspect} Fold {fold+1}: POST X_tr_gpu build")
                    _log_mem(f"{aspect} post X_tr_gpu")
                    _log_gpu_mem(f"{aspect} post X_tr_gpu")

                    for start in tqdm(
                        range(0, n_targets, TARGET_CHUNK),
                        total=(n_targets + TARGET_CHUNK - 1) // TARGET_CHUNK,
                        desc=f'{aspect} Fold {fold+1} target chunks',
                        unit='chunk',
                        leave=False,
                    ):
                        end = min(start + TARGET_CHUNK, n_targets)

                        if start == 0:
                            _stage(f"[AUDIT] {aspect} Fold {fold+1} chunk0 PRE Y_tr_chunk")
                            _log_mem(f"{aspect} chunk0 pre Y")
                            _log_gpu_mem(f"{aspect} chunk0 pre Y")

                        cols = aspect_indices[start:end]
                        y_host = np.asarray(Y_full[np.ix_(idx_tr, cols)], dtype=np.float32)
                        Y_tr_chunk = cp.asarray(y_host)

                        if start == 0:
                            _stage(f"[AUDIT] {aspect} Fold {fold+1} chunk0 PRE fit")
                            _log_mem(f"{aspect} chunk0 pre fit")
                            _log_gpu_mem(f"{aspect} chunk0 pre fit")

                        clf_chunk = cuOVR(cuLogReg(solver='qn', penalty='l2', C=1.0, max_iter=2000, tol=1e-3))
                        clf_chunk.fit(X_tr_gpu, Y_tr_chunk)

                        if start == 0:
                            _stage(f"[AUDIT] {aspect} Fold {fold+1} chunk0 POST fit")
                            _log_mem(f"{aspect} chunk0 post fit")
                            _log_gpu_mem(f"{aspect} chunk0 post fit")

                        # Validation probs (batched) -> write directly into oof memmap
                        _predict_proba_gpu_batched(
                            clf_chunk,
                            val_src,
                            val_rows,
                            mean_gpu,
                            scale_gpu,
                            bs=VAL_BS,
                            out_np=oof_pred,
                            out_rows=idx_val,
                            col_slice=slice(start, end),
                            desc=f'{aspect} Fold {fold+1} val proba',
                        )

                        # Test probs (batched) -> accumulate
                        for b0 in range(0, int(X_test_ram.shape[0]), TEST_BS):
                            b1 = min(b0 + TEST_BS, int(X_test_ram.shape[0]))
                            xb = np.asarray(X_test_ram[b0:b1], dtype=np.float32)
                            xb_gpu = cp.asarray(xb)
                            xb_gpu = (xb_gpu - mean_gpu) / (scale_gpu + 1e-12)

                            p_te = clf_chunk.predict_proba(xb_gpu)
                            if hasattr(p_te, 'get'):
                                p_te = p_te.get()
                            elif hasattr(p_te, 'to_numpy'):
                                p_te = p_te.to_numpy()

                            test_pred[b0:b1, start:end] += (np.asarray(p_te, dtype=np.float32) / float(n_splits))
                            del xb, xb_gpu, p_te

                        del y_host, Y_tr_chunk, clf_chunk
                        try:
                            cp.get_default_memory_pool().free_all_blocks()
                            cp.get_default_pinned_memory_pool().free_all_blocks()
                        except Exception:
                            pass
                        gc.collect()

                    del X_tr_gpu, mean_gpu, scale_gpu
                    try:
                        cp.get_default_memory_pool().free_all_blocks()
                        cp.get_default_pinned_memory_pool().free_all_blocks()
                    except Exception:
                        pass
                    gc.collect()
                    gpu_success = True

                except Exception as e:
                    _stage(f"[CRITICAL] {aspect} Fold {fold+1}: GPU Training Failed (likely OOM): {e}")
                    _stage('[AUDITOR] Cleaning up GPU memory and falling back to CPU...')
                    gpu_success = False
                    if HAS_RAPIDS and cp is not None:
                        try:
                            cp.get_default_memory_pool().free_all_blocks()
                            cp.get_default_pinned_memory_pool().free_all_blocks()
                        except Exception:
                            pass
                    gc.collect()

            if (not use_gpu) or (not gpu_success):
                _stage(f"[AUDITOR] {aspect} Fold {fold+1}: Using CPU SGD (fallback)")

                # CPU fallback: build one scaled X_tr memmap, then fit/predict target chunks.
                X_trs_path = tmp_dir / f'X_tr_scaled_{aspect}_fold{fold}.npy'
                if not X_trs_path.exists():
                    mm = np.lib.format.open_memmap(str(X_trs_path), mode='w+', dtype=np.float32, shape=(len(idx_tr), X.shape[1]))
                    for i, j, xb in _iter_indexed_rows(X, idx_tr, bs=2048, desc=f'{aspect} Fold {fold+1} scale X_tr (CPU)'):
                        xb = (xb - mean) / (scale + 1e-12)
                        mm[i:j, :] = xb
                    mm.flush()
                    del mm
                    gc.collect()

                X_trs = np.load(X_trs_path, mmap_mode='r')

                for start in tqdm(
                    range(0, n_targets, TARGET_CHUNK),
                    total=(n_targets + TARGET_CHUNK - 1) // TARGET_CHUNK,
                    desc=f'{aspect} Fold {fold+1} CPU target chunks',
                    unit='chunk',
                    leave=False,
                ):
                    end = min(start + TARGET_CHUNK, n_targets)
                    cols = aspect_indices[start:end]
                    Y_tr_chunk = np.asarray(Y_full[np.ix_(idx_tr, cols)], dtype=np.float32)

                    clf_logreg = OneVsRestClassifier(
                        SGDClassifier(loss='log_loss', penalty='l2', alpha=0.0001, max_iter=1, tol=None, n_jobs=4),
                        n_jobs=-1,
                    )
                    clf_logreg.fit(X_trs, Y_tr_chunk)

                    # Validation preds (batched, scale on the fly)
                    for i, j, xb in _iter_indexed_rows(val_src, val_rows, bs=VAL_BS, desc=f'{aspect} Fold {fold+1} val predict (CPU)'):
                        xb = (xb - mean) / (scale + 1e-12)
                        pb = clf_logreg.predict_proba(xb).astype(np.float32)
                        oof_pred[idx_val[i:j], start:end] = pb

                    # Test preds (batched)
                    for b0 in tqdm(range(0, int(X_test_ram.shape[0]), TEST_BS), desc=f'{aspect} Fold {fold+1} test predict (CPU)', unit='batch', leave=False):
                        b1 = min(b0 + TEST_BS, int(X_test_ram.shape[0]))
                        xb = np.asarray(X_test_ram[b0:b1], dtype=np.float32)
                        xb = (xb - mean) / (scale + 1e-12)
                        pb = clf_logreg.predict_proba(xb).astype(np.float32)
                        test_pred[b0:b1, start:end] += pb / float(n_splits)

                    del Y_tr_chunk, clf_logreg
                    gc.collect()

                del X_trs
                gc.collect()

                # Cleanup temp scaled fold
                try:
                    os.remove(X_trs_path)
                except Exception:
                    pass

            # Fold diagnostics (sampled, capped columns to avoid huge allocations)
            try:
                sample_n = int(min(20000, len(idx_val)))
                sample_k = int(min(2000, n_targets))
                if sample_n > 0 and sample_k > 0:
                    sample_probs = np.asarray(oof_pred[idx_val[:sample_n], :sample_k], dtype=np.float32)
                    cols = aspect_indices[:sample_k]
                    sample_true = np.asarray(Y_full[np.ix_(idx_val[:sample_n], cols)], dtype=np.float32)

                    best_f1 = 0.0
                    best_thr = 0.0
                    for thr in np.linspace(0.01, 0.20, 20):
                        vp = (sample_probs > thr).astype(np.int8)
                        score = f1_score(sample_true, vp, average='micro')
                        if score > best_f1:
                            best_f1, best_thr = score, float(thr)

                    _stage(f"  >> {aspect} Fold {fold+1} (sample) micro-F1={best_f1:.4f} best_thr={best_thr:.2f} (k={sample_k})")
            except Exception as e:
                _stage('  [WARNING] Diagnostics skipped: ' + repr(e))

            oof_pred.flush()
            test_pred.flush()
            marker_path.touch()
            _stage(f"{aspect} Fold {fold+1} completed and flushed.")
            _stage(f"[TIMER] {aspect} Fold {fold+1} wall: {time.time() - t0_fold:.1f}s")
            _log_mem(f"{aspect} fold {fold+1} end")
            _log_gpu_mem(f"{aspect} fold {fold+1} end")

        oof_pred.flush()
        test_pred.flush()
        del oof_pred, test_pred
        gc.collect()

        oof_pred_logreg_by_aspect[aspect] = np.load(lr_oof_path, mmap_mode='r')
        test_pred_logreg_by_aspect[aspect] = np.load(lr_test_path, mmap_mode='r')
        _stage(f"[AUDITOR] {aspect}: saved {lr_oof_path.name}, {lr_test_path.name}")

    # -----------------------------
    # Backwards compatibility: combined outputs are only assembled once BP+MF+CC exist.
    # -----------------------------
    if set(['BP', 'MF', 'CC']).issubset(set(oof_pred_logreg_by_aspect.keys())):
        lr_oof_full_path = PRED_DIR / 'oof_pred_logreg.npy'
        lr_test_full_path = PRED_DIR / 'test_pred_logreg.npy'

        mode_oof = 'r+' if lr_oof_full_path.exists() else 'w+'
        mode_test = 'r+' if lr_test_full_path.exists() else 'w+'
        oof_full = np.lib.format.open_memmap(str(lr_oof_full_path), mode=mode_oof, dtype=np.float32, shape=(X.shape[0], len(top_terms)))
        te_full = np.lib.format.open_memmap(str(lr_test_full_path), mode=mode_test, dtype=np.float32, shape=(X_test.shape[0], len(top_terms)))
        if mode_oof == 'w+':
            oof_full[:] = 0.0
        if mode_test == 'w+':
            te_full[:] = 0.0

        for asp in ['BP', 'MF', 'CC']:
            idx = aspect_indices_map[asp]
            oof_full[:, idx] = np.asarray(oof_pred_logreg_by_aspect[asp], dtype=np.float32)
            te_full[:, idx] = np.asarray(test_pred_logreg_by_aspect[asp], dtype=np.float32)

        oof_full.flush()
        te_full.flush()
        del oof_full, te_full
        gc.collect()

        oof_pred_logreg = np.load(lr_oof_full_path, mmap_mode='r')
        test_pred_logreg = np.load(lr_test_full_path, mmap_mode='r')
        _stage(f"[AUDITOR] Combined preds saved: {lr_oof_full_path.name}, {lr_test_full_path.name}")
    else:
        oof_pred_logreg = None
        test_pred_logreg = None

    # Final checkpoint push (per-aspect; combined only if present)
    if 'STORE' in globals() and STORE is not None:
        required = [str(top_terms_path.as_posix())]
        for asp in oof_pred_logreg_by_aspect.keys():
            required += [
                str((PRED_DIR / f'oof_pred_logreg_{asp}.npy').as_posix()),
                str((PRED_DIR / f'test_pred_logreg_{asp}.npy').as_posix()),
                str((PRED_DIR / f'top_terms_{asp}.json').as_posix()),
            ]
        if oof_pred_logreg is not None and test_pred_logreg is not None:
            required += [
                str((PRED_DIR / 'oof_pred_logreg.npy').as_posix()),
                str((PRED_DIR / 'test_pred_logreg.npy').as_posix()),
            ]

        try:
            STORE.maybe_push(
                stage='stage_07a_level1_logreg_aspect_split',
                required_paths=required,
                note='Level-1 Logistic Regression predictions (OOF + test), split by GO aspect (BP/MF/CC).',
            )
        except Exception as e:
            _stage(f"[WARN] STORE push failed: {e}")

    _stage(f"[TIMER] LogReg total wall: {time.time() - t0_all:.1f}s")


In [ ]:
# CELL 13D - DNN (PyTorch, multi-branch per modality) + checkpoint push
# ===================================================================
# Rank-1 style correction: each modality gets its own head before fusion.
# Extreme ensembling correction: 5 seeds × 5 folds = 25 models.
# Implementation guardrails:
#   - IA-weighted BCE is mandatory (class_weight per term).
#   - Outputs must remain full 13,500 columns for the Phase 3 GCN contract.

if not TRAIN_LEVEL1:
    print('Skipping DNN (TRAIN_LEVEL1=False).')
else:
    import gc
    import json
    from pathlib import Path
    import numpy as np
    import pandas as pd
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from sklearn.model_selection import KFold

    WORK_ROOT = Path(WORK_ROOT)
    FEAT_DIR = WORK_ROOT / 'features'
    PRED_DIR = FEAT_DIR / 'level1_preds'
    PRED_DIR.mkdir(parents=True, exist_ok=True)

    dnn_oof_path = PRED_DIR / 'oof_pred_dnn.npy'
    dnn_test_path = PRED_DIR / 'test_pred_dnn.npy'

    if dnn_oof_path.exists() and dnn_test_path.exists():
        oof_pred_dnn = np.load(dnn_oof_path)
        test_pred_dnn = np.load(dnn_test_path)
        print('Loaded existing DNN preds:', dnn_oof_path.name, dnn_test_path.name)
    else:
        if 'features_train' not in globals() or 'features_test' not in globals():
            raise RuntimeError('Missing `features_train`/`features_test`. Run Cell 13a first (Phase 2 setup).')
        if 'Y' not in globals():
            raise RuntimeError('Missing Y. Run Cell 13a first (targets).')

        # ---- IA class weights (per-term) ----
        top_terms_path = FEAT_DIR / 'top_terms_13500.json'
        if not top_terms_path.exists():
            raise FileNotFoundError(f'Missing {top_terms_path}. Run Cell 13a first.')
        top_terms = [str(t) for t in json.loads(top_terms_path.read_text(encoding='utf-8'))]

        ia_path = WORK_ROOT / 'IA.tsv'
        if not ia_path.exists():
            raise FileNotFoundError(f'Missing IA.tsv at {ia_path}')
        ia_df = pd.read_csv(ia_path, sep='\t')
        # robust column naming (some IA files use 'term' or '#term')
        term_col = 'term' if 'term' in ia_df.columns else ('#term' if '#term' in ia_df.columns else ia_df.columns[0])
        ia_col = 'ia' if 'ia' in ia_df.columns else (ia_df.columns[1] if len(ia_df.columns) > 1 else ia_df.columns[0])
        ia_map = dict(zip(ia_df[term_col].astype(str).values, ia_df[ia_col].astype(np.float32).values))
        weights = np.asarray([ia_map.get(t, np.float32(1.0)) for t in top_terms], dtype=np.float32)
        # weights broadcast over classes: (1, L)
        w_t = torch.from_numpy(weights).view(1, -1)
        print(f'[DNN] IA weights ready: shape={weights.shape} min={float(weights.min()):.4f} max={float(weights.max()):.4f}')

        # ---- label contract (must stay 13,500-wide) ----
        out_dim = int(Y.shape[1])
        if out_dim != len(top_terms):
            raise RuntimeError(f'DNN label contract mismatch: Y has {out_dim} cols but top_terms has {len(top_terms)}')
        if out_dim != 13500:
            raise RuntimeError(f'DNN expects 13,500 labels; got out_dim={out_dim}')

        # ---- modality inputs ----
        # Use per-modality arrays directly (no flat X copy).
        required_keys = ['t5', 'esm2_650m', 'esm2_3b', 'ankh', 'text', 'taxa']
        missing = [k for k in required_keys if k not in features_train]
        if missing:
            raise FileNotFoundError(f'Missing mandatory DNN modalities: {missing}. Run embeddings/text/taxa stages to materialise them.')

        # Optional 7th branch: PB/GBDT probabilities (teacher features).
        # If Cell 13b already produced OOF/test predictions, treat them as an additional modality input.
        # This is leakage-safe because we use OOF for train rows and a proper test_pred for test rows.
        gbdt_oof_path = PRED_DIR / 'oof_pred_gbdt.npy'
        gbdt_test_path = PRED_DIR / 'test_pred_gbdt.npy'
        use_pb = bool(gbdt_oof_path.exists() and gbdt_test_path.exists())

        # Build local feature dicts for the DNN (may include pb)
        dnn_train = dict(features_train)
        dnn_test = dict(features_test)
        dnn_keys = list(required_keys)

        if use_pb:
            pb_oof = np.load(gbdt_oof_path, mmap_mode='r')
            pb_test = np.load(gbdt_test_path, mmap_mode='r')
            if int(pb_oof.shape[0]) != int(Y.shape[0]):
                raise RuntimeError(f'PB/GBDT OOF rows mismatch: {pb_oof.shape[0]} vs Y rows {Y.shape[0]}')
            if int(pb_oof.shape[1]) != out_dim or int(pb_test.shape[1]) != out_dim:
                raise RuntimeError(f'PB/GBDT pred cols mismatch: expected {out_dim}, got oof={pb_oof.shape}, test={pb_test.shape}')
            dnn_train['pb'] = pb_oof
            dnn_test['pb'] = pb_test
            dnn_keys.append('pb')
            print('[DNN] Using 7th modality branch: pb (= GBDT OOF/test probabilities)')
        else:
            print('[DNN] PB/GBDT branch not available (missing GBDT OOF/test preds); proceeding with 6 modalities.')

        print(f'DNN modality heads: {dnn_keys} (n={len(dnn_keys)})')
        dims = {k: int(dnn_train[k].shape[1]) for k in dnn_keys}

        class MultiModalDataset(Dataset):
            def __init__(self, X_dict, y, keys, idx):
                self.X_dict = X_dict
                self.y = y
                self.keys = keys
                self.idx = np.asarray(idx, dtype=np.int64)
            def __len__(self):
                return int(self.idx.shape[0])
            def __getitem__(self, i):
                j = int(self.idx[i])
                xs = [np.asarray(self.X_dict[k][j], dtype=np.float32) for k in self.keys]
                if self.y is None:
                    return xs
                yy = np.asarray(self.y[j], dtype=np.float32)
                return xs, yy

        def _collate(batch):
            # batch: list of (xs, y) or xs
            if isinstance(batch[0], tuple):
                xs_list, ys = zip(*batch)
                xs_by_key = list(zip(*xs_list))
                xs_t = [torch.from_numpy(np.stack(v, axis=0)) for v in xs_by_key]
                y_t = torch.from_numpy(np.stack(ys, axis=0))
                return xs_t, y_t
            else:
                xs_by_key = list(zip(*batch))
                xs_t = [torch.from_numpy(np.stack(v, axis=0)) for v in xs_by_key]
                return xs_t

        class ModalityHead(nn.Module):
            def __init__(self, in_dim: int, p: float = 0.2):
                super().__init__()
                # Keep heads uniform but avoid huge parameter blow-ups for very wide modalities
                if int(in_dim) >= 8000:
                    hidden1, hidden2 = 1024, 1024
                elif int(in_dim) >= 2000:
                    hidden1, hidden2 = 2048, 1024
                else:
                    hidden1, hidden2 = 1024, 512
                self.out_dim = int(hidden2)
                self.net = nn.Sequential(
                    nn.Linear(int(in_dim), int(hidden1)),
                    nn.BatchNorm1d(int(hidden1)),
                    nn.ReLU(),
                    nn.Dropout(p),
                    nn.Linear(int(hidden1), int(hidden2)),
                    nn.BatchNorm1d(int(hidden2)),
                    nn.ReLU(),
                    nn.Dropout(p),
                )
            def forward(self, x):
                return self.net(x)

        class MultiBranchDNN(nn.Module):
            def __init__(self, dims_by_key: dict, out_dim: int, p: float = 0.2):
                super().__init__()
                self.keys = list(dims_by_key.keys())
                self.heads = nn.ModuleDict({k: ModalityHead(in_dim=int(dims_by_key[k]), p=p) for k in self.keys})
                fused_dim = int(sum(self.heads[k].out_dim for k in self.keys))
                self.trunk = nn.Sequential(
                    nn.Linear(fused_dim, 2048),
                    nn.BatchNorm1d(2048),
                    nn.ReLU(),
                    nn.Dropout(p),
                    nn.Linear(2048, out_dim),
                )
            def forward(self, xs):
                hs = []
                for k, x in zip(self.keys, xs):
                    hs.append(self.heads[k](x))
                h = torch.cat(hs, dim=1)
                return self.trunk(h)

        def train_one_seed_fold(train_idx, val_idx, seed: int, epochs: int, batch_size: int, lr: float, device: torch.device):
            torch.manual_seed(42 + seed)
            np.random.seed(42 + seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            model = MultiBranchDNN(dims_by_key=dims, out_dim=out_dim, p=0.2).to(device)
            opt = torch.optim.AdamW(model.parameters(), lr=lr)
            w = w_t.to(device)

            ds_tr = MultiModalDataset(dnn_train, Y, dnn_keys, train_idx)
            ds_va = MultiModalDataset(dnn_train, Y, dnn_keys, val_idx)
            dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=0, collate_fn=_collate)
            dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=_collate)

            for ep in range(1, epochs + 1):
                model.train()
                for xs, yb in dl_tr:
                    xs = [x.to(device, non_blocking=True).float() for x in xs]
                    yb = yb.to(device, non_blocking=True).float()
                    logits = model(xs)
                    # IA-weighted BCE: class weights applied per term (broadcast over batch)
                    loss_per = F.binary_cross_entropy_with_logits(logits, yb, reduction='none')
                    loss = (loss_per * w).mean()
                    opt.zero_grad(set_to_none=True)
                    loss.backward()
                    opt.step()

                # quick val metric (kept simple to avoid extra compute)
                if 'ia_weighted_f1' in globals():
                    model.eval()
                    with torch.no_grad():
                        va_scores = []
                        va_true = []
                        for xs, yb in dl_va:
                            xs = [x.to(device, non_blocking=True).float() for x in xs]
                            logits = model(xs)
                            va_scores.append(torch.sigmoid(logits).cpu().numpy())
                            va_true.append(yb.numpy())
                        va_scores = np.vstack(va_scores)
                        va_true = np.vstack(va_true)
                        s = ia_weighted_f1(va_true, va_scores, thr=0.3)
                    print(f'  seed={seed} ep={ep}/{epochs} IA-F1={s}')

            return model

        def predict_on_split(model: nn.Module, X_dict, idx, batch_size: int, device: torch.device):
            ds = MultiModalDataset(X_dict, None, dnn_keys, idx)
            dl = DataLoader(ds, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=_collate)
            preds = []
            model.eval()
            with torch.no_grad():
                for xs in dl:
                    xs = [x.to(device, non_blocking=True).float() for x in xs]
                    logits = model(xs)
                    preds.append(torch.sigmoid(logits).cpu().numpy())
            return np.vstack(preds).astype(np.float32)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print('DNN device:', device)

        n_splits = 5
        n_seeds = 5
        epochs = 3
        batch_size = 256
        lr = 1e-3

        oof_pred_dnn = np.zeros((int(Y.shape[0]), out_dim), dtype=np.float32)
        test_n = int(dnn_test[dnn_keys[0]].shape[0])
        test_pred_dnn = np.zeros((test_n, out_dim), dtype=np.float32)
        counts = np.zeros((int(Y.shape[0]), 1), dtype=np.float32)

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        for fold, (tr_idx, va_idx) in enumerate(kf.split(np.arange(int(Y.shape[0]))), start=1):
            print(f'Fold {fold}/{n_splits} (DNN multi-branch)')
            fold_test = np.zeros_like(test_pred_dnn)
            for seed in range(n_seeds):
                print(f'  seed {seed+1}/{n_seeds}')
                model = train_one_seed_fold(tr_idx, va_idx, seed=seed, epochs=epochs, batch_size=batch_size, lr=lr, device=device)

                # OOF preds
                preds_va = predict_on_split(model, dnn_train, va_idx, batch_size=1024, device=device)
                oof_pred_dnn[va_idx] += preds_va
                counts[va_idx] += 1.0

                # TEST preds
                te_idx = np.arange(test_n, dtype=np.int64)
                preds_te = predict_on_split(model, dnn_test, te_idx, batch_size=1024, device=device)
                fold_test += preds_te

                # free VRAM aggressively (important on Colab/Kaggle)
                del model
                gc.collect()
                if device.type == 'cuda':
                    torch.cuda.empty_cache()

            test_pred_dnn += (fold_test / float(n_seeds))

        oof_pred_dnn = (oof_pred_dnn / np.maximum(counts, 1.0)).astype(np.float32)
        test_pred_dnn = (test_pred_dnn / float(n_splits)).astype(np.float32)

        # Contract guardrail: MUST remain (n_train, 13500) and (n_test, 13500)
        if int(oof_pred_dnn.shape[1]) != 13500 or int(test_pred_dnn.shape[1]) != 13500:
            raise RuntimeError(f'DNN output contract violated: oof={oof_pred_dnn.shape} test={test_pred_dnn.shape}')

        np.save(dnn_oof_path, oof_pred_dnn)
        np.save(dnn_test_path, test_pred_dnn)
        print('Saved:', dnn_oof_path)
        print('Saved:', dnn_test_path)

    # Checkpoint push (always)
    STORE.maybe_push(
        stage='stage_07c_level1_dnn',
        required_paths=[
            str((WORK_ROOT / 'features' / 'top_terms_13500.json').as_posix()),
            str(dnn_oof_path.as_posix()),
            str(dnn_test_path.as_posix()),
        ],
        note='Level-1 DNN predictions (OOF + test).',
    )

    # Diagnostics: probability histograms + IA-F1 vs threshold (sampled)
    try:
        import os
        import numpy as np
        import matplotlib.pyplot as plt
        plt.rcParams.update({'font.size': 12})
        DIAG_N = int(os.getenv('CAFA_DIAG_N', '20000'))
        def _sub(y_true: np.ndarray, y_score: np.ndarray):
            n = int(y_true.shape[0])
            m = min(n, int(DIAG_N))
            if m <= 0:
                return y_true[:0], y_score[:0]
            idx = np.linspace(0, n - 1, num=m, dtype=np.int64)
            return y_true[idx], y_score[idx]
        y_t, y_s = _sub(Y, oof_pred_dnn)
        row_max_oof = y_s.max(axis=1)
        row_mean_oof = y_s.mean(axis=1)
        plt.figure(figsize=(10, 4))
        plt.hist(row_max_oof, bins=60, alpha=0.7)
        plt.title('DNN OOF: max probability per protein (sampled)')
        plt.xlabel('max prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()
        plt.figure(figsize=(10, 4))
        plt.hist(row_mean_oof, bins=60, alpha=0.7)
        plt.title('DNN OOF: mean probability per protein (sampled)')
        plt.xlabel('mean prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()
        if test_pred_dnn is not None:
            te_s = test_pred_dnn
            te_m = min(int(te_s.shape[0]), int(DIAG_N))
            te_idx = np.linspace(0, int(te_s.shape[0]) - 1, num=te_m, dtype=np.int64) if te_m > 0 else np.array([], dtype=np.int64)
            row_max_te = te_s[te_idx].max(axis=1) if te_m > 0 else np.array([], dtype=np.float32)
            plt.figure(figsize=(10, 4))
            plt.hist(row_max_oof, bins=60, alpha=0.5, label='OOF')
            plt.hist(row_max_te, bins=60, alpha=0.5, label='test')
            plt.title('DNN: max probability per protein (OOF vs test; sampled)')
            plt.xlabel('max prob')
            plt.ylabel('count')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.show()
        if 'ia_weighted_f1' in globals():
            thrs = np.linspace(0.05, 0.60, 23)
            curves = {k: [] for k in ['ALL', 'MF', 'BP', 'CC']}
            for thr in thrs:
                s = ia_weighted_f1(y_t, y_s, thr=float(thr))
                for k in curves.keys():
                    curves[k].append(s[k])
            plt.figure(figsize=(10, 3))
            for k in ['ALL', 'MF', 'BP', 'CC']:
                plt.plot(thrs, curves[k], label=k)
            plt.title('DNN OOF: IA-weighted F1 vs threshold (sampled)')
            plt.xlabel('threshold')
            plt.ylabel('IA-F1')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
    except Exception as e:
        print('DNN diagnostics skipped:', repr(e))


In [ ]:
# CELL 13E - KNN (cosine; ESM2-3B) + checkpoint push
if not TRAIN_LEVEL1:
    print('Skipping KNN (TRAIN_LEVEL1=False).')
else:
    from sklearn.neighbors import NearestNeighbors
    from sklearn.model_selection import KFold
    import json
    PRED_DIR = WORK_ROOT / 'features' / 'level1_preds'
    PRED_DIR.mkdir(parents=True, exist_ok=True)
    knn_oof_path = PRED_DIR / 'oof_pred_knn.npy'
    knn_test_path = PRED_DIR / 'test_pred_knn.npy'
    # Backwards-compatible copies (some downstream code loads from WORK_ROOT/features)
    knn_oof_compat = WORK_ROOT / 'features' / 'oof_pred_knn.npy'
    knn_test_compat = WORK_ROOT / 'features' / 'test_pred_knn.npy'
    if knn_oof_path.exists() and knn_test_path.exists():
        oof_pred_knn = np.load(knn_oof_path)
        test_pred_knn = np.load(knn_test_path)
        oof_max_sim = None
    else:
        if 'features_train' not in globals() or 'features_test' not in globals():
            raise RuntimeError('Missing `features_train`/`features_test`. Run the Phase 2 feature load cell first.')
        if 'esm2_3b' not in features_train:
            raise FileNotFoundError("Missing required modality 'esm2_3b' in features_train. Ensure features/train_embeds_esm2_3b.npy exists.")
        X_knn = features_train['esm2_3b'].astype(np.float32)
        X_knn_test = features_test['esm2_3b'].astype(np.float32)
        # Enforce TOP_K alignment using the persisted term list.
        top_terms_path = WORK_ROOT / 'features' / 'top_terms_13500.json'
        if top_terms_path.exists():
            top_terms_knn = json.loads(top_terms_path.read_text())
            if Y.shape[1] != len(top_terms_knn):
                raise ValueError(
                    f'KNN shape mismatch: Y has {Y.shape[1]} cols but top_terms_13500.json has {len(top_terms_knn)} terms.'
                )
        else:
            # Fall back to in-memory top_terms if present (should be created in Phase 2 cell).
            if 'top_terms' not in globals():
                raise RuntimeError('Missing top_terms_13500.json and in-memory top_terms. Run the Phase 2 cell first.')
            top_terms_knn = list(top_terms)
            if Y.shape[1] != len(top_terms_knn):
                raise ValueError(
                    f'KNN shape mismatch: Y has {Y.shape[1]} cols but top_terms has {len(top_terms_knn)} terms.'
                )
        # KNN needs binary targets (presence/absence), not counts.
        Y_knn = (Y > 0).astype(np.float32)
        def _l2_norm(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
            n = np.linalg.norm(x, axis=1, keepdims=True)
            return x / np.maximum(n, eps)
        # Cosine distance is best-behaved on L2-normalised vectors
        X_knn = _l2_norm(X_knn)
        X_knn_test = _l2_norm(X_knn_test)
        KNN_K = int(globals().get('KNN_K', 50))
        KNN_BATCH = int(globals().get('KNN_BATCH', 256))
        n_splits = 5
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        oof_pred_knn = np.zeros((X_knn.shape[0], Y_knn.shape[1]), dtype=np.float32)
        test_pred_knn = np.zeros((X_knn_test.shape[0], Y_knn.shape[1]), dtype=np.float32)
        oof_max_sim = np.zeros((X_knn.shape[0],), dtype=np.float32)
        for fold, (tr_idx, va_idx) in enumerate(kf.split(X_knn), start=1):
            print(f'Fold {fold}/{n_splits} (KNN)')
            knn = NearestNeighbors(n_neighbors=KNN_K, metric='cosine', n_jobs=-1)
            knn.fit(X_knn[tr_idx])
            dists, neigh = knn.kneighbors(X_knn[va_idx], return_distance=True)
            sims = np.clip((1.0 - dists).astype(np.float32), 0.0, 1.0)
            oof_max_sim[va_idx] = sims.max(axis=1)
            neigh_global = tr_idx[neigh]  # map to global row indices into Y_knn
            for i in range(0, len(va_idx), KNN_BATCH):
                j = min(i + KNN_BATCH, len(va_idx))
                neigh_b = neigh_global[i:j]
                sims_b = sims[i:j]
                denom = np.maximum(sims_b.sum(axis=1, keepdims=True), 1e-8)
                Y_nei = Y_knn[neigh_b]  # (B, K, L)
                scores = (np.einsum('bk,bkl->bl', sims_b, Y_nei) / denom).astype(np.float32)
                oof_pred_knn[va_idx[i:j]] = scores
            if 'ia_weighted_f1' in globals():
                print('  IA-F1:', ia_weighted_f1(Y_knn[va_idx], oof_pred_knn[va_idx], thr=0.3))
        # Final model on full train -> test
        knn_final = NearestNeighbors(n_neighbors=KNN_K, metric='cosine', n_jobs=-1)
        knn_final.fit(X_knn)
        dists_te, neigh_te = knn_final.kneighbors(X_knn_test, return_distance=True)
        sims_te = np.clip((1.0 - dists_te).astype(np.float32), 0.0, 1.0)
        denom_te = np.maximum(sims_te.sum(axis=1, keepdims=True), 1e-8)
        for i in range(0, X_knn_test.shape[0], KNN_BATCH):
            j = min(i + KNN_BATCH, X_knn_test.shape[0])
            neigh_b = neigh_te[i:j]
            sims_b = sims_te[i:j]
            Y_nei = Y_knn[neigh_b]
            scores = (np.einsum('bk,bkl->bl', sims_b, Y_nei) / denom_te[i:j]).astype(np.float32)
            test_pred_knn[i:j] = scores
        np.save(knn_oof_path, oof_pred_knn)
        np.save(knn_test_path, test_pred_knn)
        np.save(knn_oof_compat, oof_pred_knn)
        np.save(knn_test_compat, test_pred_knn)
        print('Saved:', knn_oof_path)
        print('Saved:', knn_test_path)
        print('Saved:', knn_oof_compat)
        print('Saved:', knn_test_compat)
    # Checkpoint push (always)
    STORE.maybe_push(
        stage='stage_07d_level1_knn',
        required_paths=[
            str((WORK_ROOT / 'features' / 'top_terms_13500.json').as_posix()),
            str(knn_oof_path.as_posix()),
            str(knn_test_path.as_posix()),
        ],
        note='Level-1 KNN (cosine) predictions using ESM2-3B embeddings (OOF + test).',
    )
    # Diagnostics: similarity distribution + IA-F1 vs threshold
    try:
        import matplotlib.pyplot as plt
        if oof_max_sim is not None:
            plt.figure(figsize=(10, 4))
            plt.hist(oof_max_sim, bins=50)
            plt.title('KNN OOF diagnostic: max cosine similarity to neighbours (per protein)')
            plt.xlabel('max similarity')
            plt.ylabel('count')
            plt.grid(True, alpha=0.3)
            plt.show()
        if 'ia_weighted_f1' in globals():
            thrs = np.linspace(0.05, 0.60, 23)
            curves = {k: [] for k in ['ALL', 'MF', 'BP', 'CC']}
            for thr in thrs:
                s = ia_weighted_f1(Y, oof_pred_knn, thr=float(thr))
                for k in curves.keys():
                    curves[k].append(s[k])
            plt.figure(figsize=(10, 3))
            for k in ['ALL', 'MF', 'BP', 'CC']:
                plt.plot(thrs, curves[k], label=k)
            plt.title('KNN OOF: IA-weighted F1 vs threshold')
            plt.xlabel('threshold')
            plt.ylabel('IA-F1')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
    except Exception as e:
        print('KNN diagnostics skipped:', repr(e))


In [ ]:
# CELL 13F - Calibrate per-aspect thresholds (BP/MF/CC) from OOF (read-only)
# ========================================================================
# Goal: derive thresholds per GO aspect and persist to features/aspect_thresholds.json.
# These thresholds are used later in Phase 3/4 (stacking + submission filtering).
#
# We fit thresholds on a deterministic subsample of rows to keep runtime bounded.
# This is *calibration*, not training.

import json
from pathlib import Path
import numpy as np
import pandas as pd

WORK_ROOT = Path(WORK_ROOT)
FEAT_DIR = WORK_ROOT / 'features'
PRED_DIR = FEAT_DIR / 'level1_preds'
FEAT_DIR.mkdir(parents=True, exist_ok=True)
PRED_DIR.mkdir(parents=True, exist_ok=True)

thr_path = FEAT_DIR / 'aspect_thresholds.json'
thr_meta_path = FEAT_DIR / 'aspect_thresholds_meta.json'

if thr_path.exists():
    thr_map = json.loads(thr_path.read_text(encoding='utf-8'))
    print('Loaded existing aspect thresholds:', thr_map)
else:
    # ---- term contract ----
    top_terms_path = FEAT_DIR / 'top_terms_13500.json'
    if not top_terms_path.exists():
        raise FileNotFoundError(f'Missing {top_terms_path}. Run Phase 2 setup first.')
    top_terms = [str(t) for t in json.loads(top_terms_path.read_text(encoding='utf-8'))]
    if len(top_terms) != 13500:
        raise RuntimeError(f'Expected top_terms=13500, got {len(top_terms)}')

    # ---- GO namespaces -> aspects ----
    if 'go_namespaces' in globals() and isinstance(globals()['go_namespaces'], dict):
        go_namespaces = globals()['go_namespaces']
    elif 'term_to_ns' in globals() and isinstance(globals()['term_to_ns'], dict):
        go_namespaces = globals()['term_to_ns']
    else:
        import obonet
        obo_path = None
        for p in [WORK_ROOT / 'Train' / 'go-basic.obo', WORK_ROOT / 'go-basic.obo', Path('Train/go-basic.obo'), Path('go-basic.obo')]:
            if p.exists():
                obo_path = p
                break
        if obo_path is None:
            raise FileNotFoundError('go-basic.obo not found for namespace mapping')
        graph = obonet.read_obo(obo_path)
        go_namespaces = {node: data.get('namespace', 'unknown') for node, data in graph.nodes(data=True)}

    ns_to_aspect = {
        'biological_process': 'BP',
        'molecular_function': 'MF',
        'cellular_component': 'CC',
    }
    aspects = np.array([ns_to_aspect.get(go_namespaces.get(t, 'unknown'), 'UNK') for t in top_terms], dtype='<U3')

    # ---- IA weights ----
    ia_path = WORK_ROOT / 'IA.tsv'
    if not ia_path.exists():
        raise FileNotFoundError(f'Missing IA.tsv at {ia_path}')
    ia_df = pd.read_csv(ia_path, sep='\t')
    cols = [c.lower() for c in ia_df.columns]
    term_col = ia_df.columns[cols.index('term')] if 'term' in cols else ia_df.columns[0]
    if 'ia' in cols:
        ia_col = ia_df.columns[cols.index('ia')]
    elif 'information_accretion' in cols:
        ia_col = ia_df.columns[cols.index('information_accretion')]
    else:
        ia_col = ia_df.columns[1] if len(ia_df.columns) > 1 else ia_df.columns[0]
    ia_map = dict(zip(ia_df[term_col].astype(str), ia_df[ia_col].astype(np.float32)))
    weights = np.asarray([ia_map.get(t, np.float32(1.0)) for t in top_terms], dtype=np.float32)

    # ---- load Y + OOF predictions ----
    if 'Y' in globals():
        Y_full = np.asarray(globals()['Y'], dtype=np.float32)
    else:
        train_terms = pd.read_parquet(WORK_ROOT / 'parsed' / 'train_terms.parquet')
        train_ids_raw = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')['id'].astype(str)
        train_ids = train_ids_raw.str.extract(r"\|(.*?)\|", expand=False).fillna(train_ids_raw)
        train_terms_top = train_terms[train_terms['term'].isin(top_terms)]
        y_df = train_terms_top.pivot_table(index='EntryID', columns='term', aggfunc='size', fill_value=0)
        y_df = y_df.reindex(train_ids.tolist(), fill_value=0)
        y_df = y_df.reindex(columns=top_terms, fill_value=0)
        Y_full = y_df.values.astype(np.float32)
        del train_terms, train_ids_raw, train_ids, train_terms_top, y_df

    def _load_oof(name: str) -> np.ndarray | None:
        for p in [PRED_DIR / name, FEAT_DIR / name]:
            if p.exists():
                return np.load(p)
        return None

    oof_candidates = {
        'logreg': _load_oof('oof_pred_logreg.npy'),
        'gbdt': _load_oof('oof_pred_gbdt.npy'),
        'dnn': _load_oof('oof_pred_dnn.npy'),
        'knn': _load_oof('oof_pred_knn.npy'),
    }
    oof_candidates = {k: v for k, v in oof_candidates.items() if v is not None}
    if not oof_candidates:
        raise FileNotFoundError('No OOF preds found. Expected features/level1_preds/oof_pred_*.npy')

    for k, v in oof_candidates.items():
        if v.shape != Y_full.shape:
            raise RuntimeError(f'OOF shape mismatch for {k}: got {v.shape}, expected {Y_full.shape}')

    oof_mean = np.mean(np.stack(list(oof_candidates.values()), axis=0), axis=0).astype(np.float32)
    print('Threshold calibration base = mean OOF of:', sorted(oof_candidates.keys()))

    # deterministic subsample
    DIAG_N = int(globals().get('CAFA_DIAG_N', 20000)) if 'CAFA_DIAG_N' in globals() else 20000
    n = int(Y_full.shape[0])
    m = min(n, int(DIAG_N))
    idx = np.linspace(0, n - 1, num=m, dtype=np.int64)
    Y_sub = (Y_full[idx] > 0).astype(np.uint8)
    S_sub = oof_mean[idx].astype(np.float32, copy=False)

    THRS = np.linspace(0.05, 0.60, 23, dtype=np.float32)
    COL_CHUNK = 512

    def _ia_f1_for_cols(cols: np.ndarray, thr: float) -> float:
        w_tp = 0.0
        w_pred = 0.0
        w_true = 0.0
        for c0 in range(0, int(cols.shape[0]), COL_CHUNK):
            c = cols[c0 : c0 + COL_CHUNK]
            yt = Y_sub[:, c].astype(bool, copy=False)
            yp = (S_sub[:, c] >= float(thr))
            tp = (yt & yp).sum(axis=0).astype(np.float64)
            pred = yp.sum(axis=0).astype(np.float64)
            true = yt.sum(axis=0).astype(np.float64)
            w = weights[c].astype(np.float64)
            w_tp += float((w * tp).sum())
            w_pred += float((w * pred).sum())
            w_true += float((w * true).sum())
        p = (w_tp / w_pred) if w_pred > 0 else 0.0
        r = (w_tp / w_true) if w_true > 0 else 0.0
        return (2 * p * r / (p + r)) if (p + r) > 0 else 0.0

    def _best_thr(cols: np.ndarray) -> tuple[float, float]:
        best_t = float(THRS[0])
        best_s = -1.0
        for t in THRS:
            s = _ia_f1_for_cols(cols, float(t))
            if s > best_s:
                best_s = float(s)
                best_t = float(t)
        return best_t, best_s

    cols_all = np.arange(len(top_terms), dtype=np.int64)
    cols_bp = np.where(aspects == 'BP')[0].astype(np.int64)
    cols_mf = np.where(aspects == 'MF')[0].astype(np.int64)
    cols_cc = np.where(aspects == 'CC')[0].astype(np.int64)
    cols_unk = np.where(aspects == 'UNK')[0].astype(np.int64)

    thr_all, s_all = _best_thr(cols_all)
    thr_bp, s_bp = _best_thr(cols_bp)
    thr_mf, s_mf = _best_thr(cols_mf)
    thr_cc, s_cc = _best_thr(cols_cc)
    thr_unk, s_unk = _best_thr(cols_unk) if int(cols_unk.shape[0]) > 0 else (thr_all, s_all)

    thr_map = {
        'ALL': thr_all,
        'BP': thr_bp,
        'MF': thr_mf,
        'CC': thr_cc,
        'UNK': thr_unk,
    }
    thr_path.write_text(json.dumps(thr_map, indent=2), encoding='utf-8')
    thr_meta_path.write_text(
        json.dumps(
            {
                'calibration_base': 'mean_ensemble_oof',
                'models_used': sorted(oof_candidates.keys()),
                'n_rows_total': int(Y_full.shape[0]),
                'n_rows_used': int(m),
                'thr_grid': THRS.tolist(),
                'ia_f1_at_best': {'ALL': s_all, 'BP': s_bp, 'MF': s_mf, 'CC': s_cc, 'UNK': s_unk},
                'aspect_counts': {'BP': int(cols_bp.shape[0]), 'MF': int(cols_mf.shape[0]), 'CC': int(cols_cc.shape[0]), 'UNK': int(cols_unk.shape[0])},
            },
            indent=2,
        ),
        encoding='utf-8',
    )
    print('Saved aspect thresholds:', thr_map)

# Expose to later cells
ASPECT_THRESHOLDS = thr_map

In [ ]:
# CELL 14 - Solution: 5. PHASE 3: HIERARCHY-AWARE STACKING (GCN)
# 5. PHASE 3: HIERARCHY-AWARE STACKING (GCN)
# ==========================================
# NOTE: This cell is kept as a lightweight alternative stacker.
# The main stacker used by Phase 4 is implemented in the next cell.
TRAIN_STACKER = False

if TRAIN_STACKER:
    import numpy as np
    from pathlib import Path
    import torch
    import torch.nn as nn
    from sklearn.metrics import f1_score
    import json

    WORK_ROOT = Path(WORK_ROOT)

    # Per-aspect thresholds (computed from OOF)
    thr_path = WORK_ROOT / 'features' / 'aspect_thresholds.json'
    if not thr_path.exists():
        raise FileNotFoundError(f"Missing {thr_path}. Run threshold calibration (Cell 13F) first.")
    ASPECT_THRESHOLDS = json.loads(thr_path.read_text(encoding='utf-8'))
    print('Aspect thresholds:', ASPECT_THRESHOLDS)

    def _load_level1_pred(fname: str) -> np.ndarray:
        """Load Level-1 prediction arrays from either features/level1_preds or features/."""
        cand = [
            WORK_ROOT / 'features' / 'level1_preds' / fname,
            WORK_ROOT / 'features' / fname,
        ]
        for p in cand:
            if p.exists():
                return np.load(p).astype(np.float32)
        raise FileNotFoundError(f'Missing Level-1 prediction: {fname}. Run Phase 2 first.')

    print("Loading OOF Predictions for stacking...")
    oof_logreg = _load_level1_pred('oof_pred_logreg.npy')

    try:
        oof_gbdt = _load_level1_pred('oof_pred_gbdt.npy')
    except Exception:
        oof_gbdt = np.zeros_like(oof_logreg)

    try:
        oof_dnn = _load_level1_pred('oof_pred_dnn.npy')
    except Exception:
        oof_dnn = np.zeros_like(oof_logreg)

    try:
        oof_knn = _load_level1_pred('oof_pred_knn.npy')
    except Exception:
        oof_knn = np.zeros_like(oof_logreg)

    X_stack = np.hstack([oof_logreg, oof_gbdt, oof_dnn, oof_knn])
    Y_stack = Y

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    X_stack_t = torch.tensor(X_stack, dtype=torch.float32).to(device)
    Y_stack_t = torch.tensor(Y_stack, dtype=torch.float32).to(device)

    class Stacker(nn.Module):
        def __init__(self, input_dim, output_dim):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(input_dim, 2048),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(2048, output_dim),
                nn.Sigmoid(),
            )

        def forward(self, x):
            return self.net(x)

    stacker = Stacker(X_stack.shape[1], Y_stack.shape[1]).to(device)
    optimizer = torch.optim.Adam(stacker.parameters(), lr=1e-3)
    criterion = nn.BCELoss()

    # Build aspect vector aligned to the 13,500 term contract
    terms_path = WORK_ROOT / 'features' / 'top_terms_13500.json'
    if not terms_path.exists():
        raise FileNotFoundError(f'Missing {terms_path}. Run Phase 2 first.')
    top_terms = json.loads(terms_path.read_text(encoding='utf-8'))
    if 'go_namespaces' not in globals():
        raise RuntimeError('Missing go_namespaces mapping; run Phase 2 setup first.')
    ns_to_aspect = {'molecular_function': 'MF', 'biological_process': 'BP', 'cellular_component': 'CC'}
    term_aspects = np.array([ns_to_aspect.get(go_namespaces.get(t, 'unknown'), 'UNK') for t in top_terms], dtype='<U3')
    thr_vec = np.array([float(ASPECT_THRESHOLDS.get(a, ASPECT_THRESHOLDS.get('ALL', 0.3))) for a in term_aspects], dtype=np.float32)

    stacker.train()
    for epoch in range(20):
        optimizer.zero_grad()
        out = stacker(X_stack_t)
        loss = criterion(out, Y_stack_t)
        loss.backward()
        optimizer.step()
        if epoch % 5 == 0:
            with torch.no_grad():
                out_np = out.detach().cpu().numpy()
                preds = (out_np > thr_vec[None, :]).astype(np.int8)
                f1 = f1_score(Y_stack[:1000], preds[:1000], average='micro')
            print(f"Epoch {epoch}: Loss {loss.item():.4f}, Approx micro-F1@per-aspect-thr {f1:.4f}")

    torch.save(stacker.state_dict(), WORK_ROOT / 'features' / 'final_stacker.pth')
    print("Saved: final_stacker.pth")

    stacker.eval()
    with torch.no_grad():
        final_preds = stacker(X_stack_t).cpu().numpy()
        final_bin = (final_preds > thr_vec[None, :]).astype(int)
        final_f1 = f1_score(Y_stack, final_bin, average='micro')
    print(f"Final Stacker micro-F1@per-aspect-thr: {final_f1:.4f}")
else:
    print("Skipping Phase 3 (lightweight stacker).")

In [ ]:
# CELL 15 - Solution: 5b. PHASE 3: HIERARCHY-AWARE STACKING (GRAPH SMOOTHING GCN)
# 5b. PHASE 3: HIERARCHY-AWARE STACKING (GRAPH SMOOTHING GCN)
# =========================================================
# Critical correction: GCN input features must be the *raw* Level-1 OOF predictions per model,
# not an average. We therefore treat per-term base-model probabilities as per-node feature channels.
# External priors (IEA) remain injected conservatively at weight=0.25 (compliant).

# Option B strictness: if PROCESS_EXTERNAL=True, we REQUIRE the propagated prior files to exist.
PROCESS_EXTERNAL = globals().get('PROCESS_EXTERNAL', True)

import numpy as np
import pandas as pd
import json
from pathlib import Path
from sklearn.metrics import f1_score
import torch
import torch.nn as nn

WORK_ROOT = Path(WORK_ROOT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Load per-aspect thresholds (computed from OOF)
thr_path = WORK_ROOT / 'features' / 'aspect_thresholds.json'
if not thr_path.exists():
    raise FileNotFoundError(
        f"Missing {thr_path}. Run the threshold calibration cell after Level-1 models (Cell 13F) first."
    )
ASPECT_THRESHOLDS = json.loads(thr_path.read_text(encoding='utf-8'))
print('Aspect thresholds:', ASPECT_THRESHOLDS)

# Load the persisted term contract (must match Phase 2)
terms_path = WORK_ROOT / 'features' / 'top_terms_13500.json'
if not terms_path.exists():
    raise FileNotFoundError(f'Missing {terms_path}. Run Phase 2 target construction first.')
top_terms = [str(t) for t in json.loads(terms_path.read_text(encoding='utf-8'))]
TOP_K = int(len(top_terms))
print(f'TOP_K (term contract) = {TOP_K}')
if TOP_K != 13500:
    raise RuntimeError(f'Expected TOP_K=13500, got {TOP_K}')

def _load_level1_pred(fname: str):
    """Load Level-1 prediction arrays from either features/level1_preds or features/."""
    cand = [
        WORK_ROOT / 'features' / 'level1_preds' / fname,
        WORK_ROOT / 'features' / fname,
    ]
    for p in cand:
        if p.exists():
            return np.load(p, mmap_mode='r')
    return None

# 1. Load Level-1 predictions (keep as memmaps; batch-slice later for RAM safety)
MODEL_FILES = [
    ('logreg', 'oof_pred_logreg.npy', 'test_pred_logreg.npy'),
    ('gbdt', 'oof_pred_gbdt.npy', 'test_pred_gbdt.npy'),
    ('dnn', 'oof_pred_dnn.npy', 'test_pred_dnn.npy'),
    ('knn', 'oof_pred_knn.npy', 'test_pred_knn.npy'),
 ]

oof_by_model = {}
test_by_model = {}
for name, oof_f, te_f in MODEL_FILES:
    oof = _load_level1_pred(oof_f)
    te = _load_level1_pred(te_f)
    if oof is None or te is None:
        continue
    oof_by_model[name] = oof
    test_by_model[name] = te

if not oof_by_model:
    raise FileNotFoundError('No Level-1 OOF predictions found. Run Phase 2 first.')
models_used = sorted(oof_by_model.keys())
print('GCN input channels (raw OOF features):', models_used)
n_channels = int(len(models_used))

# 2. Build Y label matrix for the persisted term contract (prefer existing Y if present)
if 'Y' in globals():
    Y_full = np.asarray(globals()['Y'], dtype=np.float32)
    if Y_full.ndim != 2 or Y_full.shape[1] != TOP_K:
        raise RuntimeError(f'Global Y has wrong shape: {Y_full.shape}, expected (_, {TOP_K})')
    print('Using in-memory Y from Phase 2 setup:', Y_full.shape)
    # also ensure train_ids for priors alignment
    train_ids_raw = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')['id'].astype(str)
    train_ids = train_ids_raw.str.extract(r'\|(.*?)\|', expand=False).fillna(train_ids_raw)
else:
    train_terms = pd.read_parquet(WORK_ROOT / 'parsed' / 'train_terms.parquet')
    train_ids_raw = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')['id'].astype(str)
    train_ids = train_ids_raw.str.extract(r'\|(.*?)\|', expand=False).fillna(train_ids_raw)
    train_terms_top = train_terms[train_terms['term'].isin(top_terms)]
    Y_df = train_terms_top.pivot_table(index='EntryID', columns='term', aggfunc='size', fill_value=0)
    Y_df = Y_df.reindex(train_ids, fill_value=0)
    Y_df = Y_df.reindex(columns=top_terms, fill_value=0)
    Y_full = Y_df.values.astype(np.float32)
    print('Targets:', Y_full.shape)

n_train = int(Y_full.shape[0])
for m in models_used:
    arr = oof_by_model[m]
    if int(arr.shape[0]) != n_train or int(arr.shape[1]) != TOP_K:
        raise RuntimeError(f'OOF shape mismatch for {m}: got {arr.shape}, expected ({n_train}, {TOP_K})')

# 2b. External priors (Phase 1 Step 4 outputs) -> inject as *conservative* extra signal
EXTERNAL_PRIOR_WEIGHT = 0.25
ext_dir = WORK_ROOT / 'external'
prior_train_path = ext_dir / 'prop_train_no_kaggle.tsv.gz'
prior_test_path = ext_dir / 'prop_test_no_kaggle.tsv.gz'
prior_train_np = None
prior_test_np = None

if PROCESS_EXTERNAL:
    if not (prior_train_path.exists() and prior_test_path.exists()):
        raise FileNotFoundError(
            f'Option B requires external priors, but missing: {prior_train_path} or {prior_test_path}. '
            'Run Phase 1 Step 4 propagation or ensure your checkpoint dataset contains these files (run setup: STORE.pull()).'
        )
    prior_train = pd.read_csv(prior_train_path, sep='\t')
    prior_train = prior_train[prior_train['term'].isin(top_terms)]
    prior_mat = prior_train.pivot_table(index='EntryID', columns='term', values='score', aggfunc='max', fill_value=0.0)
    prior_mat = prior_mat.reindex(train_ids.tolist(), fill_value=0.0)
    prior_mat = prior_mat.reindex(columns=top_terms, fill_value=0.0)
    prior_train_np = prior_mat.values.astype(np.float32)
    print(f'Loaded external IEA train prior: {prior_train_np.shape} (weight={EXTERNAL_PRIOR_WEIGHT})')

    test_ids = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')['id'].astype(str)
    prior_test = pd.read_csv(prior_test_path, sep='\t')
    prior_test = prior_test[prior_test['term'].isin(top_terms)]
    prior_t = prior_test.pivot_table(index='EntryID', columns='term', values='score', aggfunc='max', fill_value=0.0)
    prior_t = prior_t.reindex(test_ids.tolist(), fill_value=0.0)
    prior_t = prior_t.reindex(columns=top_terms, fill_value=0.0)
    prior_test_np = prior_t.values.astype(np.float32)
    print(f'Loaded external IEA test prior: {prior_test_np.shape} (weight={EXTERNAL_PRIOR_WEIGHT})')

# 3. Graph adjacency from go-basic.obo (reload if needed)
if 'go_parents' not in locals() or 'go_namespaces' not in locals():
    print('Reloading GO graph (parse_obo)...')
    def parse_obo(path: Path):
        parents = {}
        namespaces = {}
        cur_id, cur_ns = None, None
        with path.open('r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line == '[Term]':
                    if cur_id and cur_ns:
                        namespaces[cur_id] = cur_ns
                    cur_id, cur_ns = None, None
                elif line.startswith('id: GO:'):
                    cur_id = line.split('id: ', 1)[1]
                elif line.startswith('namespace:'):
                    cur_ns = line.split('namespace: ', 1)[1]
                elif line.startswith('is_a:') and cur_id:
                    parent = line.split('is_a: ', 1)[1].split(' ! ')[0]
                    parents.setdefault(cur_id, set()).add(parent)
            if cur_id and cur_ns:
                namespaces[cur_id] = cur_ns
        return parents, namespaces
    go_parents, go_namespaces = parse_obo(PATH_GO_OBO)

def build_adjacency(terms_list, parents_dict):
    term_to_idx = {t: i for i, t in enumerate(terms_list)}
    n_terms = len(terms_list)
    src, dst = [], []
    for child in terms_list:
        parents = parents_dict.get(child, set())
        if not parents:
            continue
        child_idx = term_to_idx[child]
        for parent in parents:
            if parent in term_to_idx:
                parent_idx = term_to_idx[parent]
                # undirected edges (sym)
                src.append(child_idx)
                dst.append(parent_idx)
                src.append(parent_idx)
                dst.append(child_idx)
    # self loops
    src.extend(range(n_terms))
    dst.extend(range(n_terms))
    indices = torch.tensor([src, dst], dtype=torch.long, device=device)
    values = torch.ones(len(src), dtype=torch.float32, device=device)
    return torch.sparse_coo_tensor(indices, values, (n_terms, n_terms)).coalesce()

# 4. Ontology split (BP/MF/CC)
ns_to_aspect = {'molecular_function': 'MF', 'biological_process': 'BP', 'cellular_component': 'CC'}
aspects = [ns_to_aspect.get(go_namespaces.get(t, 'unknown'), 'UNK') for t in top_terms]
bp_idx = [i for i, a in enumerate(aspects) if a == 'BP']
mf_idx = [i for i, a in enumerate(aspects) if a == 'MF']
cc_idx = [i for i, a in enumerate(aspects) if a == 'CC']
print(f'Aspect split (in contract): BP={len(bp_idx)} MF={len(mf_idx)} CC={len(cc_idx)} UNK={aspects.count("UNK")}')

class ChannelGCN(nn.Module):
    """Graph smoothing over GO terms with per-node multi-channel features."""
    def __init__(self, n_channels: int, hidden_dim: int, adj_matrix: torch.Tensor):
        super().__init__()
        self.adj = adj_matrix
        self.fc1 = nn.Linear(int(n_channels), int(hidden_dim))
        self.fc2 = nn.Linear(int(hidden_dim), 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.30)
    def forward(self, x):
        # x: (B, N, C) -> logits: (B, N)
        h = self.relu(self.fc1(x))
        h = self.dropout(h)
        logits = self.fc2(h).squeeze(-1)
        # smooth across GO graph on the node axis (N)
        logits = torch.sparse.mm(self.adj, logits.t()).t()
        return torch.sigmoid(logits)

def _batch_stack(arrs: list, rows: np.ndarray, cols: list[int], prior_np: np.ndarray | None) -> torch.Tensor:
    # returns (B, N, C) on device
    # Slice per model to (B, N) and stack to (B, N, C) without materialising full tensors for all terms.
    feats = []
    for a in arrs:
        x = np.asarray(a[rows][:, cols], dtype=np.float32)
        feats.append(x)
    x = np.stack(feats, axis=2)  # (B, N, C)
    if prior_np is not None:
        p = np.asarray(prior_np[rows][:, cols], dtype=np.float32)
        x = np.maximum(x, (EXTERNAL_PRIOR_WEIGHT * p)[:, :, None])
    return torch.from_numpy(x).to(device=device, dtype=torch.float32, non_blocking=True)

def _batch_y(Y_np: np.ndarray, rows: np.ndarray, cols: list[int]) -> torch.Tensor:
    y = np.asarray(Y_np[rows][:, cols], dtype=np.float32)
    return torch.from_numpy(y).to(device=device, dtype=torch.float32, non_blocking=True)

def train_one(aspect_name: str, idx_cols: list[int]):
    if not idx_cols:
        print(f'[{aspect_name}] No terms; skipping.')
        return None

    terms_sub = [top_terms[i] for i in idx_cols]
    adj = build_adjacency(terms_sub, go_parents)

    # Model sees C channels per node (raw OOF preds from each base model)
    model = ChannelGCN(n_channels=n_channels, hidden_dim=128, adj_matrix=adj).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCELoss()

    n_samples = int(n_train)
    BS = 128
    EPOCHS = 5
    thr = float(ASPECT_THRESHOLDS.get(aspect_name, ASPECT_THRESHOLDS.get('ALL', 0.3)))

    # Build deterministic eval subset indices (kept small)
    eval_n = min(2000, n_samples)
    eval_rows = np.arange(eval_n, dtype=np.int64)

    model.train()
    print(f'\n=== Training GCN[{aspect_name}] nodes={len(idx_cols)} channels={n_channels} thr={thr:.3f} ===')
    for epoch in range(EPOCHS):
        total_loss = 0.0
        perm = np.random.RandomState(42 + epoch).permutation(n_samples)
        for i in range(0, n_samples, BS):
            rows = perm[i:i + BS]
            optimizer.zero_grad(set_to_none=True)
            xb = _batch_stack([oof_by_model[m] for m in models_used], rows, idx_cols, prior_train_np)
            yb = _batch_y(Y_full, rows, idx_cols)
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            total_loss += float(loss.item())

        # quick eval micro-F1 (subset)
        model.eval()
        with torch.no_grad():
            xb = _batch_stack([oof_by_model[m] for m in models_used], eval_rows, idx_cols, prior_train_np)
            yb = _batch_y(Y_full, eval_rows, idx_cols)
            pred = (model(xb) > thr).float().cpu().numpy()
            f1 = f1_score(yb.cpu().numpy(), pred, average='micro')
        model.train()
        print(f'Epoch {epoch+1}/{EPOCHS} Loss={total_loss:.4f} micro-F1@thr({thr:.2f})={f1:.4f}')

    return model

# 5. Write test_pred_gcn.npy (required by Phase 4)
out_path = WORK_ROOT / 'features' / 'test_pred_gcn.npy'
if out_path.exists():
    print(f'GCN stacker output already exists; skipping: {out_path}')
else:
    # Baseline for non-modelled cols = mean ensemble across available Level-1 test preds (+ prior floor if enabled)
    print('\nPreparing baseline test prediction (mean ensemble)...')
    test_n = int(next(iter(test_by_model.values())).shape[0])
    for m in models_used:
        arr = test_by_model[m]
        if int(arr.shape[0]) != test_n or int(arr.shape[1]) != TOP_K:
            raise RuntimeError(f'Test pred shape mismatch for {m}: got {arr.shape}, expected ({test_n}, {TOP_K})')
    base = np.zeros((test_n, TOP_K), dtype=np.float32)
    for m in models_used:
        base += np.asarray(test_by_model[m], dtype=np.float32)
    base /= float(n_channels)
    if prior_test_np is not None:
        base = np.maximum(base, EXTERNAL_PRIOR_WEIGHT * prior_test_np)
    final_test = base.astype(np.float32, copy=False)

    def _predict_aspect(model: nn.Module, idx_cols: list[int], bs: int = 128) -> np.ndarray:
        model.eval()
        outs = []
        for i in range(0, test_n, bs):
            rows = np.arange(i, min(i + bs, test_n), dtype=np.int64)
            xb = _batch_stack([test_by_model[m] for m in models_used], rows, idx_cols, prior_test_np)
            with torch.no_grad():
                out = model(xb).detach().float().cpu().numpy()
            outs.append(out)
        return np.vstack(outs).astype(np.float32)

    for aspect_name, idx_cols in [('MF', mf_idx), ('BP', bp_idx), ('CC', cc_idx)]:
        if not idx_cols:
            continue
        model = train_one(aspect_name, idx_cols)
        if model is None:
            continue
        pred_te = _predict_aspect(model, idx_cols, bs=64)
        if pred_te.shape != (test_n, len(idx_cols)):
            raise RuntimeError(f'GCN[{aspect_name}] shape mismatch: got {pred_te.shape} expected ({test_n}, {len(idx_cols)})')
        final_test[:, idx_cols] = pred_te

    np.save(out_path, final_test.astype(np.float32, copy=False))
    print('Saved:', out_path)
    if 'STORE' in globals() and STORE is not None:
        try:
            STORE.maybe_push('stage_08_stacker_gcn', [out_path, terms_path], note='GCN graph-smoothing stacker (test preds)')
        except Exception as e:
            print('WARN: failed to push stage_08_stacker_gcn:', repr(e))

In [ ]:
# CELL 16 - Solution: 6. PHASE 4: POST-PROCESSING & SUBMISSION
# 6. PHASE 4: POST-PROCESSING & SUBMISSION
# ========================================
# HARDWARE: CPU / GPU
# ========================================
# This phase applies the "Strict Post-Processing" rules (Max/Min Propagation)
# and generates the final submission file.
import json
from pathlib import Path
import numpy as np
import pandas as pd

# Check if submission already exists
if (WORK_ROOT / 'submission.tsv').exists():
    print("submission.tsv already exists. Skipping Phase 4.")
else:
    print("Starting Phase 4: Post-processing & submission...")

    WORK_ROOT = Path(WORK_ROOT)

    # Ensure go_parents is available (from Phase 1)
    if 'go_parents' not in locals() or 'go_namespaces' not in locals():
        print("Reloading GO graph (parse_obo)...")
        def parse_obo(path: Path):
            parents = {}
            namespaces = {}
            cur_id, cur_ns = None, None
            with path.open('r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if line == '[Term]':
                        if cur_id and cur_ns:
                            namespaces[cur_id] = cur_ns
                        cur_id, cur_ns = None, None
                    elif line.startswith('id: GO:'):
                        cur_id = line.split('id: ', 1)[1]
                    elif line.startswith('namespace:'):
                        cur_ns = line.split('namespace: ', 1)[1]
                    elif line.startswith('is_a:') and cur_id:
                        parent = line.split('is_a: ', 1)[1].split(' ! ')[0]
                        parents.setdefault(cur_id, set()).add(parent)
                if cur_id and cur_ns:
                    namespaces[cur_id] = cur_ns
            return parents, namespaces
        go_parents, go_namespaces = parse_obo(PATH_GO_OBO)

    # Load per-aspect thresholds (computed from OOF)
    thr_path = WORK_ROOT / 'features' / 'aspect_thresholds.json'
    if not thr_path.exists():
        raise FileNotFoundError(
            f"Missing {thr_path}. Run the threshold calibration cell after Level-1 models (Cell 13F) first."
        )
    ASPECT_THRESHOLDS = json.loads(thr_path.read_text(encoding='utf-8'))
    print('Aspect thresholds:', ASPECT_THRESHOLDS)

    # Load test IDs
    test_ids = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')['id']

    # Load stacker predictions
    pred_path = WORK_ROOT / 'features' / 'test_pred_gcn.npy'
    if not pred_path.exists():
        raise FileNotFoundError("Missing `test_pred_gcn.npy`. Run Phase 3 (GCN stacker) first.")
    preds = np.load(pred_path).astype(np.float32, copy=False)

    # Load term list (must match Phase 2/3)
    terms_path = WORK_ROOT / 'features' / 'top_terms_13500.json'
    if not terms_path.exists():
        raise FileNotFoundError(f"Missing {terms_path}. Re-run Phase 2 target construction.")
    with open(terms_path, 'r', encoding='utf-8') as f:
        top_terms = [str(t) for t in json.load(f)]
    if preds.shape[1] != len(top_terms):
        raise ValueError(f"Shape mismatch: preds has {preds.shape[1]} terms, top_terms has {len(top_terms)}.")

    # ------------------------------------------
    # Strict post-processing (Max/Min Propagation)
    # ------------------------------------------
    print(f"Applying hierarchy rules on {len(top_terms)} terms...")
    df_pred = pd.DataFrame(preds, columns=top_terms)
    term_set = set(top_terms)
    term_to_parents = {}
    term_to_children = {}
    for term in top_terms:
        parents = go_parents.get(term, set())
        if not parents:
            continue
        parents = parents.intersection(term_set)
        if not parents:
            continue
        term_to_parents[term] = list(parents)
        for p in parents:
            term_to_children.setdefault(p, []).append(term)

    # Max Propagation (Child -> Parent)
    # Previous code used range(2), which is not enough for GO depth. Use >=10 iterations.
    N_PROP_ITERS = 12
    for _ in range(N_PROP_ITERS):
        for child, parents in term_to_parents.items():
            child_scores = df_pred[child].values
            for parent in parents:
                df_pred[parent] = np.maximum(df_pred[parent].values, child_scores)

    # Min Propagation (Parent -> Child)
    for _ in range(N_PROP_ITERS):
        for parent, children in term_to_children.items():
            parent_scores = df_pred[parent].values
            for child in children:
                df_pred[child] = np.minimum(df_pred[child].values, parent_scores)

    # ------------------------------------------
    # Per-aspect thresholding BEFORE melt/prune
    # ------------------------------------------
    ns_to_aspect = {
        'molecular_function': 'MF',
        'biological_process': 'BP',
        'cellular_component': 'CC',
    }
    aspects = np.array([ns_to_aspect.get(go_namespaces.get(t, 'unknown'), 'UNK') for t in top_terms], dtype='<U3')
    thr_vec = np.array([
        float(ASPECT_THRESHOLDS.get(a, ASPECT_THRESHOLDS.get('ALL', 0.0))) for a in aspects
    ], dtype=np.float32)
    pred_np = df_pred.values.astype(np.float32, copy=False)
    pred_np = np.where(pred_np >= thr_vec[None, :], pred_np, 0.0).astype(np.float32, copy=False)
    df_pred = pd.DataFrame(pred_np, columns=top_terms)

    # ------------------------------------------
    # Submission formatting (CAFA rules)
    # - tab-separated, no header
    # - score in (0, 1.000]
    # - up to 3 significant figures
    # - <= 1500 terms per target (MF/BP/CC combined)
    # ------------------------------------------
    df_pred['EntryID'] = test_ids.values
    submission = df_pred.melt(id_vars='EntryID', var_name='term', value_name='score')
    # Enforce score range + remove zeros
    submission['score'] = submission['score'].clip(lower=0.0, upper=1.0)
    submission = submission[submission['score'] > 0.0]
    # Keep top 1500 per protein (rule)
    submission = submission.sort_values(['EntryID', 'score'], ascending=[True, False])
    submission = submission.groupby('EntryID', sort=False).head(1500)
    # Write with <= 3 significant figures
    submission.to_csv(
        WORK_ROOT / 'submission.tsv',
        sep='\t',
        index=False,
        header=False,
        float_format='%.3g',
    )
    print(f"Done! Submission saved to {WORK_ROOT / 'submission.tsv'}")
    if 'STORE' in globals() and STORE is not None:
        STORE.maybe_push('stage_09_submission', [WORK_ROOT / 'submission.tsv'], note='final submission')

In [ ]:
# CELL 17 - Solution: 7. PHASE 5: FREE TEXT PREDICTION (EXTRA)
# 7. PHASE 5: FREE TEXT PREDICTION (EXTRA)
# ==========================================
# HARDWARE: CPU
# ==========================================
# Official CAFA constraints (summary):
# - Combined file (GO + Text) allowed
# - Text: up to 5 lines per protein; ASCII printable; no tabs; <=3000 chars per protein total
# - Scores should be in (0, 1.000] and up to 3 significant figures
if (WORK_ROOT / 'submission_with_text.tsv').exists():
    print("submission_with_text.tsv already exists. Skipping Phase 5.")
elif not (WORK_ROOT / 'submission.tsv').exists():
    print("submission.tsv not found. Please run Phase 4 first.")
else:
    print("Starting Phase 5: Text Generation...")
    # 1. Load Submission & GO Graph
    print("Loading submission and GO data...")
    submission = pd.read_csv(
        WORK_ROOT / 'submission.tsv',
        sep='\t',
        header=None,
        names=['EntryID', 'term', 'score'],
    )
    if 'graph' not in locals():
        import obonet
        graph = obonet.read_obo(PATH_GO_OBO)
    # 2. Generate Text Descriptions
    print("Generating descriptions...")
    # Pre-fetch term names to avoid graph lookups in loop
    term_names = {node: data.get('name', 'unknown function') for node, data in graph.nodes(data=True)}
    text_rows = []
    unique_ids = submission['EntryID'].unique()
    for protein_id in tqdm(unique_ids, desc="Generating Text"):
        prot_preds = submission[submission['EntryID'] == protein_id]
        top_go = prot_preds.sort_values('score', ascending=False).head(3)
        if top_go.empty:
            # If we have no GO lines for this protein, skip text (keeps score>0 rule clean)
            continue
        term_descs = []
        for _, row in top_go.iterrows():
            term_id = row['term']
            term_descs.append(term_names.get(term_id, term_id))
        joined_terms = ", ".join(term_descs)
        description = f"{protein_id} is predicted to be involved in: {joined_terms}."
        # Ensure no tabs in description
        description = description.replace('\t', ' ')
        # Score: strictly > 0 and <= 1
        score = float(top_go.iloc[0]['score'])
        score = min(max(score, 0.001), 1.0)
        # One line per protein (<=5 allowed)
        text_rows.append({
            'EntryID': protein_id,
            'term': 'Text',
            'score': score,
            'description': description,
        })
    df_text = pd.DataFrame(text_rows)
    print("Saving combined submission...")
    with open(WORK_ROOT / 'submission_with_text.tsv', 'w', encoding='utf-8') as f:
        # 1) GO preds (3 cols) already CAFA-formatted in Phase 4
        submission.to_csv(f, sep='\t', index=False, header=False, float_format='%.3g')
        # 2) Text preds (4 cols)
        for _, row in df_text.iterrows():
            # Up to 3 significant figures
            score_str = format(float(row['score']), '.3g')
            f.write(f"{row['EntryID']}\tText\t{score_str}\t{row['description']}\n")
    print(f"Done! Combined submission saved to {WORK_ROOT / 'submission_with_text.tsv'}")

In [ ]:
# CELL 18 - Diagnostics: what actually contributes (OOF ablations; read-only)
# Goal: identify which Level-1 predictors help/hurt an OOF mean-ensemble under IA-F1.
# This cell ONLY reads artefacts and plots; it does not affect training or saved outputs.
import os
from pathlib import Path
import numpy as np
import pandas as pd

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    plt.rcParams.update({'font.size': 12})
except Exception as e:
    plt = None
    sns = None
    print('Plotting libs not available; will print tables only:', repr(e))
WORK_ROOT = Path(WORK_ROOT)
DIAG_N = int(os.getenv('CAFA_DIAG_N', '20000'))
THRS = np.linspace(0.05, 0.60, 23)

def _load_pred(stem: str) -> np.ndarray | None:
    # Prefer level1_preds folder, fall back to legacy root folder
    cands = [
        WORK_ROOT / 'features' / 'level1_preds' / f'{stem}.npy',
        WORK_ROOT / 'features' / f'{stem}.npy',
    ]
    for p in cands:
        if p.exists():
            return np.load(p)
    return None

def _load_targets_topk() -> tuple[np.ndarray, list[str]]:
    train_terms_path = WORK_ROOT / 'parsed' / 'train_terms.parquet'
    train_seq_path = WORK_ROOT / 'parsed' / 'train_seq.feather'
    top_terms_path = WORK_ROOT / 'features' / 'top_terms_13500.json'
    if not train_terms_path.exists() or not train_seq_path.exists():
        raise FileNotFoundError('Missing parsed targets; expected parsed/train_terms.parquet and parsed/train_seq.feather')
    train_terms = pd.read_parquet(train_terms_path)
    train_ids_raw = pd.read_feather(train_seq_path)['id'].astype(str)
    train_ids = train_ids_raw.str.extract(r'\|(.*?)\|', expand=False).fillna(train_ids_raw).tolist()
    if top_terms_path.exists():
        import json
        top_terms = json.loads(top_terms_path.read_text())
    else:
        # Fallback: recompute top terms from training labels
        top_terms = train_terms['term'].value_counts().head(13500).index.tolist()
    train_terms_top = train_terms[train_terms['term'].isin(top_terms)]
    y_df = train_terms_top.pivot_table(index='EntryID', columns='term', aggfunc='size', fill_value=0)
    y_df = y_df.reindex(train_ids, fill_value=0)
    y_df = y_df.reindex(columns=top_terms, fill_value=0)
    y = y_df.values.astype(np.float32)
    return y, top_terms

def _load_ia_weights(top_terms: list[str]) -> np.ndarray:
    # IA.tsv is required under DATASET_ROOT; still guard for robustness
    ia_path = WORK_ROOT / 'IA.tsv'
    if not ia_path.exists():
        return np.ones((len(top_terms),), dtype=np.float32)
    ia_df = pd.read_csv(ia_path, sep='\t', names=['term', 'ia'])
    ia_map = dict(zip(ia_df['term'], ia_df['ia']))
    w = np.array([float(ia_map.get(t, 0.0) or 0.0) for t in top_terms], dtype=np.float32)
    # Avoid all-zeros (would collapse the metric)
    if float(w.sum()) <= 0:
        w = np.ones_like(w)
    return w

def _subsample_rows(y_true: np.ndarray, *ys: np.ndarray) -> tuple[np.ndarray, list[np.ndarray]]:
    n = int(y_true.shape[0])
    m = min(n, int(DIAG_N))
    if m <= 0:
        return y_true[:0], [a[:0] for a in ys]
    idx = np.linspace(0, n - 1, num=m, dtype=np.int64)
    out = [np.asarray(a[idx], dtype=np.float32) for a in ys]
    return np.asarray(y_true[idx], dtype=np.float32), out

def _ia_f1(y_true: np.ndarray, y_score: np.ndarray, weights: np.ndarray, thr: float) -> float:
    # IA-weighted F1 over ALL terms (no aspect split here; goal is model contribution ranking)
    y_true = (y_true > 0).astype(np.int8)
    y_pred = (y_score >= float(thr)).astype(np.int8)
    tp = (y_pred & y_true).sum(axis=0).astype(np.float64)
    pred = y_pred.sum(axis=0).astype(np.float64)
    true = y_true.sum(axis=0).astype(np.float64)
    w = weights.astype(np.float64)
    w_tp = float((w * tp).sum())
    w_pred = float((w * pred).sum())
    w_true = float((w * true).sum())
    p = (w_tp / w_pred) if w_pred > 0 else 0.0
    r = (w_tp / w_true) if w_true > 0 else 0.0
    return (2 * p * r / (p + r)) if (p + r) > 0 else 0.0

def _best_over_thrs(y_true: np.ndarray, y_score: np.ndarray, weights: np.ndarray) -> tuple[float, float, np.ndarray]:
    scores = np.array([_ia_f1(y_true, y_score, weights, thr=float(t)) for t in THRS], dtype=np.float32)
    best_i = int(np.argmax(scores))
    return float(THRS[best_i]), float(scores[best_i]), scores
# ----------------------------
# Load Y + predictions
# ----------------------------
try:
    Y_full, top_terms = _load_targets_topk()
    w = _load_ia_weights(top_terms)
except Exception as e:
    print('Cannot run contributor analysis (missing targets/IA):', repr(e))
    raise
models = {
    'logreg': _load_pred('oof_pred_logreg'),
    'gbdt': _load_pred('oof_pred_gbdt'),
    'dnn': _load_pred('oof_pred_dnn'),
    'knn': _load_pred('oof_pred_knn'),
}
models = {k: v for k, v in models.items() if v is not None}
if not models:
    print('No OOF prediction matrices found. Expected files like features/level1_preds/oof_pred_logreg.npy')
    raise SystemExit(0)
# Sanity: shape alignment
for k, v in models.items():
    if v.shape != Y_full.shape:
        raise ValueError(f'Model {k} shape mismatch: got {v.shape}, expected {Y_full.shape}')
# Baseline = mean ensemble
stack = np.mean(np.stack(list(models.values()), axis=0), axis=0).astype(np.float32)
Y_sub, [stack_sub] = _subsample_rows(Y_full, stack)
thr0, s0, curve0 = _best_over_thrs(Y_sub, stack_sub, w)
print(f'Baseline mean-ensemble best IA-F1={s0:.4f} @ thr={thr0:.3f} (sampled N={len(Y_sub)})')
# Ablations
rows = []
keys = list(models.keys())
for drop in keys:
    keep = [k for k in keys if k != drop]
    st = np.mean(np.stack([models[k] for k in keep], axis=0), axis=0).astype(np.float32)
    Y_sub, [st_sub] = _subsample_rows(Y_full, st)
    thr, s, _ = _best_over_thrs(Y_sub, st_sub, w)
    rows.append({'drop': drop, 'best_ia_f1': s, 'best_thr': thr, 'delta_vs_base': s - s0})
ab = pd.DataFrame(rows).sort_values('delta_vs_base', ascending=True)
print('\nAblation deltas (negative = dropping hurts):')
print(ab)
if plt is not None:
    plt.figure(figsize=(10, 4))
    plt.barh(ab['drop'], ab['delta_vs_base'])
    plt.axvline(0.0, color='k', linewidth=1)
    plt.title('OOF mean-ensemble: IA-F1 change when dropping one model (sampled)')
    plt.xlabel('Δ IA-F1 vs baseline')
    plt.ylabel('dropped model')
    plt.grid(True, alpha=0.3)
    plt.show()